# 1. kodutöö - klassifitseerimine ja regressioon tõenäosusliku modelleerimise abil

Importe võite lisada, ent kõikvõimalikud moodulid, mida pole seni praktikumi materjalides kasutatud ja mis vajaksid lisainstalli, tuleb Moodle'i tekstiväljas eraldi raporteerida ja piisavalt (väga põhjalikult ja veenvalt) põhjendada.

In [3]:
import sklearn
import pandas as pd
from sklearn.metrics import mean_squared_error, accuracy_score

In [4]:
#Failide lugemiseks Colabis
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/")

Mounted at /content/drive


In [5]:
faili_asukoht = "imdb.csv"
testfaili_asukoht = "imdb_dev.csv"


## 1. Loe andmed failist sisse

All on väike kontroll selle kohta, kas andmed said sisse loetud samal kujul.

In [6]:
# Loe sisse kõik treeningandmed pandas DataFrame'iks muutujasse filmid
filmid = pd.read_csv(faili_asukoht, delimiter=",",index_col=0)
# Kontrolli, kas andmed on sobival kujul
assert filmid.shape == (4538, 28)

In [ ]:
filmid.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

## 2. Filmireitingute ennustamine  

**2 punkti** saab mudeli eest, mis ennustab filmi reitingu (tabelis tulp *'content_rating'*). Tegu on klassifitseerimisülesandega

### 2.1 Töötle andmeid

**2 punkti** saab andmete eeltöötluse eest. See tähendab, et tuleb andmetele otsa vaadata, sobivad välja valida, vajadusel nende väärtusi või formaati muuta jne. Soovi korral võid siin teha väljatrükke, visualiseerimisi jms.  

Korrektse andmete eeltöötluse tulemus on muuhulgas see, et mõlemad ennustusmeetodid (*predictScore* ja *predictRating*, vt allpool) töötavad. Soovi korral võib siin teha ka nt mitu eraldi eeltöötlust, vastavalt edaspidi kasutatavale mudelile.  

**NB!** Tagastatavaid struktuure ja väärtusi võite ise valida ja muuta, oluline on see, et treening- ja testfunktsioon nendega toime tuleksid. Vaadake üle, et töölehe puhtal läbijooksutamisel midagi kaotsi ei läheks.

In [7]:
def numerate(processedDataFrame, name):
  directors=dict()
  for i in processedDataFrame[name].values:
    if i not in directors.keys():
      directors.update({i:len(directors.keys())})
  return processedDataFrame[name].map(directors)

In [ ]:
def preprocessRating(dataFrame, mode="train"):
  '''Tegeleb sisse saadud DataFrame'i töötlemisega.
  '''
  # võimalikud testandmete eeltöötluseks vajalikud mudelid-teadmised (nt keskmised väärtused, one-hot tulpade nimed vms)
  andmestikud = []
  # eeltöötluse tulemus - muudetud DataFrame
  processedDataFrame = dataFrame.drop('movie_imdb_link', axis=1)

  #color tegemine 1 ja 0ks
  processedDataFrame['color'] = processedDataFrame['color'].map({'Color': 1, 'Black and White': 0})

  #teeme numbriliseks osad columnid, selleks et onehot vectorite arv ei muutuks liiga suureks
  directors=dict()
  for i in processedDataFrame['director_name'].values:
    if i not in directors.keys():
      directors.update({i:len(directors.keys())})
  processedDataFrame['director_name'] = processedDataFrame['director_name'].map(directors)
  directors=dict()
  for i in processedDataFrame['actor_2_name'].values:
    if i not in directors.keys():
      directors.update({i:len(directors.keys())})
  processedDataFrame['actor_2_name'] = processedDataFrame['actor_2_name'].map(directors)
  directors=dict()
  for i in processedDataFrame['actor_3_name'].values:
    if i not in directors.keys():
      directors.update({i:len(directors.keys())})
  processedDataFrame['actor_3_name'] = processedDataFrame['actor_3_name'].map(directors)
  directors=dict()
  for i in processedDataFrame['actor_1_name'].values:
    if i not in directors.keys():
      directors.update({i:len(directors.keys())})
  processedDataFrame['actor_1_name'] = processedDataFrame['actor_1_name'].map(directors)
  processedDataFrame['language']=numerate(processedDataFrame,'language')
  processedDataFrame['country']=numerate(processedDataFrame,'country')
  #ratings = {'TV-Y': 0,'TV-Y7': 1,'G': 2,'TV-G': 3,'Approved': 4,'PG': 5,'TV-PG': 6,'GP': 7,'PG-13': 8,'TV-14': 9,'M': 10,'R': 11,'NC-17': 12,'X': 13,'Not Rated': 14,'Passed': 15,'Unrated': 16,'TV-MA': 17}
  #try:
    #processedDataFrame['content_rating']=processedDataFrame['content_rating'].map(ratings)
  #except:
    #print('no such column')
  #Teeme onehot vektorid filmide shanritele
  genres=set()
  genrecolumns=dict()

  for i in processedDataFrame.genres:
    for j in i.split("|"):
      genres.add(j)
  genrecolumns.update({'movie_title':list()})
  print(processedDataFrame.shape[0])
  genres=['War', 'Music', 'Comedy', 'Documentary', 'Film-Noir', 'Drama', 'Action', 'Biography', 'Game-Show', 'Sci-Fi', 'Crime', 'Sport', 'Fantasy', 'Short', 'News', 'Adventure', 'Thriller', 'Western', 'Mystery', 'History', 'Romance', 'Animation', 'Musical', 'Reality-TV', 'Horror', 'Family']
  for genre in genres:
    genrecolumns.update({genre:[]})
  for index, movie in processedDataFrame.iterrows():
    movieGenres=[0]*len(genrecolumns);
    currentGenre=""
    genrecolumns['movie_title'].append(movie['movie_title'])
    j=0
    for index2, currentGenre in enumerate(genres):
      if currentGenre in movie['genres'].split("|"):
        movieGenres.append(1)
        genrecolumns[currentGenre].append(1)
      else:
        movieGenres.append(0)
        genrecolumns[currentGenre].append(0)
      j+=1
  df2=pd.DataFrame(genrecolumns)
  processedDataFrame=processedDataFrame.merge(df2, on='movie_title', how='inner')
  processedDataFrame = processedDataFrame.drop('genres', axis=1)

  keycolumns=dict()
  #Teeme onehot vektorid filmide võtmesõnadele
  keywords=set()
  keywordsBefore=dict()
  #for i in processedDataFrame.plot_keywords:
    #try:
      #for j in i.split("|"):
        #if j in keywordsBefore.keys():
          #keywordsBefore[j]=keywordsBefore[j]+1
        #else:
          #keywordsBefore[j]=1
    #except:
      #i="None"
  #for i in keywordsBefore.keys():
    #if keywordsBefore[i]>5:
      #keywords.add(i)
  #keycolumns.update({'movie_title':list()})
  #print(len(keywords))
  keywords = ['bomb', 'bus', 'elevator', 'explosive', 'rescue', 'acorn', 'herd', 'iceberg', 'ocean', 'pirate', 'bad', 'dragon', 'grandfather', 'orb', 'warrior', 'apartment', 'bedtime story', 'eagle', 'stuttering', 'swimming pool', 'autopsy', 'lantern', 'planet', 'ring', 'test pilot', 'arranged marriage', 'bride', 'corpse', 'groom', 'woods', 'actor', 'detective sergeant', 'hotel', 'nightclub', 'thief', 'blood', 'publisher', 'publishing house', 'werewolf', 'wolf', '1990s', 'brother brother relationship', 'nantucket island', 'one word title', 'sister sister relationship', '2010s', 'ape', 'ballet', 'psychic', 'supernatural', 'artist', 'drug', 'tree sap', 'vampire', 'amusement park', 'on the road', 'zombie', 'zombie apocalypse', 'zombie spoof', 'countess', 'game reality crossover', 'ghost', 'group of friends', 'tower', 'airline', 'airplane', 'bathroom', 'sex', 'toilet', 'church', 'college', 'color in title', 'philosophy', 'texas', 'basketball', 'burglary', 'police', 'robbery', 'teenage girl', 'boss', 'holiday', 'letter', 'north pole', 'santa', 'adulterous wife', 'bipolar disorder', 'obsessive compulsive personality disorder', 'unhappy marriage', 'widow', 'dam', 'flood', 'money', 'sheriff', 'antarctica', 'geologist', 'investigation', 'u.s. marshal', 'winter', 'immigration', 'jewish', 'pogrom', 'tradition', 'tradition versus modernity', 'avant garde', 'british pop', 'character name in title', 'musician', 'number in title', 'girl', 'maori', 'new zealand', 'whale', '19th century', 'colombia', 'love', 'marriage', '1950s', 'archeologist', 'korea', 'korean war', 'soldier', 'author', 'disney', 'loss of father', 'reference to walt disney', 'suppression of emotion', 'evil', 'killer', 'murder', 'psychopath', 'serial killer', 'blue', 'new york', 'new york city', 'no opening credits', 'smurf', 'based on tv series', 'fictional band', 'girl band', 'music industry', 'pop band', 'alien', 'drug lord', 'gang', 'hunter', 'predator', 'nevada', 'pledge', 'porcupine', 'search', 'arms dealer', 'atf', 'bail', 'multiple perspectives', 'detective', 'detective novel', 'fantasy sequence', 'hallucination', 'writer', 'blockbuster', 'bumblebee the character', 'semi truck and trailer', 'truck', 'colonel', 'lieutenant', 'nuclear warhead', 'team', 'washington d.c.', 'black comedy', 'bully', 'friend', 'house', 'practical joke', 'buried alive', 'coffin', 'iraq', 'one person', 'single set production', 'athlete', 'jew', 'missionary', 'olympic games', 'race', 'apache', 'apache indian', 'confederate', 'mexico', 'scout', 'biblical plague', 'louisiana', 'plague', 'small town', 'freddy krueger', 'lucid dream', 'slasher', 'sleep deprivation', 'bloopers during credits', 'bucket of popcorn', 'chastity belt', 'guru', 'toronto maple leafs', 'dying planet', "regaining one's memory", 'sacrifice', 'stabbed to death', 'bank', 'bank clerk', 'gangster', 'loki', 'mask', 'artificial intelligence', 'box office flop', 'fighter jet', 'navy', 'pilot', 'car accident', 'japan', 'newlywed', 'photograph', 'photographer', 'butchery', 'catholic', 'catholic school', '1970s', 'afterlife', 'heaven', 'pedophile', 'rape', 'bare chested male bondage', 'european', 'female frontal nudity', 'topless woman', 'war paint', 'boston', 'mole', 'undercover', 'undercover cop', 'captain', 'expedition', 'sea', 'submarine', 'battle', 'creature', 'hyena', 'monkey', 'village', 'airport', "asperger's syndrome", 'autism', 'muslim', 'racial profiling', 'astronaut', 'mars', 'mars the planet', 'outer space', 'zero gravity', 'bangladesh', 'buckingham palace', 'september 11 2001', 'title directed by female', 'austrian', 'breasts', 'cholera', 'provence', 'destiny', 'imagination', 'sex scene', 'crown', 'dream', 'king neptune', 'plankton', 'cup', 'derby county', 'flashback', 'football', 'leeds united', 'latter day saints', 'pickup truck', 'video camera', 'blackmail', 'panties pulled down', 'prostitution', 'bar', 'bullet', 'corruption', 'lawyer', 'professor', 'research', 'sexuality', 'student', 'based on film', 'crime family', 'remake', 'southern california', 'boy', 'brother', 'mafia', 'senator', 'ambush', 'assassin', 'priest', 'prostitute', 'sweden', 'absurd humor', 'egg', 'gross out humor', 'lesbian', 'hong kong', 'missing person', 'missing son', 'best friend', 'car race', 'conspiracy', 'gadget car', 'spy', 'cardinal richelieu', 'critically bashed', 'duel', 'musketeer', 'american politics', 'iraq war', 'occupation', 'white house', 'east germany', 'secret police', 'stasi', 'bicyclist', 'doll', 'star wars reference', 'voice impersonation', 'arab', 'bare chested male', 'foreign legion', 'french foreign legion', 'morocco', 'banker', 'female protagonist', 'live action and animation', 'magic', 'nanny', 'cultural conflict', 'politics', 'president', 'slavery', 'u.s. president', 'adoption', 'divorce', 'family relationships', 'farce', 'wedding', 'camping', 'forest', 'lake', 'stabbed with glass', 'vomiting', 'campaign', 'election', 'presidential election', 'satire', 'danger', 'drug cartel', 'female cop', 'female police officer', 'fugitive', 'womanizer', 'barefoot', 'endorsement', 'surfboard', 'virtual reality', 'waves', 'death', 'doctor', 'grave', 'patient', 'venezuela', 'blood splatter', 'camera shot from inside human body', 'combat', 'die hard scenario', 'fbi', 'convict', 'karaoke', 'middle america', 'salesman', 'singer', 'new mexico', 'prison', 'smuggling', 'theft', 'hostage', 'shot in the chest', 'shot in the heart', 'outlaw', 'rancher', 'small western town', 'stagecoach', 'train', '1880s', 'cowboy', 'low budget film', 'shoot out', 'wild west', 'chase', 'clue', 'movie star', 'alligator', 'golf', 'panda', 'stunt', 'area 51', 'military', 'spacecraft', 'spaceship', 'hip hop', 'liberal', 'truth', 'game', 'showdown', 'bus driver', 'money scheme', 'poverty', 'sewer', 'lynchian', 'reality', 'police officer', 'police officer taken hostage', 'communications', 'computer', 'genius', 'job', 'athletic training', 'football practice', 'funeral', 'gymnasium', 'man crying', 'christian film', 'hymn', 'religion', 'slave', 'fuel', 'rocket', 'space travel', 'chipmunk', 'cruise ship', 'football ball', 'overboard', 'tennis ball', 'based on true story', 'cia', 'cocaine', 'journalist', 'smear campaign', 'female pubic hair', 'girl in panties', 'leg spreading', 'public nudity', 'sex in bed', 'prank gone wrong', 'scantily clad female', 'shotgun', 'sorority house', 'stabbed in the neck', 'hawaii', 'hawaiian', 'historically inaccurate', 'princess', 'tearing up a letter', '1800s', 'bird', 'blue bird of happiness', 'foreign language adaptation', 'past', 'hand job', 'hugging', 'long hair', 'two men one woman', 'alien invasion', 'iron man', 'courtroom', 'dysfunctional family', 'indiana', 'trial', 'cancer', 'survival rate', 'therapist', 'lapd', 'mexican', 'violence', 'package', 'slow motion scene', 'topless female nudity', 'wrong address', 'bloody body of child', 'dead boy', 'monster', 'afghanistan', 'death of marine', 'dog', 'dog adoption', 'man dog relationship', 'female nudity', 'male nudity', 'parody', 'reference to myspace', 'sequel', 'pakistan', 'road trip', 'adultery', 'extramarital affair', 'infidelity', 'unfaithfulness', 'american president', 'cbs', 'expose', 'fear', 'paranoia', 'reporter', 'buddy movie', 'general', 'inventor', 'steampunk', 'utah', 'pyramid', 'satellite', 'connecticut', 'witch', 'witchcraft', 'basketball player', 'boarding school', "shakespeare's othello", 'african american', 'old age', 'bear', 'grizzly', 'wilderness', 'friendship', 'frontier', 'honor', 'outpost', 'love across time', 'parallel time', 'dysfunctional marriage', 'sex offender', 'suburb', 'breaking the fourth wall', 'high school friends', 'imperative in title', 'skateboard', 'amateur detective', 'boat builder', 'stepfather', 'witness', 'fame', 'fashion', 'gay lead character', 'mockumentary', 'crying woman', 'plot', 'suspect', 'waitress', 'younger man older woman relationship', 'defense lawyer', 'marine', 'villager', 'agent', 'federal bureau of investigation', 'gay relationship', 'secret', 'emaciation', 'plane crash', 'prisoner of war', 'torture', 'beating', 'muscleman', 'tied up', 'tough guy', 'engineer', 'ethics', 'mountain', 'railroad', 'south america', 'journalism', 'news anchor', 'newscast', 'newscaster', 'tv journalism', 'abbreviation in title', 'sudden wealth', 'unknowing heir', 'music business', 'record executive', 'record label', 'black september', 'israel', 'munich olympics', 'cannibal', 'evil brother', 'giant', 'london england', 'queen', 'cadillac', 'disguise', 'secret agent', 'novelist', "writer's block", 'writing', 'engagement', 'maid', 'song', 'california', 'car', 'drag racing', 'high school', "rock 'n' roll", 'drink', 'manager', 'partner', 'pool hall', 'black hole', 'father daughter relationship', 'saving the world', 'wormhole', 'canadian armed forces', 'canadian soldier', 'directed by star', 'veteran', 'gang rape', 'hmong', 'teenager', 'widower', 'bathory', 'legend', 'virgin blood', 'curse', 'tourism', 'travel', 'england', 'prejudice', 'upper class', 'working class', 'cellular memory', 'eye', 'eye transplant', 'seeing dead people', 'violinist', 'name calling', 'penis slur', 'wyatt earp', 'count', 'escape', 'island', 'revenge', 'sailor', 'latina', 'south central los angeles', 'exploitation', 'four word title', 'hallucinogen', 'telepathy', 'explosion', 'klingon', 'moon', 'peace', 'farm', 'hannah montana', 'horseback riding', 'overalls', 'japanese soldier', 'physical torture', 'pow camp', 'reference to adolf hitler', 'baby', 'christmas spirit', 'elf', 'anti semitism', 'drugs', 'highway', 'state trooper', 'vermont', 'japanese tourist', 'secret service', 'wizard of oz', 'kid', 'oxygen', 'rugrats', 'trapped underwater', 'underwater', 'chamberlain', 'theatre', 'bangkok', 'thailand', 'digit in title', 'human form', 'number 3 in title', 'talking on a rooftop', 'terrorism', 'told in flashback', 'wearing a sound wire', 'coach', 'football player', 'misfits', 'strike', 'sumo wrestler', 'betrayal', 'final showdown', 'on the run', 'babysitting', 'cell phone', 'nightmare', 'egypt', 'pharaoh', 'miniature person', 'shrinking', 'sword and fantasy', 'temple', 'based on real person', 'female inventor', 'mop', 'sales pitch', 'woman holding a shotgun', 'analyst', 'nuclear bomb', 'nuclear weapons', 'russia', 'king', 'king george vi', 'speech', 'speech therapist', 'throne', 'birthday', 'clone', 'cloning', 'character says i have a bad feeling about this', 'hand to hand combat', 'seven word title', 'wilhelm scream', 'criminal', 'maine', 'only son', 'school choir', 'summer', 'cat', 'first part', 'lasagna', 'veterinarian', 'iran', 'islamic revolution', 'revolution', 'circus', 'magician', 'oz', 'american football', 'football star', 'high school football', 'left tackle', 'alcoholism', 'little girl', 'orphan', 'troubled marriage', 'american south', 'baseball', 'farmland', 'motel', 'diner', 'private school', 'school', 'android', 'ex cop', 'virtual character come to life', 'cartoon on tv', 'second part', 'videotape', 'education', 'factory', 'mexican american', 'religious icon', 'backup singer', 'band', 'bechdel test passed', 'concert footage', 'pushed off a cliff', 'rat', 'secret passage', "alzheimer's disease", 'book', 'golden fleece', 'half brother', 'magical tree', 'tree', 'art teacher', 'homework', 'principal', 'ballerina', 'madness', 'ice', 'ice age', 'sleeping', 'sloth', 'memory', 'short term memory', 'telephone', 'oceanographer', 'shark', 'boyfriend girlfriend relationship', 'erection', 'arab israeli conflict', 'media', 'middle east', 'palestine', 'dissociative identity disorder', 'limousine', 'multiple personality', 'rhode island', '1980s', 'c 130 hercules', 'panzer', 'san francisco california', 'hero', 'masked hero', 'texas ranger', 'commitment', 'pizza', 'title appears in writing', "woman wearing only a man's shirt", 'jason voorhees', 'massacre', 'slaughter', 'telekinesis', 'alaska', 'deportation', 'father son relationship', 'male objectification', 'sitka alaska', 'abortion', 'anti choice', 'christian preach', 'malpractice', 'unprofessional conduct', '2nd century', 'emperor', 'european history', 'loosely based on historical events', 'roman empire', 'captain nemo', 'dorian gray', 'invisible man', 'dance', 'emperor penguin', 'penguin', 'bathtub', 'eaten alive', 'ferry boat', 'lobster trap', 'sprinkler system', '1960s', 'car hitting an animal', 'folk singer', 'hold up', 'stockholm syndrome', 'swat team', 'vegetarian', 'mob', 'mobster', 'montana', 'coma', 'sword', 'cheating wife', 'film camera', 'hasselblad camera', 'leica camera', 'medium format camera', 'flip phone', 'rape threat', 'vulnerable', 'dog movie', 'golden retriever', 'atlantis', 'crew', 'journal', 'museum', 'anti war', 'interview', 'war on terrorism', '1930s', 'city', 'invisibility', 'the shadow', 'male full back nudity', 'practicing a suicide', 'suicide contemplation', 'anthology', 'heavy metal', 'secretary', 'segments', 'earth', 'giant robot', 'scientist', 'face slap', 'loss of virginity', 'pierced nipple', 'ambassador', 'cannes', 'identity', 'suicide', 'heist', 'pretending to be pregnant', 'toy', 'wish', 'fight', 'mina', 'terrorist', 'exorcism', 'sin eater', 'ufo', 'head coach', 'deputy', 'childhood', 'love triangle', 'organ donation', 'air traffic controller', 'spoof', 'embassy', 'gym', 'memoir', 'lobbyist', 'state dinner', 'celtic mythology', 'cottage', 'fisherman', 'irish', 'selkie', 'demon', 'fbi agent', 'religious fanatic', 'neighborhood watch', 'ohio', 'civil war', 'post civil war', 'reference to friedrich schiller', 'u.s. civil war', 'prisoner', 'reverend', 'terminal illness', 'two word title', 'acceptance', 'coming out to family', 'gay', 'large family', '40 year old', 'cheating on girlfriend', 'collection', 'porno movie', 'earth viewed from space', 'race against time', 'race car driver', 'title at the end', 'death of sister', 'dystopia', 'execution', 'rebellion', 'desert island', 'non statutory female on male rape', 'promotion', 'sexual harassment', 'angkor wat', 'human trafficking', 'sex slavery', 'sex tourism', 'brownstone', 'burglar', 'panic', 'panic room', 'air marshal', 'passenger', 'transatlantic flight', 'based on video game', 'fictional language', 'fictional war', 'orc', 'sword and sorcery', 'day care', 'teddy bear', 'jester', 'kansas', 'tornado', 'suspicion', 'cavalry', 'stallion', 'wild stallion', 'blindness', 'cave', 'spirit', 'wolf cub', 'human versus dinosaur', 'lizard', 'primate', 'time travel', 'tyrannosaurus rex', 'ira', 'limerick', 'freeway', 'trailer park', 'contract killer', 'murder of a girl', 'serial murderer', 'abusive husband', 'chick flick', 'north carolina', 'main characters killed off', 'agoraphobia', 'copycat murder', 'psychologist', 'hairy chest', 'social worker', 'spinning in chair', 'cunnilingus', 'drug trafficking', 'no panties', 'shootout', 'dubstep', 'tied feet', 'based on short film', 'based on short story', 'latino', 'life or death', 'attic', 'surrealism', 'yuppie', 'china', 'faith', 'panama', 'photography', 'suspense', 'dock', 'longshoreman', 'union', 'experiment', 'human versus shark', 'underwater headquarters', 'french revolution', 'old testament', 'part of an unfinished series', 'reference to john wayne', 'roman emperor', 'horse', 'horse racing', 'jockey', 'limp', 'seabiscuit', 'classified ad', 'intern', 'magazine', 'time machine', 'hindu', 'india', 'sikh', 'protection', 'russian', 'sergeant', 'animal name in title', 'ape abducts a woman', 'gorilla', 'king kong', 'bare breasts', 'bra and panties', 'nudity', 'syringe', 'alliance', 'chinese', 'strategy', 'conflicted hero', 'mission', 'rescue mission', 'political cover up', 'political satire', 'carlsbad california', 'security camera', 'fire', 'homicide', 'rubble', 'trapped', 'misunderstanding', 'obese woman', 'overweight woman', 'gay kiss', 'homoeroticism', 'lesbian kiss', 'transsexual', 'code', 'false accusation', 'megalomaniac', 'torah', 'aristocrat', 'english', 'painter', 'painting', 'horse ranch', 'ranch', 'wild mustang', 'chef', 'italy', 'restaurant', 'epic', 'greece', 'spartan', 'career', 'casino', 'judge', 'stock', 'stock broker', 'cow', 'farmer', 'howl', 'howling at the moon', 'cult film', 'flash gordon', 'mind reading', 'meaning of life', 'moral challenge', 'morality', 'search for meaning', 'ten commandments', 'atlanta georgia', 'competition', 'dancer', 'fraternity', 'continental congress', 'debate', 'declaration of independence', 'massachusetts', 'pennsylvania', 'biker', 'biker gang', 'mississippi', 'cat burglar', 'female lead', 'future', 'mixed martial arts', 'post apocalypse', 'female full frontal nudity', 'gladiator', 'group sex', 'slave girl', 'international cooperation', 'left for dead', 'nasa', 'space habitat', 'cabin in the woods', 'drinking acid', 'introvert', 'truth or dare', 'cover up', 'newspaper', 'sexual abuse', 'beach', 'party', 'resort', 'based on novel', 'imperial japan', 'japanese army', 'world war two', 'boston marathon', 'marathon', 'miracles', 'running', 'missing', 'accidental death', 'dead woman', 'dead woman with eyes open', 'death of mistress', 'cybercrime', 'hacker', 'film within a film', 'filmmaking', 'guerrilla filmmaking', 'making of a movie', 'scientology parody', 'disappearance', 'hearing voices', 'hypnosis', 'post hypnotic suggestion', 'seeing a ghost', 'africa', 'capture', 'jungle', 'tarzan', 'aphrodisiac', 'court jester', 'written and directed by cast member', 'black panther', 'hells angels', 'racist', 'racist cop', 'ballot', 'vote', 'voting', 'orgy', 'pianist', 'sexual fantasy', 'hair', 'taliban', 'aunt', 'death of pet', 'boarding house', 'great depression', 'mistress', 'schizophrenic', 'heroin', 'questioned by police', 'bounty hunter', 'necromonger', 'female star appears nude', 'pants falling down', 'samurai', 'white rat', 'woman in a towel', 'bouquet of roses', 'bright light', 'cardiopulmonary resuscitation', 'white rose', 'charles dickens', 'christmas', 'christmas eve', 'scrooge', 'alice in wonderland', 'mistaking reality for dream', 'shrinking potion', 'colorado', 'niece', 'female neighbor', 'trip', 'emir', 'map', 'sports agent', 'training', 'desert', 'tragic event', 'blackbeard', 'bigfoot', 'sasquatch', 'tourist', 'independence day', 'lady penelope', 'the hood', 'scene of the crime', 'therapy', 'alabama', 'divorce papers', 'husband wife relationship', 'john keats', 'passion', 'poet', 'domestic abuse', 'public humiliation', 'sex symbol', 'suicide attempt', 'coming of age', 'sidekick', 'superpower', 'art deco', 'bible quote', 'silent film', 'worker', 'nubile woman', 'policeman', 'brain tumor', 'psionic power', 'pity', 'readjusting to civilian life', 'world war two veteran', 'corporation', 'ninja', 'robocop', 'exercise bicycle', 'face in food', 'police brutality', 'inheritance', 'uncle', 'wine', 'boxing', 'heavyweight champion', 'iceman', 'mail carrier', 'poetry', 'postal worker', 'choking', 'hospital', 'scam', 'daughter', 'gun', 'sociopath', 'cro magnon', 'dialogue driven storyline', 'single setting', 'storytelling', 'blind date', 'crossword puzzle', 'eccentric', 'fall', 'mine shaft', 'broken leg', 'urination', 'physical abuse', 'radio flyer', 'stepfather stepson relationship', 'halloween', 'killer robot', 'wrestler costume', 'bum', 'mutant', 'street', 'body swap', 'camera shot of feet', 'foot closeup', 'reference to keith richards', 'reference to the rolling stones', 'gay best friend', 'gay man', 'gay marriage proposal', 'south africa', 'homosexual', 'homosexuality', 'hustler', 'male rear nudity', '3d in title', 'animal in title', 'dolphin', 'manatee', 'fake commercial', 'fake newscast', 'fake trailer', 'amazon', 'anaconda', 'boat', 'snake', 'breeding', 'caretaker', 'evacuation', 'orphanage', 'butler', 'civil rights', 'andy warhol', 'art', 'street artist', 'bordeaux', 'mayor', 'pamphlet', 'political thriller', 'long distance runner', 'olympics', 'oregon', 'runner', 'watching television', 'friendship between men', 'hollywood california', 'sitcom', 'female rear nudity', 'lesbian cunnilingus', 'rear entry sex', 'justice', 'nazi', 'body count', 'dictator', 'jumping from an airplane', 'knife fight', 'one man army', 'manipulation', 'simulation', 'anime', 'castle', 'masturbation', 'conspiracy theory', 'kennedy assassination', 'mock documentary', 'bank heist', 'bank robbery', 'heist gone wrong', 'pizza delivery boy', 'unlikely criminal', 'robot', 'robot as menace', 'survival', 'undead', 'roommate', 'thong panties', 'upskirt', 'story within the story', 'talking animal', 'bank manager', 'criminal gang', 'mob hit', 'opera', 'repeating history', 'atf agent', 'ball gag', 'cult leader', 'pastor', 'pretending to be dead', 'confederacy', 'prequel', 'stonewall jackson', 'war hero', '17th century', 'based on fairy tale', 'flower', 'box office hit', 'materialism', 'snowflake', 'defection', 'soviet submarine', 'boston massachusetts', 'true crime', 'falling in love', 'glowing eyes', 'insult', 'three word title', 'boredom', 'luxury hotel', 'dvd', 'girlfriend', 'loan', 'self help', 'makeover', 'matchmaker', 'popularity', 'drifter', 'hope', 'postman', 'usa', 'mental institution', 'post traumatic stress', 'psychiatrist', 'voyeur', 'neo nazi', 'punk band', 'skinhead', 'fake documentary', 'hidden camera', 'narrative', 'reality show', 'austria', 'cake', 'france', 'versailles', 'filming a wedding', 'lingerie slip', 'bitch', 'popular girl', 'prom', 'alamut', 'dagger', 'persia', 'cowboy boots', 'dancing', 'high school dance', 'black humor', 'dentist', 'hitman', 'family home', 'piano', 'skyscraper', 'tightrope walker', 'russian spy', 'strong female lead', 'concert', 'live concert', 'pop group', 'spice girl', 'business', 'crime scene cleanup', 'male frontal nudity', 'tv mini series', 'bosnian war', 'armor', 'human bomb', 'missile attack', 'fighter plane', 'jet fighter', 'refueling', 'soviet union', 'bachelor', 'character says i love you', 'flirting', 'social satire', 'mother son relationship', 'rabbit', 'winnie the pooh', 'don juan', 'retirement', 'cotton', 'flogging', 'los angeles california', 'one night stand', 'single mother', 'white panties', '16 year old', 'tinnitus', 'kung fu', 'martial artist', 'martial arts', 'sister', 'ballistics', 'food', 'raccoon', 'turtle', 'gold', 'mining', 'prospector', 'gay interest', 'critically acclaimed', 'juarez mexico', 'u.s. mexico border', 'war on drugs', 'desperation', 'fog', 'giant insect', 'survival horror', 'balladeer', 'bachelor party', 'activist', 'mystery killer', 'audition', 'deaf', 'singing', 'beach house', 'blonde bombshell', 'cheating husband', 'wife meets mistress', 'hockey team', 'respect', 'chariot race', 'jerusalem', 'dialogue driven', 'jury', 'cabin', 'going in circles', 'pac man', 'stranger', 'cooking', 'guardian', 'kitchen', 'neo noir', 'father daughter incest', 'mental hospital', 'fellatio', 'isolated house', 'overprotective father', 'social isolation', 'unsimulated sex', 'animal', 'donkey', 'lamb', 'pig', 'rooster', 'alien space craft', 'giantess', 'gore', 'country singer', 'music group', 'air raid', 'beer', 'strip club', 'stripper', 'limousine driver', 'cartoon cat', 'puss in boots', '15th century', 'cathedral', 'gypsy', 'paris', 'victor hugo', 'bodyguard', 'death of husband', 'male nanny', 'wrestling match', 'abandoned building', 'chernobyl disaster', 'exploration', 'tour guide', 'van', 'act of kindness', 'neighbor', 'vacation', 'anger management', 'argument', 'irony', 'sarcasm', 'actress', 'assistant director', 'wardrobe', 'pop singer', 'secret service agent', 'heroine', 'kindred spirit', 'rural setting', '2000s', 'golf caddy', 'golf course', 'golfer', 'question mark in title', 'alcohol', 'alcoholic drink', 'drunken man', 'american dream', 'convertible', 'michael myers', 'digging', 'escape attempt', 'tunnel', '1940s', 'child hero', 'fantasy world', 'reference to peter pan', 'army', 'avatar', 'kingdom', 'tribe', 'water', 'crystal meth', 'search for father', 'male pubic hair', 'mall', 'police detective', 'pubic hair', 'security guard', 'gunfight', 'hong kong police', 'police escort attacked', 'denmark', 'wristwatch', 'independent film', 'french', 'police detective hero', 'cage', 'year 2029', 'camp', 'governor', 'senate', 'u.s. senate', 'robot human relationship', 'turing test', 'video surveillance', 'delivery boy', 'dinner', 'punishment', 'attempted murder', 'based on comic book', 'escape from prison', 'lost at sea', 'wealth', 'australia', 'body paint', 'kangaroo', 'camera focus on female butt', 'gay bar', 'millionaire', 'singing in a car', 'corrupt', 'motor scooter', 'newborn baby', 'scooter', 'single', 'army rangers', 'abusive mother', 'adopted daughter', 'borderline personality disorder', 'perfection', 'sadistic cruelty', 'premarital sex', 'sex standing up', 'autumn', 'four seasons', "looking at one's self in a mirror", 'unhappiness', 'journey', 'tiger', 'bdsm', 'employer employee relationship', 'based on stage musical', 'cinderella', 'little red riding hood', 'chauffeur', 'disorganized crime', 'ransom', 'slapstick', 'broadway play', 'replacement', 'stage', 'acronym in title', 'period in title', 'title ends with period', 'cane', 'iowa', 'lawn mower', 'tractor', 'wisconsin', 'santa claus', 'son', 'teacher', 'congressman', 'cook', 'birthday party', 'high school senior', 'millennial generation', 'disposing of a dead body', 'high school student', 'teenage boy', 'stillborn child', 'flat tire', 'gay activist', 'gay pickup', 'gay rights', 'cigarette smoking', 'hand on butt', 'niagara falls', 'plastic bubble', 'dental braces', 'dental headgear', 'former child star', 'panties', 'gambling', 'novel', 'stenographer', 'deer', 'grizzly bear', 'employee employer relationship', 'ship', 'wake', 'italian', 'kidnapping a child', 'reunion', 'commandant', 'russian mafia', 'security system', 'racism', 'separation from family', 'social injustice', 'whipping', 'abusive father', 'coincidence', 'fictional game show', 'fictional quiz show', 'multiple storyline', 'noodles', 'salary', 'illuminati', 'planetary alignment', 'time', 'tomb', 'tomb raider', 'antique', 'cockney accent', 'hatchet', 'drinking', 'female dressed as male', 'nebraska', 'transgender', 'boxer', 'diamond', 'carnage', 'purple panties', 'sexual attraction', 'shaved labia', 'fairy tale', 'true love', 'hollywood', 'scandal', 'diorama', 'idiot', 'irs', 'mouse', 'aspiring actor', 'pimp', 'suitcase full of money', 'boxing trainer', 'women boxing', '23 year time span', 'loss of loved one', 'male female friendship', 'british secret service', 'official james bond series', 'terrorist plot', 'blond boy', 'chlamydia', 'list', 'young couple', 'catholic orphanage', 'long lost siblings', 'twins', 'boob job', 'breast enlargement', 'breast surgery', 'cheating on a test', 'ejaculation', 'disarming someone', 'qing dynasty', 'stabbed with a bayonet', 'war violence', 'skinny dipping', 'humanoid robot', 'man versus machine', 'prosthetic limb', 'robot as pathos', 'robot vs. robot', 'nsa', 'nsa agent', 'prague', 'loyalty', 'genghis khan', 'starving', 'fighting', 'out of body experience', 'reference to the exploited', 'druid', 'eclipse', 'rune', 'scalping', 'warlock', 'convenience store', 'punctuation in title', 'interrogation', 'chaos', 'riot', 'shrine', 'inspiration', 'stage play', 'portal', 'teleportation', 'troubled production', 'anti hero', 'greed', 'evil wizard', 'quidditch', 'wizard', 'mechanic', 'greeting card', 'office', '1920s', 'ambiguity', 'childhood friend', 'speakeasy', 'reference to batman', 'valentine', 'lolita', 'nymphet', 'older man young girl relationship', 'latin', 'pirate ship', 'treasure', 'treasure map', 'greenland', 'u.s. soldier', 'apartment building', 'blood sample', 'crucifix', 'ussr', 'binoculars', 'electronic tag', 'house arrest', 'watching someone', 'coal', 'irish immigrant', 'night', 'latin music', 'on tour', 'tejano', 'fort', 'native american', 'trade', 'escort', 'male escort', 'wedding date', 'animal costume', 'break up', 'fatherhood', 'rapper', 'talking baby', 'camera focus on a female butt', 'hairy buttocks', 'seal the animal', 'wedding planner', 'long distance relationship', 'porn actor in mainstream movie', 'cigarette butt', 'drone', 'funeral procession', 'nellis air force base', 'college campus', 'dance craze', 'depression', 'personal ad', 'catfight', 'dominatrix', 'spanking', 'special forces', 'american', 'driver', 'custody battle', 'gay friend', 'assassination', 'blood brother', 'dynasty', 'stabbing a woman', 'exploding truck', 'machine', 'skynet', 'accidental killing', 'drug humor', 'profanity', 'sexual humor', 'stand up comedy', 'directorial debut', 'nurse', 'old age home', '13 year old', '13th birthday', '30 year old', 'year 1987', '18 wheeler', 'businessman', 'historical fiction', 'kuwait', 'magic kit', 'stage magician', 'vegas', 'debutante', 'green beret', 'insanity', 'vietnam', 'hoax', 'mafia boss', 'mafiosi', 'mafioso', 'lifting someone into the air', 'paranormal phenomena', 'voice over letter', 'year 1818', 'little league', 'con artist', 'con man', 'deception', 'rivalry', 'seduction', 'shoe salesman', 'written by director', 'karate', 'apartheid', 'nation', 'nelson mandela', 'rugby', 'mob boss', 'thug', 'alcatraz', 'blanket', 'sesame street', 'damsel in distress', 'inspired by true events', 'minnesota', 'year 1990', 'psychic vision', 'staff physician', 'dying words', 'guitar', 'art thief', 'insurance', 'master thief', 'reference to iron maiden', 'candle', 'end of the world', 'record', 'dea', 'female impersonation', 'gender disguise', 'obesity', 'competition dancing', 'location in title', 'eugene oregon', 'obsession', 'employment agency', 'stage fright', 'childbirth', 'nude protest', 'nudist', 'floppy disk', 'modem', 'rival', 'repossession', 'surgery', 'bar mitzvah', 'marital breakup', 'nose job', 'rabbi', 'genital dismemberment', 'mulching someone', 'tooth fairy', 'woodchipper', 'michigan', 'child spy', 'notebook', 'apple computer', '3 dimensional', 'englishman abroad', 'reference to the doors', 'greek', 'manifestation', 'sphere', 'abduction', 'martial law', 'machete', 'army of the dead', 'necronomicon', 'reluctant hero', 'skeleton soldier', 'end credits roll call', 'sniper', 'ancient greece', 'forbidden love', 'hercules', 'chopping down a tree', 'hanging upside down', 'shopkeeper', 'vampire hunter', 'fair trade', 'non profit', 'rockumentary', 'danish', 'double agent', 'nazi occupation', 'resistance', 'ireland', 'sheep', '18th birthday', 'community center', 'resurrection', 'great white shark', 'killing an animal', 'british horror', 'pub', 'record collection', 'child', 'comic book', 'murderer', 'full frontal female nudity', 'menage a trois', 'caper', 'computer genius', 'security chief', 'fountain of youth', 'immortality', 'young man', '13 year olds', 'adolescence', 'peer pressure', 'werewolf bite', 'championship', 'harvard', 'physics', 'zamboni', 'jewel', 'b movie', 'masked killer', 'trick or treat', 'anarchist', 'greedy institution', 'prehistoric times', 'primitive', 'trio', 'crushed by a car', 'one against many', 'currency', 'film set', 'movie set', 'slow motion', 'urban legend', 'eyeball run over by car', 'latex gloves', 'suspension bridge', 'dead husband', 'forename as title', 'rodeo cowboy', 'falling through a glass roof', 'hitchcockian', 'prism', 'rented room', 'scrofula', 'sketching', 'sunrise', 'falling into water', 'medical doctor', 'roof leak', 'swimming in the ocean', 'beast', 'beastmaster', 'psychotronic', 'crocodile', 'garbage can', 'knife', 'paramount pictures', 'city symphony', 'diary', 'essay', 'surveillance', 'brutality', 'trap', 'amnesia', 'aquarium', 'fish', 'fish tank', 'gigolo', 'norwegian', 'generation y', 'strong female character', 'scepter', 'child custody', 'disability', 'mentally impaired man', 'single father', 'apocalypse', 'mercenary', 'urban setting', 'death row', 'inmate', 'oklahoma city', 'sexual promiscuity', 'innocence', 'negotiator', 'major', 'arab stereotype', 'broadway', 'demonic possession', 'exorcist', 'loss of innocence', 'goose', 'alter ego', 'teen movie', 'christmas party', 'held at gunpoint', 'arm tattoo', 'cauterizing a wound', 'dancing on a table', 'dinner party', 'book strap', 'racial discrimination', 'racial tension', 'stereotype', 'cider', 'fox', 'tale', 'mysterious villain', 'police chase', 'shooting range', 'police partner', 'serial murder', 'seven deadly sins', 'bartender', "new year's eve", 'punk', 'armenian', 'armenian genocide', 'customs agent', 'genocide', 'prayer', 'brontosaurus', 'classical music', 'dinosaur', 'dinosaur feature', 'earthquake', 'bus stop', 'community college', 'speech class', 'porn star', 'advice', 'black romance', 'reading a book', 'baseball movie', 'little league baseball', 'fantastic four', 'space', 'superheroine', 'business card', 'narcissism', 'child in peril', 'electroshock therapy', 'memory loss', 'faberge egg', 'trust', 'ancient astronaut', 'famous line', 'message from outer space', 'monolith', 'star child', 'graduation', 'graduation speech', 'tv studio', 'unemployment', 'bunker', 'car crash', 'kidnapping', 'minimal cast', 'modern silent movie', 'silent movie', 'coke', 'meeting future self', 'reference to spongebob squarepants', 'sony', 'bet', 'prom queen', 'competitiveness', 'creator', 'entrepreneur', 'facebook', 'intellectual property', 'bridge', 'midget', 'sideshow', 'british', 'indian', 'repair shop', 'nun', 'cheerfulness', 'garden gnome', 'magical realism', 'hyperthymesia', 'iguanodon', 'lemur', 'meteor', 'nesting grounds', 'single parent', 'supermarket', 'paris france', 'villain', 'brooklyn new york city', 'found footage', 'night vision', 'statue of liberty new york city', 'subway tunnel', 'dwarf', 'hobbit', 'gentleman', 'georgian', 'billionaire', 'playboy', 'trust fund baby', 'serum', 'dance competition', 'dance lesson', 'gang violence', 'alopecia', 'belfast northern ireland', 'irish republican army', 'scalper', 'robber', 'friendship between women', 'norway', 'steve', 'napoleonic wars', 'pursuit', 'royal navy', 'zorro', 'manhattan new york city', 'crop circle', 'intelligence', 'paranormal', 'phenomenon', 'universe', 'children', "children's tv show", 'adult humor', 'process server', 'roach', 'stoner', 'abandoned house', 'swing', 'forced to strip', 'porn actress', 'jackal', 'cop', 'bigger dreams', 'concert promoter', 'hotel maid', 'pep talk', 'rap concert', 'old west', 'stealing from the rich', 'filipino', 'guerrilla army', 'las vegas nevada', 'mexican border', 'embarrassing male nudity', 'embarrassing nudity', 'truck driver', 'contagion', 'cure', 'infection', 'virus', 'cthulhu', 'reputation', 'cruise', 'drag', 'female armpit hair', 'sorority', 'oil', 'tycoon', 'dating', 'divorcee', 'ex husband', 'ex wife', 'container', 'shipping container', 'storm', 'celebration', 'locust', 'repressed memory', 'cave painting', 'medical scanner', 'pregnant with an alien fetus', 'suspended animation', 'environment', 'gas chromatograph', 'nature', 'cult', 'gas station', 'supernatural ability', 'slacker', 'grief', 'hiking', 'loss of mother', 'mother daughter relationship', 'jujitsu', 'kickboxing', 'organ', 'ancient egypt', 'bracelet', 'scorpion', 'attacked by a plant', 'hill', 'wellesley college', 'car falling off a cliff', 'star died before release', 'christian', 'lesbian sex', 'oral sex', 'pregnant schoolgirl', 'adopted son', 'blog', 'clitoris', 'deep throat', 'porn industry', 'scissors', 'fire breathing dragon', 'militia', 'ex boyfriend ex girlfriend relationship', 'media tycoon', 'world war three', 'bookstore', 'simultaneity', 'angel', 'bethlehem', 'nazareth', 'virgin', 'chain', 'mysterious killer', 'warning', 'academic probation', 'school expulsion', 'smashing a guitar', 'bag of money', 'found money', 'screenplay adapted by author', 'snow', 'gay slur', 'hatred', 'celebrity criminal', 'public enemy number one', 'vegetable', 'waiter', 'arctic circle', 'volunteer', 'city name in title', 'place name in title', 'florida', 'toy store', 'canal', 'tailor', 'preacher', 'lesbianism', 'watching pornography', 'concierge', 'stalking', 'adult actor playing minor', 'politician', 'progeria', 'reference to spiderman', 'crane the bird', 'hoover dam', 'police department', 'rainy night', 'towel', 'chick', 'jail', 'monster movie', 'paper mill', 'pollution', 'dyed hair', 'fake tan', 'rock', 'twin', 'two headed person', 'couple', 'tropical', 'voyeurism', 'interracial couple', 'jesuit', 'pope', 'ss', 'helmet', 'medieval times', 'biological father', 'dad', 'product placement', 'pregnancy', 'question in title', 'crash', 'pistol', 'first lady', 'framed', 'crying during sex', 'humor', 'man with glasses', 'liverpool', 'protest', 'unrequited love', 'corrupt cop', 'murder of a police officer', 'hitler', 'no exit', 'war ruins', 'loss of memory', 'screwball', 'self discovery', 'social differences', 'abdication', 'forbidden city', 'republic', 'coastal town', 'iceland', 'snowstorm', 'winter storm', 'apprentice', 'orchestra', 'gloves', 'atrocity', 'central africa', 'ethnic warfare', 'united nations peace keeping force', 'united nations soldier', 'quest', 'the bronx', 'close up of breasts', 'power plant', 'sabotage', 'store clerk', 'video store', 'family business', 'jealousy', 'spiritual', 'good deed', 'protective male', 'prefect', 'masked man', 'moving', 'tokyo japan', 'toxic avenger', 'troma', 'farming', 'flesh eating', 'food industry', 'gluttony', 'electrocuted in bathtub', 'attack', 'pearl harbor', 'u.s. navy', "character says now that you're on the case i hope we're going to have some gratuitous sex and violence", 'cruise missile', 'missile', 'heiress', 'cia agent', 'consulate', 'safe house', "coup d'etat", 'mutiny', 'madagascar', 'zoo', 'haunted house', 'implied incest', 'squeegee', 'tied to a chair', 'mozart', 'musical prodigy', 'royal court', 'talent', 'airport security', 'confidence', 'iphone', 'internet', 'make up', 'publicist', 'purim', 'african', 'dialect', 'interpreter', 'threat', 'brains', 'exploding head', 'swarm behavior', 'spider', 'tarantula', 'super villain', 'tentacle', 'clown', 'elephant trainer', 'trapeze', 'trapeze artist', 'dynamite', 'ku klux klan', 'n word', 'atheist', 'female masturbation', 'lucifer', 'butcher', 'mute character', 'subway', 'alcoholic', 'kazakhstan', 'nomad', 'bare butt', 'defecation', 'father in law son in law relationship', 'sexual innuendo in title', 'dissection', 'medical school', 'medical student', 'benzedrine', 'cactus', 'poem', 'year 1949', 'highlander', 'illusion', 'immortal', 'playing with gun', 'working class family', 'archipelago', 'guillotine', 'redemption', 'action heroine', 'falling from height', 'mind control', 'trainhopping', 'truth serum', 'based on memoir', 'cambridge university', "lou gehrig's disease", 'physicist', 'hazmat suit', 'based on sketch comedy', 'evil corporation', 'dump', 'show', 'spectator', 'tire', 'jackpot', 'aborigine', 'colonialism', 'fence', 'half caste', 'class', 'cornwall connecticut', 'pawtucket rhode island', 'mormon', 'grindhouse', 'suffering', 'wales', 'bible', 'key', 'slum', 'wallet', 'blind prophet', 'sirens', 'texan', 'spitting', 'sushi', 'cold war', 'address as title', 'parents', 'horror movie remake', 'undead sex', 'undead sexuality', 'club', 'study', 'talmud', 'yeshiva', 'bat', 'hawaiian shirt', 'pregnant', 'gold rush', 'mirror', 'abuse', 'pickpocket', 'estranged wife', 'medication', 'graduate student', 'sprite', 'uncle nephew relationship', 'disfigurement', 'harpooner', 'motorboat', 'sequel to cult film', 'collector', 'ghost hunter', 'glass house', 'ostrich', 'waterhole', 'wildebeest', 'zebra', 'kissing in an elevator', 'mythical hero', 'silent protagonist', 'car bomb', 'card cheat', 'cash', 'gambling syndicate', 'economy', 'individual versus society', 'christianity', 'reference to justin bieber', 'reference to taylor swift', 'worship', 'abusive stepfather', 'attempted rape', 'food shortage', 'starving child', 'desire', 'resort town', 'sales', 'dance crew', 'mortgage', 'evil man', 'absent father', 'asian american', 'escaped convict', 'motorcycle', 'male full frontal nudity', 'based on cult comic book', 'home alone', 'talking cat', 'meeting', 'statistics', 'voice over', 'play', 'playwright', 'epidemic', 'paradox', 'freemason', 'jack the ripper', 'opium', 'victorian era', 'new york cosmos', 'professional sports', 'soccer', 'soccer team', 'northern ireland', 'consciousness', 'power outage', 'technology', 'jurassic park', 'paleontologist', 'based on cult favorite', 'disease', 'female hero', 'breaking and entering', 'gallows', 'hanging', 'salmon', 'high school teacher', 'teacher inspires students', 'teaching', 'changing the future', 'invention', 'nonlinear timeline', 'compulsive liar', 'fired from a job', 'gift', 'substance abuse', 'lottery', 'amputee', 'anti romantic comedy', 'sex comedy', 'comedian', 'craigslist', 'stand up comedian', 'climate', 'global warming', 'science', 'colosseum', 'homeless man', 'tombstone', 'surfing', 'surfing contest', 'wave', 'bail bond', 'jeans', 'shot in the butt', 'mennonite', 'shower', 'punched in the nose', 'rafting', 'river', 'white water rafting', 'cuba', 'cobra', 'medallion', 'waterfall', 'consensual sex', 'surfer', 'humiliation', 'man child', 'moped', 'coven', 'occult', 'outcast', 'spell', 'cookie', 'microfilm', 'cheerleader', 'valedictorian', 'recession', 'record company', 'rock star', 'plant', 'soil', 'life insurance policy', 'mother', 'recluse', 'shy', 'famous score', 'vietnam war', 'perversion', 'chainsaw', 'mansion', 'battle of agincourt', 'shakespeare play', 'sword and shield', 'bachelorette party', 'drug use', 'overdose', 'wedding dress', 'dalai lama', 'himalaya', 'tibet', 'arcade', 'candy', 'glitch', 'medal', 'lebanon', 'pittsburgh steelers', 'baltimore colts', 'kicked in the crotch', 'tough girl', 'lost world', 'squirrel', 'weasel', 'route 66', 'tow truck', 'contract', 'devil', 'soul', 'holy land', 'knight templar', 'monastery', 'radical transformation', 'dc comics', 'nuclear disarmament', 'superman', 'call in', 'date', 'hot air balloon', 'show producer', 'ant', 'grasshopper', 'overhearing', 'geek', 'reference to janet jackson', 'drug dealing', 'car chase', 'plot twist', 'police van', 'rear ending a car', 'cheerleading', 'teen comedy', 'teenage romance', 'community', 'fem bot', 'tv producer', 'abortion clinic', 'clinic', 'coyote', 'quarterback', 'immigrant', 'eighteen wheeler', 'illegal street racing', 'trucker', 'hypochondriac', 'television', 'college girl', 'midlife crisis', 'pittsburgh pennsylvania', 'courtship', 'quaker', 'health club', 'superhero', 'toxic waste', 'baltimore orioles', 'boston red sox', 'ups', 'world series', 'kissing', 'arctic', 'rhedosaurus', 'british prime minister', 'female prime minister', 'prime minister', 'black militant', 'frustration', 'pizza parlor', 'pizzeria', 'city by night', 'driving at night', 'taxi', 'family vacation', 'national lampoon series', 'clinical trial', 'neuropharmacology', 'side effect', 'nerd', 'animal rights', 'congresswoman', 'doorman', 'transvestism', 'basement', 'library', 'track', 'nihilism', 'rug', 'white russian', 'october', "child's point of view", 'real estate', 'washing machine', 'contest', 'refusing to eat', 'starving oneself', 'brother sister relationship', 'shrimper', 'south carolina', 'loneliness', 'playing against type', 'retrograde narrative', 'lava', 'volcano', 'caramel', 'suitor', 'f word', 'umbrella', 'attorney', 'cat and mouse', 'pan am', 'boogeyman', 'hometown', 'workhouse', 'godparent', 'pacific palisades', 'rodeo', 'pandemic', 'lesbian mother', 'lesbian parent', 'lesbian relationship', 'lesbian romance', 'child abuse', 'basketball coach', 'box', 'yard sale', 'marigold', 'retiree', 'bad luck', 'hole', 'fate', 'federation', 'cannibalism', 'condom', 'hook for hand', 'satanic ritual', 'driving a car down stairs', 'duct tape gag', 'held captive', 'sailing', 'parole', 'parole officer', 'puppy', 'final concert', 'thanksgiving', 'breast cancer', 'comma in title', 'detroit michigan', 'firefighter', 'master of disguise', 'football team', 'gang leader', 'italian woman', 'period piece', 'purgatory', 'bee', 'dream sequence', 'scatological humor', 'squashed bug', 'half sister', 'medicine', 'court', 'intrigue', 'spain', 'movie producer', 'sting operation', '1969 dodge charger', 'bikini', 'misogynist', 'blindfolded woman', 'control', 'male dominance', 'sex game', 'sex in public', 'heir', 'falcon', 'accountant', 'tobacco industry', 'subjective camera', 'teen angst', 'troubled teen', 'fired from the job', 'person in car trunk', 'racial impersonation', 'south asian', 'street gang', 'barn owl', 'owl', 'ruler', 'power', 'driving range', 'tournament', 'winnebago', 'hinduism', 'marriage engagement', 'caught having sex', 'telephone call', 'voice mail', 'unplanned pregnancy', 'website', 'desert planet', 'dune', 'giant worm', 'spice', 'exodus', 'moses', 'actor playing multiple roles', 'axe', 'dark comedy', 'sadism', 'handheld camera', 'impostor', 'rap', 'rock paper scissors', 'august', 'patriarch', 'bank robber', 'crime spree', 'good versus evil', 'holcomb kansas', 'truman capote', 'gay republican', 'bare midriff', 'cemetery', 'farmhouse', 'radiation', 'running out of gas', 'premonition', 'roller coaster', 'tough cop', 'spin off', 'conclave', 'reference to bernini', 'reference to raphael', 'bobsled', 'jamaica', 'kansas city', 'female bassist', 'female drummer', 'female guitarist', 'female singer', 'portland oregon', 'barricade', 'innkeeper', 'cigar smoking', 'tombstone arizona', 'street hustler', 'fake family', 'marijuana', 'drug culture', 'filthy toilet', 'heroin addict', 'heroin withdrawal', '2020s', 'near future', 'one word series title', 'year 2027', 'mythology', 'prince', 'trojan', 'troy', 'nude nun', 'russian nobleman', 'showcase', 'female stockinged feet', 'female stockinged soles', 'nude pantyhose', 'pantyhose', 'anger', 'family reunion', 'father son reunion', 'movie making', 'maximum security prison', 'addiction', 'diet pill', 'drug addiction', 'fast motion scene', 'ex partner', 'solar system', 'starship', 'year 2047', 'bisexual', 'explicit sex', 'knight', 'mentor', 'foreboding', 'hideout', 'aunt niece relationship', 'building', 'returning character killed off', 'six word title', 'writer director', 'class conflict', 'edwardian era', 'sexual awakening', 'hiding', 'device', 'disaster', 'snow white', 'estranged father', 'genetics', 'lie', 'screenwriter', 'vulgarity', 'high school basketball', 'dying', 'to do list', 'twenty something', 'underwear', 'breakfast cereal', 'department store ride', 'face painting', 'sponge bob square pants slippers', 'phantom', 'skull', 'based on poem', 'the huns', 'reference to kurt cobain', 'year 1994', 'obscenity', 'vacant lot', 'barricading a door', 'injection in leg', 'taser', 'presumed dead', 'age difference', '3d', 'cossack', 'warden', 'recording', 'saxophone', 'goblet', 'assistant', 'frankenstein', 'costume drama', 'final battle', 'napoleon', 'napoleon bonaparte', 'arizona', 'dementia', 'married man', 'sun city arizona', 'distress signal', 'family in danger', 'stealing a car', 'mortal kombat', 'disobeying orders', 'evangelist', 'revival', 'traveling salesman', 'chicken', 'locker room', 'roller skates', 'tubesocks', 'chosen one', 'enchanted object', 'radio', 'highwayman', 'nell gwynne', 'lap dance', 'lap dancing', 'striptease', 'labyrinth', 'maze', 'red light', 'debauchery', 'lust', 'palace', 'saddam hussein', 'gossip', 'cult tv', 'sandman', 'spider man', 'symbiote', 'venom', 'anchorman', 'newsroom', 'tv station', 'new home', 'rainbow', 'reference to avenged sevenfold', 'reference to slayer', 'switching lights on and off', 'heart surgeon', 'pop chart', 'gambler', 'claw fight', 'commando', 'wolverine the character', 'country club', 'summer vacation', '1000000 b.c.', 'lawn mowing', 'stone age', 'caddy', 'gopher', 'loss of husband', 'suntan pantyhose', 'male star appears nude', 'fashion magazine editor', 'abuse of power', 'prison guard', 'race relations', 'social problem', 'acting', 'briefs', 'reference to cuba gooding jr', 'reference to netflix', 'written by star', 'female removes her clothes', 'fondling', 'drug deal', 'sex on stairs', 'mad scientist', 'actress shares first name with character', 'car wash', 'd.j.', 'internship', 'stockbroker', 'ghetto', 'dark horse comics', 'reference to david letterman', 'robert crumb', 'dancing sheep', 'fetish', 'musical scene', 'racial humor', 'snake poison', 'space shuttle', 'chores', 'father figure', 'helicopter parents', 'in laws', 'mormon church', 'proposition', 'ghostbuster', 'ghostbusters', 'reboot', 'super strength', 'vision', 'manchuria', 'new york city new york', 'carl jung', 'psychoanalysis', 'sigmund freud', 'government agent', 'german', 'heavy water', 'paper bag', 'female bare foot', 'picket fence', 'baseball practice', 'gunfighter', 'poker game', 'year 1879', 'monopoly', 'brain capacity', 'fictional drug', 'taking a pill', 'libya', 'u.s. ambassador', 'irish civil war', 'irish free state', 'sinn fein', 'operating system', 'fictitious country', 'latin america', 'reference to fidel castro', 'florist', 'motivational speaker', 'self help guru', 'seminar', 'faked death', 'japanese', 'poison', 'secret headquarters', 'carnival', 'rookie cop', 'intervention', 'killer doll', 'vixen', 'lifting female in air', 'lifting male in air', 'special education', 'able to see the dead', 'surprise ending', 'classic literature', 'censorship', 'heil hitler', 'science runs amok', 'supernatural power', 'battlefield', 'drug dealer', 'neighborhood', 'based on cartoon', 'cgi film', 'computer animation', 'motorcycle gang', 'climate change', 'global', 'polar bear', 'scientific', 'death of child', 'sniper rifle', 'ice skater', 'crook', 'creature feature', 'hijacker', 'shot in the leg', 'based on the bible', 'nativity', '9 year old', 'illinois', 'new job', 'tap dancing', 'undressing', 'based on toy', 'dual personality', 'evil businessman', 'good cop bad cop', 'lego', 'moroccan', 'xenophobia', 'possession', 'courtesan', 'kama sutra', 'police shootout', 'geese', 'idaho', 'national park', 'park', 'detroit tigers', 'perfect game', 'pitcher', 'debt', 'treasure hunter', 'yacht', 'negotiation', 'deep space', 'imaginary friend', 'outback', 'town', 'lingerie', 'extraterrestrial', 'flash mob', 'target', 'mini skirt', 'marital rape', 'awol', 'demand', 'roses', 'chocolate', 'chocolate shop', 'espionage', 'secret mission', 'spy film', 'chimpanzee', 'elephant', 'shaman', 'documentary crew', 'mass suicide', 'survivor', 'tied up while barefoot', 'viking', 'main character dies', 'black cat', 'king henry viii', 'necktie', 'rock music', 'tuxedo', 'cattle', 'rejecting a marriage proposal', 'talking to a dog', 'vomiting on someone', 'lightning', 'nba', 'shoe', 'chronic traumatic encephalopathy', 'concussion', 'pathologist', 'car hitting pedestrian', 'fan', 'nineteenth century', 'reversal of fortune', 'appendicitis', 'clique', 'lip gloss', 'based on play', 'ceo', 'dubai', 'product', 'hebrew', 'hacking', 'hijacking', 'elevator girl', 'spacecraft accident', 'guardian angel', 'second chance', 'small town life', 'prison escape', 'solitary confinement', 'police officer killed', 'police officer shot in the chest', 'police officer shot in the forehead', 'police officer shot in the head', 'based on stage musical based on novel', 'alternate reality', 'girl gang', 'rambo', 'arrest', 'raid', 'europe', 'frog', 'gulag', 'prison break', 'puppet', 'twin brother and sister', 'douglas macarthur', 'inchon', 'invasion', 'elongated cry of no', 'friends become enemies', 'kicked in the butt', 'robotic midwife', 'opera singer', 'platonic marriage', 'cruelty', 'tension', 'flash photography', 'game of death', 'jigsaw', 'pig mask', 'claw crane', 'american west', 'horse riding', 'begins with narration', 'female warrior', 'inbetwequel', 'jedi knight', 'war against machines', 'holiday horror', 'stonehenge', 'music producer', 'duty', 'submarine movie', 'u boat', 'anthropomorphic animal', 'furry', 'autistic son', '1830s', 'baptism', 'intolerance', 'sibling rivalry', 'life magazine', 'female explicit nudity', 'penetration', 'con', 'grifter', 'organized crime', 'shot in the forehead', 'kick the can', 'front desk', 'apostrophe in title', 'teen sex comedy', 'krill', 'communist', 'three musketeers', 'twin brother', 'countryside', 'scar', 'golf ball', "grandmother's house", 'evil queen', 'composer', 'self esteem', 'casual sex', 'singles', 'mom', 'pta', 'reference to mad max', 'scene during end credits', 'confession', 'flickering light', 'stealing', 'traitor', 'antidote', 'llama', 'brumby', 'hammer', 'monaco', 'tony stark', 'controversy', 'doppelganger', 'group therapy', 'self mutilation', 'twin actresses for twin sisters', 'drug smuggling', 'interlinked stories', 'lifeguard', 'talent show', 'amphibian', 'frog prince', 'male nurse', 'gulf war', 'deja vu', 'time loop', 'trapped in a time loop', 'size positive', 'thin', 'wet dream', 'crime boss', 'inner city', 'parent', 'first person narration', 'prison cell search', 'wrongful imprisonment', 'shop', 'commune', 'environmentalism', 'healing power', 'marvel comics', 'mecha', 'regeneration', 'self healing', '14 year old', 'gadget', 'home invasion', 'police officer shot', 'detention', 'defense attorney', 'cartographer', 'mathematician', 'numbers', 'pattern', 'using a metal detector on a beach', 'martian', 'cleavage', 'comic violence', 'intentionally bad', 'stylized violence', 'irreverence', 'columbine', 'columbine high school killings', 'firearm', 'school shooting', 'girl group', 'fountain', 'law firm', 'high school graduation', 'gi joe', 'bow and arrow', 'rifle', 'ruins', 'missing wife', 'murder suspect', 'adult entertainment industry', 'san fernando valley california', 'bomber', 'flask', 'hell', 'the devil', 'bisexuality', 'manufacturing', 'sibling', 'el salvador', 'reagan administration', 'war journalism', 'hdtv', 'elm street', 'reservation', 'soccer coach', 'soccer mom', 'brainwashed assassin', 'breakfast', 'manitoba', 'science fiction writer', 'flying', 'gay subtext', 'machismo', 'excavation', 'relic', "character's point of view camera shot", 'fire station', 'television reporter', 'selfishness', 'turn of the century', 'smithsonian', 'terminal cancer', 'board of directors', 'industry', 'mail', 'cartoonist', 'zodiac', 'zodiac killer', 'lsd', 'missing friend', 'mk ultra', 'radio broadcast', 'castration', 'fast food', 'indian burial ground', 'bullfight', 'bullfighting', 'action hero', 'forced prostitution', 'title spoken by character', 'high society', 'florida keys', 'reference to douglas fairbanks', 'reference to jesse owens', 'reference to randolph scott', 'year 1874', 'ski resort', 'snowboard', 'snowboarding', 'gunshot', 'hunt', 'kindergarten teacher', 'australian', 'scholarship', 'bowling', 'rajah', 'sphinx', 'spleen', 'villainess', 'architect', 'superhero spoof', 'around the world', 'tattoo', 'closet', 'psychiatric hospital', 'real life sisters playing sisters', 'crepe suzette', 'mata hari spoof', 'slug', 'bellhop', 'alien friendship', 'flying car', 'loner', 'talisman', 'dream analysis', 'guilt complex', 'post traumatic stress disorder', 'car fire', 'cockroach', 'new year', 'soviet', 'live studio audience', 'christmas tree', 'fair', 'stranded', 'masked villain', 'tv', 'michael jai white', 'steve austin', 'partnership', 'vigilante', 'comic book history', 'comics', 'spawn', 'walking dead', 'birth', 'misadventure', 'suitcase', 'lycanthrope', 'shapeshifting', 'werewolf transformation', 'train wreck', 'blade the character', 'mutation', 'affair', 'altering history', 'image consultant', 'one two combo', 'blockade', 'crisis', 'cross', 'doused with gasoline', 'cross dresser', 'motion sickness', 'irony of fate', 'landlady', "kissing someone's boot", 'midsummer', 'telling someone to shut up', 'valet', 'choreographer', 'locker', 'funfair', 'luna park', 'table football', 'toy comes to life', 'wig', 'brain', 'fiance', 'laboratory', 'surgeon', 'karma', "boss' daughter", 'crush', 'house sitter', 'housesitting', 'love interest', 'opec', 'revolutionary', 'black and white', 'survivalist', '18th century', 'highlands', 'scotland', 'sexual cruelty', 'sword fight', 'skunk', 'summer camp', 'wall', 'death personified', 'fear of death', 'answer to prayer', 'breast expansion', 'pleading with god', 'walking on water', 'promiscuity', 'reference to isaac newton', 'auction', 'montreal', 'violin', 'country music', 'pink panties', 'relapse', 'songwriter', 'miracle', 'bishop', 'loan shark', 'poker', 'reference to christy turlington', 'mariachi', 'american expatriate', 'casablanca morocco', 'gestapo', 'visa', 'clerk', 'hockey', 'video', 'canuxploitation', '1910s', 'artificially created woman', 'taxi driver', 'heart transplant', 'family abandonment', 'native american protagonist', 'native american reservation', 'heroism', 'super soldier', 'world domination', 'blind', 'long take', 'leader', 'somalia', 'marital problem', 'money problems', 'nervous breakdown', 'bus trip', 'parental love', 'surprise visit', 'train ride', 'tribute', 'domestic violence', 'unwanted pregnancy', 'neurosis', 'boy scouts', 'boy scouts camp', 'young love', '1810s', 'high tech', 'resentment', 'kissing in the rain', 'minion', 'undercover spy', 'woman agent', 'death of brother', 'talking with the dead', 'aroused', 'pornography documentary', 'pornstar', 'hockey mask', 'babysitter', 'ganja', 'kumbh mela', 'little boy', 'kissing a dead body', 'repeated dialogue', 'sex in pool', 'spring break', 'frozen time', 'obese man', 'remote control', 'stopped time', 'englishman', 'leaving flowers on a grave', 'one last job', "visiting wife's grave", 'mental illness', 'shot to death', 'alternative energy', 'energy', 'green', 'flatulence', 'interracial marriage', 'k2', 'auto theft', '14th century', 'revolt', 'tyranny', 'secret society', 'riding the rails', 'corporate greed', 'natural gas', 'female chef', 'authority', 'boss subordinate relationship', 'fast food restaurant', 'gullibility', 'kissing while having sex', 'reference to princess diana', 'wedding band', 'wedding ring', 'demo tape', 'directed by cast member', 'grieving father', 'guitar player', 'blackout', 'child pornography', 'microchip', 'silicon valley', '23rd century', 'dual role', 'cattle drive', 'dominican', 'teenage pregnancy', '16th century', 'practice', 'intimate partner homicide', 'flight', 'plane', 'templar', 'manipulative behavior', 'modern day adaptation', 'narcissistic woman', 'promiscuous woman', 'survival tactics', 'audio surveillance', 'pentagon', '50s', 'illegitimate pregnancy', 'part animation', 'love at first sight', 'mumbai india', 'reference to edward scissorhands', 'sleigh', 'credit card', 'cross country', 'serendipity', 'miser', 'muppet', 'flight attendant', 'kentucky', 'shoes', 'abu ghraib', 'housemaid', 'servant', 'caveman', 'strict father', 'african american gangster', 'falling to death', "in over one's head", 'corrupt sheriff', 'flash drive', 'reference to franz beckenbauer', 'student visa', 'tropical island', 'belching', 'reference to nicki minaj', 'sneezing', 'estranged couple', 'martial arts master', 'based on comic', 'dark hero', 'image comics', 'crash survivor', 'brazilian', 'cockatoo', 'macaw', 'hearse', 'tripod', 'alcoholic father', 'stepmother', 'swaziland', 'unfaithful wife', 'greek mythology', 'mousetrap', 'stick fight', 'birthday wish', 'father disappoints child', 'workaholic', 'animate object', 'reference to macgyver', 'whispering', 'lifeboat', 'sailing ship', 'scooby doo', 'voodoo', 'actor playing himself', 'boots', 'penis', 'womb', 'chrysler building manhattan new york city', 'delusion', 'u.s. air force', 'child protagonist', 'child swearing', 'ambiguous ending', 'fight club', 'five sisters', 'isolation', 'self injury', 'cheating', 'economics', 'human nature', 'sumo wrestling', 'ares', 'hades', 'titan', 'underworld', 'zeus', 'cliff', 'abandoned', 'depot', 'newfoundland new jersey', 'hot dog bun', 'needle', 'gay pride', 'highway travel', 'road movie', 'bollywood', 'jane austen', 'convent', 'cable guy', 'foreplay', 'chinatown', 'illegal immigrant', 'triad', 'formula 1', 'sex with a nurse', 'celebrity', 'consumerism', 'media manipulation', 'merchant', 'pound of flesh', 'antibes', 'reunion island', 'man in a wheelchair', 'year 2000', 'year 2002', 'socialite', 'captain america', 'marvel cinematic universe', 'live in concert recording', 'live performance', 'saxophonist', 'virginity', "based on children's book", 'pole dancer', 'basketball movie', 'tv commercial', 'tv news', 'black magic', 'refusal to kill', 'wuxia', 'catch 22', 'kneed in the groin', 'border', 'well', 'homoerotic fight', 'homosexual subtext', 'wcw wrestling', 'basketball team', 'grateful dead', 'lithuanian', 'ellipsis in title', 'long title', 'jewish identity', 'pin', 'ruby', 'bounty', 'miner', "valentine's day", 'american civil war', 'irish american', 'german soldier', 'rage', 'west virginia', 'blogger', 'reference to the shawshank redemption', 'tasered', 'french canadian', 'hockey player', 'machinist', 'coin collector', 'u.s. army', 'fistfight', 'prison bus', 'client', 'lesson', 'mississippi river', 'fashion model', 'model', 'retired', 'hostel', 'slovakia', 'peanuts', 'red hair', 'baseball player', 'brooklyn dodgers', 'major league baseball', 'bicycle', 'new neighbor', 'drawing', 'flanders', 'vice president', 'flare gun', 'human versus vampire', 'rapist', "martha's vineyard", 'older woman younger man relationship', 'weekend', 'hulk', 'opening action scene', 'alien abduction', 'lightning bolt', 'poseidon', 'stealth', 'top secret', 'call girl', 'charlatan', 'fortune teller', 'manuscript', 'deal', 'district attorney', 'meteorologist', 'snowed in', 'snowplow', 'weather', 'ethiopia', 'marksman', 'rangefinder', 'beard', 'woman in lingerie', 'grandmother grandson relationship', 'heart attack', 'charity', 'taxi dancer', 'choir', 'leper', 'lighthouse', 'theme park', 'kicked in the face', 'punched in the face', 'woman punching a man', 'indifference', 'beauty pageant', 'pageant', 'machine gun', 'police vigilantism', 'safe deposit', 'bestiality', 'brother sister incest', 'fisting', 'hit on the head with a pipe', 'support group', 'fraud', 'army cid', 'woman in military', 'dog saved from a fire', 'growing up', 'multiple narrators', 'famous twist', 'rebel', 'snowy landscape', 'foreign', 'orgasm', 'swimming', 'medical scrubs', 'purple gloves', 'person in a car trunk', 'suitcase bomb', 'ronin', 'shogun', 'death of wife', 'dracula', 'turk', 'culture shock', 'interracial relationship', 'madea series', 'ponzi scheme', 'accused of witchcraft', 'cat people', 'leopard', 'slip the undergarment', 'federal reserve bank', 'shredder', 'african american protagonist', 'name in title', 'talk show host', 'curiosity', 'freak', 'physical deformity', 'quarantine', 'previous marriage', 'airborne troops', 'allies', 'operation market garden', 'paratrooper', 'manipulative personality', 'murder of wife', 'neglected wife', 'grandmother', 'human versus spider', 'spider feature', 'woman sheriff', 'slapstick comedy', 'deacon', 'throat slitting', 'glass', 'wager', 'gothic', 'van helsing', 'hypnotherapy', 'lost painting', 'red riding hood', 'woodsman', 'idol', 'wyoming', 'cliche', 'exchange student', 'archeological dig', 'ark of the covenant', 'cartoon tiger', 'tigger', 'appeasement', 'frontier justice', 'street shootout', 'partisan commentary', 'political propoganda', 'reenactment', 'inspired by radio program', 'pirate broadcasting', 'pirate radio', 'pirate radio station', 'canadian gothic', '12th century', 'crusader', 'drunken sex', 'mashed potatoes', 'pancakes', 'tequila', 'dying repeatedly', 'buddy', 'prequel to cult favorite', 'junkie', 'french and indian war', 'red panties', 'divorced couple', 'reference to giuseppe verdi', 'senility', 'bare chested boy', 'car explosion', 'scotland yard', 'film actress', 'nosferatu', 'evil dead', 'hometown hero', 'philadelphia pennsylvania', 'astoria', 'kindergarten', 'guantanamo', 'lance corporal', 'bonn germany', 'failed kidnapping', 'handcuffs', 'projectile', 'chemical company', 'dental assistant', 'life', 'remake by original director', 'stop motion animation', 'autistic child', 'child in danger', 'paternity', 'testicles', 'disc jockey', 'archdiocese', 'land developer', 'real estate developer', 'court martial', "no man's land", 'political conspiracy', 'sound', 'impersonation', 'see through dress', 'woman on top', 'blender', 'computer screen', 'skype', 'webcam', 'gay love', 'irish adoption board', 'harmonica', 'killing', 'woman in bathtub', '5th century', 'capri', 'roman legion', 'bond girl', 'record player', 'wings', 'colombian', 'drug mule', 'stomach', 'crucifixion', 'loincloth', 'reference to god', 'grimdark', 'mutilated corpse', 'space marine', 'ultramarines', 'warhammer 40k', 'clouseau', 'panther', 'pink', 'pink panther', 'missing brother', 'venus flytrap', 'dollar bill', 'double entendre', 'interspecies sex', 'butt', 'downward duck', 'kingfisher', 'poacher', '1520s', 'aztec', 'spanish colonization', 'spanish conquest', 'brutal cop', 'thai boxing', 'vengeance', "april fool's day", 'psycho', 'vacuum cleaner', 'conjoined twins', 'separation', 'fur coat', 'misfit', 'home office', 'released from a mental hospital', 'trauma', 'mule', 'tennis court', 'the three stooges', 'cape kennedy', 'news footage', 'reference to christopher columbus', 'trapped in space', 'housewife', 'corrupt police', 'fireworks', 'kiss', 'to be continued ending', 'usaf', 'capitalist', 'child cancer', 'falling into a hole', 'inspiring', '1850s', 'dark secret', 'heart', 'organ donor', 'private detective', 'superhero team', 'x men', 'year 1983', 'campaigning', 'candidate', 'dead body in a bathtub', 'dead body in bathtub', 'inn', 'alternate timeline', 'cyborg', 'doggystyle sex', 'rough sex', 'sodomy', 'violent sex', 'costa rica', 'online gambling', 'deal with the devil', 'narrated by character', 'evil industrialist', 'ex girlfriend', 'political leader', 'digging a grave', 'grave digging', 'investigator', 'barber', 'barbershop', 'blaxploitation', 'muse', "tiffany's", 'actress playing herself', 'american in the uk', 'jock strap', 'nintendo game boy advance', 'competing businesses', 'instant messaging', 'shop owner', 'laser gun', 'astronomer', 'comet', 'governess', 'orchestral music score', 'poltergeist', 'stairway', 'counterfeit', 'counterfeit money', 'black cop', 'hostage negotiator', 'boating accident', 'daughters', 'refugee', 'criminal profile', 'madman', 'tabloid reporter', 'tragic villain', 'drummer', 'reference to jesus christ', 'contemporary west', 'neo western', 'headbanger', 'metallica', 'roadie', 'song in title', 'standing on the roof of a car', 'the beatles', 'fable', 'old man', 'chicago illinois', 'el train', 'ephebophile', 'pederasty', 'pedophilia', 'playing hooky', 'shaving', 'pressure', 'college football', 'surrender', 'hired killer', 'bad acting', 'pulp fiction', 'resistance fighter', 'voyager golden record', 'conquest', 'greek myth', 'teeth', 'assassination attempt', 'capitalism', 'cuban american', 'cuban refugee', 'tech noir', 'ex boyfriend', 'morning after', 'prince charming', 'quarrel', 'fight to the death', 'self survival', 'protective father', 'shrew', 'extreme sports', 'comedy of manners', 'false identity', 'period drama', 'amateur film', 'sexual innuendo', 'struggling parent', 'advertising', 'biblical', 'book of esther', 'esther', 'treachery', 'fabrication', 'fake', 'bromance', 'cult figure', 'femme fatale', 'indiana jones', 'unsubtitled foreign language', 'year 1957', 'cyberpunk', 'illegality', 'snuff film', 'family dinner', 'husband wife conflict', 'husband wife kiss', 'egyptian', 'egyptian god', 'egyptian goddess', 'sharpshooter', 'the end', 'german democratic republic', 'germany', 'halfway house', 'schizophrenia', 'blue light', 'light', 'co worker', 'leukemia', 'oncology', 'sick child', 'number', 'rag doll', 'acid trip', 'gameshow', 'hot tub', 'time lord', 'weed', 'bomb shelter', 'cuban missile crisis', 'naive man', 'battleship', 'message', 'yamato', 'duchess', 'duke', 'kenya', 'tween girl', 'san diego california', 'after dark horrorfest', 'a cappella', 'anthropomorphic mouse', 'marshmallow', 'albanian', 'revenge motive', 'teenage daughter', 'freight train', 'runaway train', 'train engineer', 'wax', 'pg 13 sequel to r rated franchise', 'march', 'million man march', 'animal abuse', 'blue clay', 'diversion', 'ends with biographical notes', 'ends with historical notes', 'human branding', 'man slaps a woman', 'manhunt', 'rooftop', 'brothel', 'ex lover', 'haunted by the past', 'autobiography', 'german shepherd', 'angry spirit', 'aura', 'electronic voice phenomena', 'electronics', 'near death experience', 'mexico city', 'new jersey', 'fascist', 'sound barrier', 'space program', 'u.s. space program', 'award', 'high school reunion', 'cleopatra', 'attention deficit disorder', 'bed', 'cheetah', 'air cavalry', 'vietnamese', 'suicidal thoughts', 'movie in title', 'death of protagonist', 'follow that car', 'la dolce vita', 'corrupt politician', 'chainsaw murder', 'human monster', 'music score composed by director', 'obscene finger gesture', 'bhutan', 'buddhist', 'monk', 'weaver', 'child star', 'star', 'gun fu', 'fantasy life', 'teenage crush', 'killed in police car', 'police officer shot in the leg', 'police officer shot through the heart', 'whistleblower', 'feud', '41st century', 'space opera', 'classical composer', 'dark fantasy', 'fairy', 'punk rocker', 'televangelist', 'chorus girl', 'voice over narration', 'freedom', 'catholic guilt', 'epilepsy', 'italian american', 'bust', 'tragic hero', 'apple', 'necromancy', 'sister act', 'whistling', 'wisecrack humor', 'e mail', 'evil spirit', 'electronic music score', 'fashion industry', 'publishing', 'sherlock holmes', 'police sergeant', 'babe scientist', 'colon in title', 'five word title', 'temper', 'purity', 'stepbrother', 'temptation', 'vagina dentata', 'british agent', 'kgb', 'soviet general', 'dead girl', 'granddaughter', 'morgue', 'storage unit', 'reference to superman', 'blueberry pie', 'pie', 'art model', 'nude model', 'three sisters', 'nude woman murdered', 'trailer home', 'accident', 'electronic devices', 'gynecologist', 'little black book', 'plan', 'set up', 'fabricated facts', 'stephen glass', 'the new republic', 'expecting', 'ejected from a moving vehicle', 'gun held to head', 'shot multiple times', 'strangulation', 'killing a pet', 'madame butterfly', 'semi autobiographical', 'estrangement', 'missouri', 'mormon temple', 'bathing a baby', 'cheese', 'middle school', 'courthouse', 'private investigator', 'krug', 'dangerous friend', 'monster as victim', 'pink bra and panties', 'red bra and panties', 'gang initiation', 'gunplay', 'hazing', "shakespeare's romeo and juliet", 'battle of the sexes', 'ex husband ex wife relationship', 'punching a heavy bag', 'tied to a tree', 'collar', 'yogi bear', 'brainwashing', 'construction worker', 'sunglasses', 'lama', 'tibetan', 'labor', 'labor union', 'master', 'oklahoma', 'tennessee', 'wal mart', 'funeral home', 'hermit', 'dead children', 'missing child', 'native', 'paraplegic', 'daydream', 'snow leopard', 'catering', 'wasp', '1190s', 'archer', 'king of england', 'robin hood', 'group', 'dance contest', 'loss of sister', 'step dancing', 'miscegenation', 'showboat', 'cmnf', 'cmnf scene', 'mermaid', 'emergency room', 'huguenot', 'medicis', 'protestant', 'katana sword', 'los angeles storm drain', 'evidence', 'fingerprints', 'murder weapon', 'blue bra and panties', 'girl in bra and panties', 'killer car', 'female alien', 'peacock', 'tigress', 'black u.s. president', 'tour', 'academy award', 'dove', 'nursing', 'pigeon', 'psychological torture', 'monster sex', 'blood vomiting', 'hit squad', 'dj', 'ecstasy the drug', 'rave', 'raver', 'warehouse', 'based on supposedly true story', 'paranormal investigating', 'easter island', 'statue', 'class differences', 'nursing home', 'summer love', 'blackjack', 'card counting', 'massachusetts institute of technology', 'mumbai riots', 'marine biologist', 'plumber', 'head on a stake', 'nuclear reactor', 'partisan', 'holy grail', 'mary magdalene', 'opus dei', 'charles darwin', 'cutlass', 'queen victoria', 'road', 'stupid victim', 'autobot', 'decepticon', 'job interview', 'angel dust the drug', 'disco', 'quaalude', 'reference to princess grace of monaco', 'covert analyst', 'stock market', 'abraham', 'sodom', 'delaware', 'erotica', 'drunk', 'religion versus science', 'titanic', 'wet', 'detective inspector', 'internal affairs', 'pet name in title', 'economic hardship', 'quadriplegic', 'wheelchair', 'barbed wire', 'barbed wire fence', 'navigator', 'army veteran', 'dignity', 'euthanasia', 'howie scream', 'bastille day', 'frog leg', 'the muppets', 'foot fetish', 'houseguest', 'air traffic control', 'female bodybuilder', 'grace', 'western u.s.', 'terror', 'ancient india', 'book of mormon', 'ray gun', 'dark past', 'spaghetti western', 'thomas jefferson', 'burned to death', 'witch hunter', 'female gunfighter', 'siege', 'child labor', 'hispanic', 'migration', 'hat', 'yellow', 'revisionist history', 'robe', 'unnecessary guilt', 'ashes', 'settler', 'wagon train', 'experimental surgery', 'regression', 'mascot', 'stupidity', 'hijack', 'commando unit', 'extra chromosome', 'first person shooter', 'hotel room', 'editor', 'new york times', 'puzzle', 'flying dagger', 'government', 'parapsychologist', 'in medias res', 'reference to planet of the apes', 'sex talk', 'darkness', 'kraken', 'corporate espionage', 'core', 'natural disaster', 'pacemaker', 'hidden truth', 'misogyny', 'new friend', 'abandoned asylum', 'abandoned hospital', 'asbestos', 'lobotomy', 'homeless', 'emotional balance', 'indonesia', 'inner peace', 'losing', 'pool', 'pool player', 'electrocuted', 'university', 'slip and fall', 'squirting orgasm', 'locked in a room', 'looking at self in mirror', 'night shift', 'parallel universe', 'unlikely friendship', 'illusionist', 'action figure', 'health care', 'tennis', 'scroll', 'tibetan monk', 'boy genius', 'futuristic city', 'woman pretending to be a man', 'police chief', 'tall woman', 'female surfer', 'swimsuit', 'krampus', 'chinatown san francisco', 'vault', 'indian american', 'spelling', 'spelling bee', 'life support', 'voice recording', 'black soldier', 'military uniform', 'abandoned by mother', 'sex trade', 'teenage prostitution', 'unpaid bill', 'full frontal nudity', 'group shower', 'police academy', 'police cadet', 'caravan', 'gangsta', 'gun fight', 'kitten', 'rock band', 'lecture', 'oxford university', 'symbol', 'drug addict', 'human android relationship', 'megacorporation', 'aging', 'billings montana', 'lincoln nebraska', 'adult diaper', 'diaper', 'sawed off shotgun', 'failure', 'partial female nudity', 'science experiment', 'haunted', 'haunting', 'paranormal investigator', 'flying saucer', 'aviator', 'based on book', 'the little prince', 'stanford university', 'diabetic', 'pharmaceutical lab', 'pitching', 'attempted suicide', 'collapsing scaffold', 'overflowing bathtub', 'tooth knocked out', 'glitter', 'reference to katharine hepburn', 'reference to spencer tracy', 'assumed identity', 'bolivia', 'butch cassidy', 'desk', 'oil refinery', 'belleville', 'tour de france', 'very little dialogue', 'baby rattle', 'anthropologist', 'forensics', 'animal experimentation', 'chimera', 'ambulance', 'paramedic', 'master apprentice relationship', 'female prisoner', 'self delusion', 'year 1954', 'psychiatric ward', 'mexican american war', 'u.s. frontier', 'british actor playing american character', 'origin of hero', 'bouncer', 'private dancer', 'giant squid', "liar's dice", 'la llorona', 'weeping', 'mission control', 'heaven and hell', 'vertigo comics', 'planet of the apes', 'tragedy', 'tragic love', 'black and white segues into color', 'color', 'infanticide', 'office politics', 'photo booth', 'snobbery', 'music therapy', 'united arab emirates', 'honeymoon', 'insurance company', 'baseball card', 'barn', 'piglet', 'giant food', 'sardine', 'marlboro', "lead actor's first film", 'public access', 'ashram', 'wiley college', 'reference to william shakespeare', 'white trash', 'blow job', 'agriculture', 'emotional abuse', 'rules', 'middle class', 'dna', 'cleveland browns', 'linebacker', 'nfl', 'team owner', 'centennial', 'next door neighbor', 'death of soldier', 'war trauma', 'crematorium', 'naked dead woman', 'poison gas', 'mammoth', 'valley', 'psychosis', 'foundry', 'foundry worker', 'caterer', 'bruise', 'teenage superhero', 'flashback within a flashback', 'split screen', 'self defense', 'murdered before giving protagonist information', 'part computer animation', 'prequel to cult film', 'head butt', 'phd', 'british royal family', 'queen elizabeth i', 'subconscious', 'nuclear war', 'stabbing', 'street life', 'case', 'dry humor', 'canada', 'human becoming an animal', 'part of trilogy', 'walrus', 'bridge construction', 'professor moriarty', 'scotland yard inspector', 'fish out of water', 'older man younger woman relationship', 'leather dress', 'law partner', 'seaman', 'drought', 'playground', 'skateboarding', 'male in shower', 'conservatory', 'music school', 'ghost story', 'manson family', 'real life event', 'reference to sharon tate', 'jewish american princess', 'based on a book', 'based on a novel', 'made for tv movie', 'support', 'sketch comedy', 'gotham', 'fbi director', 'guinea pig', 'household appliance', 'talking mouse', 'headmaster', 'oxford', 'train crash', 'great barrier reef', 'short term memory loss', 'american in paris', 'embarrassment', 'revelation', 'lost', 'masseuse', 'threesome', 'bolshevik', 'russian revolution', 'prosecutor', 'carpenter', 'christmas song', 'leave of absence', 'sunflower', 'lieutenant colonel', 'alibi', 'newspaper clipping', 'profiler', 'campy', 'castle thunder', 'gulf war veteran', 'shot in the head', 'distrust', 'dodgeball', 'portrait', 'soccer game', 'bean', 'duology', "pandora's box", 'battalion', 'skateboarder', 'surf', 'goblin', 'freak accident', 'hit in the crotch', 'nosy neighbor', 'on the job injury', 'childhood home', 'human animal relationship', 'waiting', 'pot', 'smoke', 'chun li', 'crime fighter', 'political drama', 'washington post the newspaper', 'time travel romance', 'cola', 'ice cream cone', 'playboy mansion', 'african americans', 'boxing gym', 'rumble in the jungle', 'hunting', 'minister', 'cenobites', 'roof', 'tank', '1st century', 'based on web series', 'claustrophobia', 'phobia', 'football movie', 'high school sweethearts', 'sperm', 'sperm donor', 'dead man', 'killing a lion', 'killing a tiger', 'softball', 'alienation', 'conscience', 'insomnia', 'urban gothic', 'jihad', 'mindanao', 'philippines', 'squatter', 'injection', 'scalpel', 'stabbed in the mouth', 'bildungsroman', 'flying broom', 'invisibility cloak', 'wizardry', 'courier', 'delivery', 'envelope', 'jewish nazi', 'nazism', 'store', 'trying to conceive', 'yellow panties', 'mix up', 'stutter', 'opposites attract', 'block party', 'marching band', 'children adventure', 'sea goddess', 'tsunami', 'birding', 'hipster', 'narcotics', 'art of war', 'lift skirt', 'short shorts', 'broken nose', 'marriage proposal', 'sistine chapel', 'alderman', 'presidential candidate', 'language', 'male masturbation', 'central asia', 'russian empire', 'defense', 'mothership', 'arms tied overhead', 'brooklyn bridge', 'company', 'corporate takeover', 'executive', 'new boss', "diver's watch", 'coroner', 'gay son', 'mistaken identity', 'forensic', 'boxing arena', 'boxing manager', 'directed by co star', 'wrongful conviction', 'museum of natural history', 'night watchman', 'watchman', 'housekeeper', 'manor', 'service', 'dodge viper', 'the star spangled banner', 'bound and gagged', 'hearing noises', 'uninterrupted shot', 'bengali', 'reference to gogol', 'interracial kiss', 'interracial sex', 'automobile racing', 'nascar', 'street racing', 'television station', 'tv show', 'adult as child', 'janitor', 'ice cream', 'second city', 'critique of capitalism', 'investment bank', 'market economy', 'plutocracy', 'naivety', 'gods', 'aurora borealis', 'male in underwear', 'evil politician', 'director', 'psychosomatic blindness', 'self deprecating', 'studio', 'psychotronic film', 'public hearing', 'persian', 'xerxes', 'dare', 'exclamation point in title', 'insurance salesman', 'reference to britney spears', 'world tour', 'father in law', 'ex convict', 'older brother is bad influence', 'washington heights manhattan new york city', 'girl scouts', 'quitting a job', 'death of friend', 'severed arm', 'single camera', 'subtle comedy', 'theatrical producer', 'tutu', 'ingratitude', 'hairdresser', 'reference to liberace', 'armored car', 'falling out among thieves', 'planning', 'computer hacker', 'rise and fall', 'estate', 'monte carlo', 'studio ghibli', 'cameraman', 'epic battle', 'south ossetia', 'actual animal killed', 'evil sorcerer', 'warlord', 'junkyard', 'guitarist', 'obsessed fan', 'home', 'police surveillance', 'surveillance van', 'tailing a suspect', 'venice italy', 'patricide', 'developer', 'medical ethics', 'pills', 'serbian', 'black history', 'nation of islam', 'dole', 'male stripper', 'northern england', 'steel worker', 'art dealer', 'fake painting', 'macguffin', 'moustache', 'stolen painting', 'health insurance', 'long delayed release', 'nail in the head', 'bayou', 'swamp', 'pacific northwest', 'small town sheriff', 'snowmobile', 'incest', 'son murders mother', 'brawl', 'computer cracker', 'intelligence agency', 'terrorist cell', 'south orange new jersey', 'woman in underwear', 'bound with duct tape', 'flash forward', 'riddle', 'abusive relationship', 'tenderloin san francisco', 'law clerk', 'world trade center manhattan new york city', 'woodstock', 'black smoke', 'japanese military', 'japanese navy', 'sunday', 'hotel manager', 'cleveland indians', 'league', 'owner', 'clowning', 'polo', 'possum', 'abusive parent', 'child molester', 'child protection', 'sex addict', 'erectile dysfunction', 'mugger', 'mugging', 'subway robbery', '12 year time span', 'separated parents', 'art expert', 'nazi stolen art', 'monarchy', 'princess diana', 'swastika tattoo', 'high school girl', 'doc holliday', 'robbers', 'three of spades tattoo', 'weapon', 'curtain call', 'dream girl', 'satanic cult', 'ardennes forest', 'battle of the bulge', 'behind enemy lines', 'world war two in belgium', 'based on young adult novel', 'human alien', 'jazz', 'jazz music', "texas hold 'em", 'thong', 'wrestler', 'bermuda triangle', 'shipwrecked', 'travel writer', '70mm film', 'biblical epic', 'king herod', 'dead soldier', 'impersonating a police officer', 'briefcase full of money', "character repeating someone else's dialogue", 'reference to cinderella', 'genitalia', 'black and white photograph', 'drawing with blood', 'falling through ice', 'playing piano', 'red haired woman', 'boyfriend', 'nintendo ds', 'game designer', 'pod', 'foot chase', '8 year old', 'convention', 'magic wand', 'male fairy godmother', 'inner beauty', 'climbing a tree', 'courage', 'radiation sickness', 'radioactive contamination', 'test launch', 'surveillance footage', 'timebomb', 'natural history museum', 'news reporter', 'shooting', 'cbs news', 'tobacco', 'transformation', 'young boy', 'multi protagonist', 'hair stylist', 'landlord', 'palestinian', 'aerialist', 'cirque du soleil', 'learning disability', 'tutor', 'flashing breasts', 'hitchhiker', 'interpol', 'bangkok thailand', 'forks washington', 'protecting a woman', 'title same as book', 'breath on mirror', 'engagement ring', 'guard', 'swordsman', 'lawsuit', 'loud sex', 'admiral', 'thunderstorm', 'barbecue', 'non professional cast', 'cross country trip', 'boat racing', 'hydroplane', 'scolding', 'oil field', 'oil tycoon', 'wildcatter', 'catholicism', 'autocracy', 'based on real events', 'dictatorship', 'exploding airplane', 'fighter pilot', 'hands tied', 'shaft', 'caver', 'dark', 'radical', 'triple crown', 'parachute', 'detainee', 'guantanamo bay', 'infant', 'sports violence', 'father', 'orangutan', 'barrio', 'newcastle united', 'greenhouse', 'old house', 'long con', 'australian aborigine', 'australian outback', 'virginia', 'thrown through a window', 'sadist', 'reading', 'futuristic', 'based on manga', 'gifted child', 'post thermonuclear war', 'music copyist', 'music student', 'musical composition', 'symphony', 'cryptography', 'enigma code', 'instructor', 'animal companion', 'shakespeare adaptation', 'star crossed lovers', '1890s', 'hired gun', 'johnson county war', 'gardening', 'commander', 'african american policeman', 'policewoman', 'beauty salon', 'black college', 'government conspiracy', 'oppression', 'four brothers', 'crown jewels', 'sex change', 'diane arbus', 'steel mill', 'fall to death', 'chameleon', 'western town', 'whipped cream', 'miniaturization', 'sudden change in size', 'monologue', 'television network', 'television news', '12 year old', 'blade', 'anchorage alaska', 'investment', 'investment banker', 'diplomat', 'escapade', 'gallipoli', 'world war one', 'blonde', 'law', 'law school', 'kumite', 'elvis impersonator', 'rascal', 'talk radio', 'vomiting blood', 'rain', 'rain forest', 'gay parent', 'jiminy cricket', 'pinocchio', 'ice pick', 'pitchfork', 'shack', 'lucha libre', 'wrestling', 'damselfish', 'ocean habitat', 'ocean life', 'razorfish', 'spider crab', 'devil worship', 'incubus', 'raped by a demon', 'satanism', 'baron', 'nickname', 'transylvania', 'igloo', 'estranged daughter', 'iranian', 'phone booth', 'dangerous job', 'hurricane', 'school bus', 'shark feature', 'foster home', 'murder investigation', 'elvis', 'iron curtain', 'reality spoof', 'algerian war', 'graveyard', 'murder trial', 'gang that lives together', 'outlaw gang', 'blacksmith', 'gunslinger', 'mumblecore', 'frozen', 'health food', 'penis joke', 'phallic image', 'sexy agent', 'volcano hideout', 'alien race', 'dishwasher', 'flashing', 'integrity', 'responsibility', 'obedience', 'family restaurant', 'remarriage', 'female killer', 'bar fight', 'drug abuse', 'interracial rape', 'booty shake', 'duct tape over mouth', 'female to male foot in crotch', 'alien infection', 'liquid metal', 'multiple cameos', 'sexy woman', '1770s', 'british india', 'female criminal', 'hooker', 'concrete wall', 'open ended', 'electric chair', 'blob', 'ferris wheel', 'drumline', 'multiverse', 'apatosaurus', 'asteroid', 'screwball comedy', 'artificial heart', 'horse penis', 'mayhem', 'rowing', 'artifact', 'incestuous relationship', 'card trick', 'rain machine', 'vigilantism', 'drawn and quartered', 'torture chamber', 'torture device', 'prescription drugs', '911 operator', 'calling 911', 'head held underwater', 'white villain', 'holocaust', 'warsaw ghetto', 'dark humor', 'female antagonist', 'christ', 'cnn reporter', 'matrix', 'sugarcane', 'mental patient', 'scheme', 'battle of the bands', 'school teacher', 'substitute teacher', 'dealer', 'claim in title', 'jazz singer', 'time manipulation', 'middle earth', 'thong bikini', 'tv dance show', 'muggle', 'first love', 'milf', 'card dealer', 'criminal mastermind', 'recruiting', 'ulcer', 'arena', 'boxing movie', 'robot battle', 'brazil', 'soccer movie', 'world cup', 'soccer star', 'mental breakdown', 'post september 11 2001', 'dome', 'the simpsons', 'rap music', 'ivy league', 'failing student', 'payphone', 'telephone booth', 'business tycoon', 'mayoral candidate', 'penthouse', 'reference to star trek', 'reconciliation', 'bush administration', 'hijab', 'reference to mahmoud ahmedinejad', 'jewell', 'mongol', 'nostradamus', 'stepnicka', 'vallone', 'great dane', 'animal control', 'lost animal', 'mongrel', 'pet', 'terrier', 'bayonet', 'college kid', 'hillbilly', 'flying superhero', 'phantom zone', 'challenge', 'drift racing', 'drifting', 'foreign exchange student', 'incestuous desire', 'incestuous kiss', 'bag over head', 'buddhist temple', 'loss of friend', 'male bonding', 'black servant', 'chocolate pie', 'jackson mississippi', 'writing a novel', 'backstage', 'racial slur', 'not taking no for an answer', 'obsessive woman', 'animal as human', 'sheepdog', 'mine', 'plutonium', 'ash', 'pokemon', 'amateur theater', 'travel agent', 'south pole', 'country house', 'producer', 'shooting party', 'boxing match', 'montage', 'philadelphia', 'enema', 'rocky mountains', 'dog hit by a car', 'female in shower', 'unhappily married woman', 'argentina', 'meth', 'meth lab', 'speed freak', 'mussolini', 'toy company', 'ark', 'change', 'depiction of god', 'saigon', 'bambi', 'college basketball', 'national championship', 'ncaa', 'film editor', 'movie screening', 'columbia university', 'bong', 'american actress playing british character', 'book publishing', 'calorie counting', 'rabbit costume', 'construction', 'female agent', 'waterfront', 'lying', 'reference to twitter', 'reference to youtube', 'toronto', 'blizzard', 'buddhism', 'evolution', 'reincarnation', 'stuck in elevator', 'malaysia', 'male model', 'credit card fraud', 'miami', 'returning character with different actor', 'slasher flick', 'based on opera', 'walking wounded', 'documentary filmmaker', 'documentary filmmaking', 'generation z', 'flint michigan', 'merger', 'uprising', 'reference to snow white', 'sleeping bag', 'stepmother stepdaughter relationship', 'totem pole', 'probation', 'hit and run', 'police investigation', 'volleyball', 'car racing', 'racing', 'duplex', 'tenant', 'dishonor', 'masquerade party', 'flea', 'space dog', 'baby boy', 'affection', 'black horse', 'boy horse relationship', 'rescued by a horse', 'dognapping', 'misfiring gun', 'shih tzu', 'stake', 'what happened to epilogue', 'football coach', 'big brother', 'community service', 'live action role playing', 'peter pan syndrome', 'amazing grace the hymn', 'skidding truck', 'neck breaking', 'wraparound story', 'gasoline', 'refinery', 'traffic jam', 'cowboy hat', 'grandparent grandchild relationship', 'reference to facebook', 'reference to the internet', 'full frontal male nudity', 'multiple characters voiced by same person', 'sex in hotel room', 'animal attack', 'french riviera', 'speaker of the house', 'comic relief', 'state flag', 'real estate agent', 'lancer', 'free running', 'james bond', 'tintin', 'ends with quotation', "women's prison", 'year 1964', 'all girl band', 'cross dressing', 'arc de triomphe paris', 'based on autobiography', 'elle magazine', 'goddess', 'sinbad', 'last day of school', 'comeback', 'racer', 'blood of christ', 'firefly', 'mysterious event', 'mount everest', 'mountain climbing', 'teaching oneself to read', 'war crime', 'giant monster', 'kaiju', 'family drama', 'political campaign', 'social decay', 'mormon missionary', 'illustrator', 'chewing gum', 'translation problems', 'chocolate factory', 'businesswoman', 'bruce lee', 'jeet kune do', 'letter of resignation', 'advertising executive', 'commercial', 'lion', 'railway', 'barista', 'coffee', 'coffee shop', 'boot camp', 'fictional talk show', 'phony suicide note', 'dragonfly', 'soaking wet', 'supervillian origin', 'romantic rivalry', 'rule', 'elvis presley', 'cosmonaut', 'footprint', 'eastern philosophy', 'healing', 'reference to george washington', 'ritual sacrifice', 'drag queen', 'physical appearance', 'leonardo da vinci', 'nuclear waste', 'deformed', 'roller derby', 'bank fraud', 'prey', 'best friends', 'reference to matthias steiner', 'railway station', 'magical potion', 'time travel comedy', 'rape victim', 'sexual humiliation', 'classroom', 'fund raising', 'roman', 'theater', 'cheerleader uniform', 'gang banger', "shakespeare's hamlet", 'celibacy', 'synagogue', 'yellow ribbon', 'joy', 'running away', 'sadness', 'campus', 'arcade game', 'driving in reverse', 'video gamer', 'hit by a pickup truck', 'resuscitation', 'greek restaurant', 'travel agency', 'anniversary', 'cellist', 'hardware store', 'camcorder', 'lost passport', 'black ops', 'repeated scene', 'black pantyhose', 'female stockinged legs', 'flubber', 'flying rubber', 'drug overdose', 'gang war', 'adult child friendship', 'acrophobia', 'editing', 'role playing', 'buick', 'last will and testament', 'savant', 'lifted by the throat', 'star trek', 'vulcan', 'new school', 'dragon rider', 'teenage hero', 'boy in a bathtub', 'laptop computer', 'pulled underwater', 'walking in the dark', 'operative', 'mushroom', 'mute', 'roman soldier', 'screaming', 'wood carving', 'college student', 'disfigured face', 'physical therapist', 'retrofuturism', 'living dead', 'blue ribbon', 'jacket', 'jock', 'yogurt', 'coal mine', 'israeli', 'hearing characters thoughts', 'feminism', 'kicked in the testicles', 'paintball', 'paintball gun', 'pepper spray', 'based on multiple works', 'mysterious stranger', 'mail order bride', '15 year old', 'corndog', 'funeral in the rain', 'weatherman', 'high definition television', 'reverse chronology', 'boxing training', 'heavy rain', 'anilingus', 'double cross', 'two timing', 'chihuahua', 'potted plant', 'tijuana mexico', 'desi', 'heritage', 'sail', 'smoker', 'plastic surgeon', 'woman wearing a string bikini', 'exhibition', 'gilbert and sullivan', "gilbert and sullivan's princess ida", 'southern accent', 'title based on the bible', 'brunette', 'film producer', 'liar', 'prank', 'secret government agency', 'troll', 'coin toss', 'tracking device', 'bear attack', 'lingerie model', 'm.i.b.', 'pug', 'geisha', 'chess', 'american soldier', 'naval uniform', 'patrol boat', 'uniform', 'female on male rape', 'lent', 'ice storm', 'first crush', 'social outcast', 'taunting', 'ramanujan', 'nazis', 'bullying', 'cyberbullying', 'septicemia', 'radio show', 'alone', 'aron ralston', 'canyon', 'binge drinking', 'psychic power', 'psychometry', 'ski mask', 'stolen money', 'face', 'face transplant', 'brother in law', 'forgiveness', 'job offer', 'slow motion action scene', 'slow motion sequence', 'crime lord', 'dead body', 'dead body in a freezer', 'truck explosion', 'reference to anastasia romanov', 'romanov', 'promise', 'introduction', 'tron', 'summer job', 'gay cop', 'getaway driver', 'witness protection', 'algeria', 'algerian', 'mario', 'reward', "st trinian's", 'abstract', 'double life', 'rogue agent', 'siberian husky', 'sled dog', 'macabre', 'uncle fester', 'flyer', 'champagne', 'nuremberg', 'alien planet', 'promised land', '20 years later', 'ford mustang', 'james bond 007', 'yanked off bikini top', 'd day', 'normandy', 'rollerskating rink', 'spelman college', 'penknife', 'united kingdom', 'press', 'clock', 'mad hatter', 'fishing', 'fishing trip', 'golden age', 'midnight', 'student film', 'wonder years', 'museum of natural history manhattan new york city', 'tablet', 'champion', 'golf tournament', 'nostalgia', 'maid of honor', 'female therapist', 'therapy session', 'casper', 'friendly ghost', 'kung fu classic', 'industrialist', 'murder disguised as suicide', 'reference to howard university', 'norad', 'road rage', 'hobo', 'rampage', 'english teacher', 'coronation', 'swashbuckler', 'premier', 'alien contact', 'death of girlfriend', 'ouija', 'head cut off', 'necklace', 'safecracker', 'wine dealer', 'godzilla', 'yakuza', 'new orleans louisiana', 'execution by hanging', 'los angeles police department', 'miscarriage of justice', 'convoy', 'crow', 'cynicism', 'false promise of the american dream', 'potion', 'sanctuary', 'beauty', 'bosnia', 'war criminal', 'buenos aires argentina', 'jaguar', 'mayan', 'solar eclipse', 'killing spree', 'one night', 'authority figure as villian', 'gun duel', 'saved from hanging', 'dart board', 'ladder factory', 'cryogenics', "boys' school", "girls' soccer", 'year 2054', 'groupie', 'rock n roll', 'dialectics', 'eiffel tower at night', 'implied insemination', 'pfizer', 'viagra', 'zoloft', 'shot in the crotch', 'mourning', 'ball', 'steam', 'female scientist', 'caribbean', 'anchor', 'chorus line', 'gala premiere', 'environmental', 'oil spill', 'oktoberfest', 'bouquet', 'house fire', 'praying', 'year 1959', 'new hampshire', 'linguistics professor', 'reference to angels in america the play', 'ogre', 'nosebleed', 'inspired by documentary', 'presidential campaign', 'jamming communications', 'space station', 'taunt', 'fourth of july', 'breast', 'rap star', 'reference to tupac shakur', 'blocked road', 'zookeeper', 'batman', 'sequel to a reboot', 'sister love', 'snowman', 'horror movie prequel', 'paranormal activity', 'self sacrifice', 'hibernation', 'dead body wrapped in carpet', 'school janitor', 'criminal organization', 'kgb agent', 'cedar rapids iowa', 'musical quartet', 'filmmaker', 'anticipate', 'dead in absentia', 'foetus', 'unreliable narration', 'phone sex', 'queens new york city', 'sex addiction', 'animal abuse(non graphic)', 'cat killer', 'dead cat', 'death of cat', 'jazz age', 'vaudeville', 'delivery man', 'military police', 'rio de janeiro brazil', 'miami florida', 'murder of family', 'girl next door', 'growing vegetables', 'high standards', 'sent away', 'tennis coach', 'rise to power', 'addict', 'tv reporter', 'warp speed', 'garage', 'narcotics anonymous', 'recovery', 'dick tracy', 'forger', 'passport', 'venture capital', 'slave labor', 'house construction', 'death star', 'empire', 'galactic war', 'toddler', 'teen horror', "noah's ark", 'wrath of god', 'market', 'white picket fence', 'gallantry', 'honesty', 'rape and revenge', 'performing arts school', 'based on article', 'massage therapist', 'regaining sight', 'champagne bottle', 'coca cola', 'jewelry box', 'red bull', 'shaolin', 'political prisoner', 'bankruptcy', 'crystal', 'microwave oven', 'alternate history', 'future time travel', 'adidas', 'waste container', '21st century', 'ear wax cleaning', 'fake newsreel', 'paddington station london', 'taxidermist', 'luck', 'masquerade', 'kryptonite', 'smallville', 'macau', 'selfie', 'prodigy', 'rhapsody', 'hall of mirrors', 'mont saint michel france', 'palace of versailles', 'turkey the country', 'yellowstone national park', 'baseball scout', 'hit on the head with a ball', 'sexy', 'wild horse', 'bombardier', 'loss of wife', 'female female kiss', 'outdoor sex', 'banana boat', 'chips', 'fictional government agency', 'vatican', 'dauphin', 'wartime rape', 'creation', 'paddock', 'supercross', 'winning', 'dentures', 'false teeth', 'auctioneer', 'sorcerer', 'abusive marriage', 'prison wagon', 'roast pig', 'suicide by hanging', 'christian propaganda', 'zealot', 'hostility', 'ultimate fighting championship', 'underdog', 'marijuana joint', 'porn magazine', 'sex obsession', 'breasts growing', 'coors light', 'spinning in grave', 'berlin wall', 'specialist', 'overachiever', 'the flintstones', 'title appears in song', 'work', 'controlled environment', 'fictional reality show', 'innocence lost', 'simulated reality', 'ice hockey', 'turkish bath', 'rags to riches', 'secret letter', 'squire', 'injury', 'hybrid', 'scream', 'disaster movie', 'san andreas fault', 'aging rocker', 'cobbler', 'art exhibition', 'augmented reality', 'speculative fiction', 'hide and seek', 'domineering mother', 'stamp', 'eurocopter ec120 colibri', 'fortune cookie', 'lottery ticket', 'sex lesson from a prostitute', 'spreadeagle', 'topless', 'boxing knockout', 'implied sex', 'keys', 'half brothers', 'white raised as indian', 'breath holding contest', 'plantation', 'pipeline', 'sexual ambiguity', 'sheik', 'armageddon', 'end times', 'reference to satan', 'film in film', 'reference to steven spielberg', 'self referential', 'shopping mall', 'sports team', 'title based on song', 'sydney australia', 'computer virus', 'fight the system', 'locomotive', 'steam locomotive', 'planet invasion', 'year 2046', 'drowning', 'hanging from a flagpole', 'hanging from a rope', 'whiskey', 'surprise', 'tesseract', 'war bond', 'business trip', 'glory hole', 'loser', 'chop socky', 'bahamas', 'hook', 'beverly hills california', 'soap opera', 'erotic thriller', 'marital infidelity', "wife's lover murdered by husband", 'buffalo new york', 'embalming', 'mortuary', 'fiancee', 'animate doll', 'rebirth', 'cafeteria', 'bobby darin', 'sandra dee', 'contortionist', 'gag humor', 'human cannonball', 'perfume', 'radio dj', 'radio station', 'vinyl', 'coming out', 'gay brother', 'toilet sex', 'harem', 'reference to vanilla ice', 'statutory rape', 'teacher student relationship', 'teacher student sex', 'bad taste humour', 'crude humor', 'self obsession', 'cabinet meeting', 'catholic church', 'borneo', 'orchid', 'colonoscopy', 'washington square manhattan new york city', 'based on ed gein', 'hyperactive', 'the doors', 'financial crisis', 'cuban flag', 'fidel castro', 'havana cuba', 'alliterative title', 'breaking down a door', 'haunted hotel', 'identical twins', 'twin sisters', 'selma alabama', 'u.s. politics', 'park ranger', 'drunkenness', 'religious mother', 'strict mother', 'war hammer', 'cb radio', 'arrow', 'imperial seal', 'gratitude', 'upholsterer', 'serial rapist', 'safe', 'parallel worlds', 'secret door', 'bloody mary the drink', 'rules of profession', 'padded cell', 'monster slayer', 'domino', 'fiery redhead', 'nassau', 'underwater battle', 'helicopter', 'stubbornness', 'reference to frankenstein', 'reference to jimi hendrix', 'studio logo segues into film', 'white dinner jacket', 'wien', 'fashion show', 'falling off a roof', 'nypd', 'compassion', 'drive in', 'woman shot in the forehead', 'hungary', 'diarrhea', 'middle aged man', 'sexy nurse', 'balloon', 'skeleton', 'secret base', 'syracuse university', 'guest', 'car breakdown', 'jeep', 'halloween party', 'romcom', 'national spelling bee', 'school principal', 'alien creature', 'body snatching', 'parasite', 'han river', 'seoul', 'happiness', 'tuskegee airmen', 'patrol', 'decoder', 'sexism', 'underage sex', 'promiscuous daughter', 'sex in a bed', 'killed in action', 'child with gun', 'invulnerability', 'population', 'state in title', 'southern gothic', 'newsletter', 'elderly man', 'medical', 'transhumanism', 'sausage', 'sleep', 'portugal', 'switzerland', 'deformed baby', 'premature aging', 'long lost brother', 'marriage bureau', 'professional rivalry', "president's daughter", 'legs', 'cult director', 'drive in classic', 'falling in love with a prostitute', 'rich man poor woman', 'tuition', 'knife held to throat', 'hoop', 'chat room', 'identity theft', 'housing project', 'trapped in an elevator', 'oil drilling', 'kleptomaniac', 'self destructiveness', 'student council president', 'student government', 'water boy', 'case file', 'psycho thriller', 'self expression', 'white rapper', 'aviation', 'spruce goose', 'test flight', 'rocker', 'u2', 'halleys comet', 'life force', 'space vampire', 'suicide forest', '1988 winter olympics', 'ski jumper', 'winter olympics', 'bombing mission', 'munitions', 'royal flying corps', 'surrogate mother', 'homecoming queen', 'insecurity', 'alien probe', 'humpback whales', 'probe', 'female assassin', 'office cubicle', 'gnosticism', 'rare book', 'villain arrested', 'divorce lawyer', 'storyteller', 'time paradox', 'retired boxer', 'airfield', 'car stunt', 'private jet', 'menstrual blood', 'nursing school', 'performing arts', 'sorceress', 'dream world', 'lake tahoe nevada', 'melancholy', 'cold fusion', 'fusion', 'the saint', 'hmo', 'las vegas', 'german expressionism', 'basic training', "reference to shakespeare's hamlet", 'side saddle', 'adult returns home', 'girl with glasses', 'when animals attack', 'boulder', 'villa', 'belief in the afterlife', 'stigmata', 'wound', '1959 cadillac', 'diamonds', 'female villain', 'force field', 'child abduction', 'genius recluse', 'text messaging', 'beverly hills hotel', 'troop', 'easter', 'easter bunny', 'piaggio vespa gtv 250', 'gun running', 'terrorist attack', 'death of main character', 'eiffel tower paris', 'two suitors', 'hit with a door', 'pit boss', 'orange county california', 'ex con', 'young woman', 'friday the thirteenth', 'murder spree', 'class president', 'statue of liberty', 'television van', 'jordan', 'wire fu', 'georgia', 'trailer', 'pakistani', 'thatcher', 'tooth', 'bodybuilder', 'punch into the camera', 'reference to julia roberts', 'song and dance', 'arsonist', 'blood splatter on face', 'cave bear', 'neanderthal', 'return', 'egyptologist', 'mummy', 'exam', 'northeast region of brazil', 'portuguese language', 'eyeglasses', 'woman slaps a man', 'woman slaps man', 'hamburger', 'woman in a bikini', 'toad', 'dirty cop', 'limping', 'individualism', 'arkansas', 'jesus crust pizza parlor', 'burned alive', 'olympus', 'dog fight', 'interracial romance', 'football game', 'coastline', 'capital management', 'financial disaster', 'investment fraud', 'management', 'somali', 'luminol', 'omen', '1st century b.c.', 'lightsaber', 'local blockbuster', 'existentialism', 'catacomb', "philosopher's stone", 'blindfold', 'automaton', 'mechanical', 'clark kent', 'year 1978', 'congo', 'laser', 'lost city', 'art director', 'riding a horse', 'african american lesbian', 'lesbian couple', 'giving birth', 'ultrasonography', 'crotch shot', 'disney spoof', 'one woman army', 'catwoman', 'riddler', 'brighton', 'saloon', 'saloon singer', 'musical filmmaking', 'tango', 'alternate world', 'fairy tale parody', 'island name in title', 'haute cuisine', 'indian cuisine', 'michelin guide', 'evil scientist', 'groundhog day', 'mothman', 'point pleasant west virginia', 'washington d.c. lobbyist', 'entertainer', 'carried in a chair', 'rite of passage', 'mysterious man', 'rhythm and blues', 'dominance submissiveness', 'human pet', 'power exchange', 'nipples visible through clothing', 'pronounced dead', 'sword duel', 'austin texas', 'pap smear', 'hawk', 'lady', 'planet enclosed within shield', 'sci fi spoof', 'solitude', 'football video game', "talking to one's self in a mirror", 'friar', 'inquisition', 'peru', 'rope bridge', 'pain', 'expecting twins', 'chaucer', 'jousting', 'looking at the camera', 'dreaming', 'contraband', 'bird in a cage', 'reunited family', 'poetess', 'ecuador', 'theatre director', 'goat', 'naked woman', 'joker', 'tattoo on breast', 'the joker', 'prawn', 'refugee camp', 'cabinet', 'temptress', 'meteorite', 'cold calculation', 'caregiver', 'department store', 'imprisonment', 'lawlessness', 'hit by a car', 'living statue', 'melting woman', 'food poisoning', 'health inspector', 'rationing', 'vampire society', 'year 2019', 'general motors', 'shangri la', 'treasure hunt', 'dorset', "women's land army", 'nobel prize', 'kidnapper', 'mountaineering', 'scottish highlands', 'nude black woman', 'asylum', 'full moon', 'duplicate earth', 'synchronicity', 'grandson', 'stonewall riots', 'slave revolt', 'call center', 'seminary', 'iron lung', 'sex surrogate', 'bully comeuppance', 'destruction', 'nuclear missile', 'tears', 'opening night', 'swedish', 'gold mine', 'surveyor', 'beavis and butt head', 'score', 'television set', 'autograph', 'journey to the west', 'monkey king', 'talking to inanimate object', 'orga', 'male friendship', 'muscular', 'starvation', 'white castle', 'sertao', 'chinese american', 'lesbian love', 'overweight', 'anesthetic', 'anesthetic awareness', 'heart surgery', 'boxer rebellion', 'peking china', 'quick stop', 'pun', 'gardener', 'oscar wilde', 'pigalle', 'archery', 'defiance', 'toilet training', 'older woman younger woman relationship', 'sex with student', 'avoidant personality disorder', 'emotionally vulnerable', 'video store clerk', 'dunes', 'socialist', 'critic', 'theater critic', 'firm', 'hooliganism', 'west ham supporter', 'yukon', 'minimal dialogue', 'decapitation', 'scavenger hunt', 'sleepover', 'punched in the crotch', 'sparring', 'moral dilemma', 'armored truck', 'husband murders wife', 'overhead shot', 'desert storm', 'fragmentation grenade', 'woman kills attacker', 'man wearing a jock strap', 'man wearing a thong', 'rear male nudity', 'stripped prison uniform', 'lunar eclipse', 'crusades', 'modern minstrel', '10 year old', 'grumpy old man', 'school presentation', 'stroke', 'stroke recovery', 'film school', 'ex soldier', 'chief of staff', 'imdb', 'mediaatlarge.com', 'mediaatlargellc', 'dress', 'fairy godmother', 'pumpkin', 'press conference', 'chatting', 'missing person poster', 'online dating', 'sexual predator', 'mini dress', 'imaginary son', 'vampire slayer', 'clan', 'year 1743', 'fart joke', 'male nipple', 'human versus alien', 'death of loved one', 'nero', 'diva', 'open marriage', 'social climber', 'gambling debt', 'off screen murder', 'cinderella story', 'correspondence', 'evil step mother', 'family secret', 'tour bus', 'bad cop', 'school nurse', 'arachnophobia', 'crystal ball', 'outrage', 'walking through a wall', '1860s', 'schoolteacher', 'siam', 'jewish family', 'based on anime', 'speed', 'irish accent', 'torn between two worlds', 'artificial organ', 'enforcer', 'hippie', 'music festival', 'the who', 'federal agent', 'year 1997', 'divorced parents', 'exploding car', 'black panties', 'perfect man', 'secret admirer', 'lex luthor', 'lois lane', 'return to earth', 'capoeira', 'freeze', 'queen of sheba', 'butterfly', 'entomologist', 'morpho butterfly', 'indian army', 'hot temper', '1955 chevrolet', 'peeping tom', 'reference to brooklyn dodgers', 'spying', 'tire iron', 'parenthesis in title', 'british military', 'ottoman empire', 't.e. lawrence', 'queen of england', 'tall man', 'princeton university', 'working women', 'finger gun', 'killed in an elevator', 'december', '35 mm digital camera', 'anthropophagus', 'blood and gore', 'double feature', 'real twins playing twins', 'turkey', 'tunisia', 'debunking', 'driving through a gate', 'trade union', 'flash of genius', 'mental instability', 'sudden infant death syndrome', 'first date', 'harlem manhattan new york city', 'hello kitty', 'comic book artist', 'genie', 'reference to a thousand and one nights', 'carjacking', 'johannesburg south africa', 'post apartheid', 'singer songwriter', 'male anorexia', 'gremlin', 'horror host', 'hulk hogan', 'brothers hugging', 'parole violation', 'sex in a car', 'sex video', 'sexual desire', '2030s', 'jet', 'grace kelly', 'hollywood star', 'childhood trauma', 'dog show', 'gay character', 'gay couple', 'news leak', 'whistleblowing', 'wikileaks', 'eastern europe', 'ghost rider', 'loudmouth', 'money laundering', "shakespeare's a midsummer night's dream", 'kremlin', 'jumping on a bed', 'librarian', 'pregnant woman in bathtub', 'marshal', 'future war', 'shootout in a police station', 'sabbath', 'ten stories', 'mediterranean', 'leatherface', 'coast guard', 'oil tanker', 'sinking ship', 'forbidden sex', 'sex with a student', 'taboo sex', 'columnist', 'los angeles times', 'wearing a traffic cone as a hat', 'efficiency expert', 'insurance investigator', 'sheffield', 'yorkshire', 'pop star', 'head dunked in water', 'military school', 'plastic bag', 'playing a video game', 'suffocating', 'gay sibling', 'stalingrad', 'american revolution', 'standoff', 'female basketball player', 'personal vendetta', 'reference to ralph nader', 'live television', 'burlesque', 'small town girl', 'italian stereotype', 'man in a swimsuit', 'verona italy', 'university of oregon', 'basketball game', 'monstars', 'college roommate', 'arch villain', 'mexican immigrant', 'network', 'pine forest', 'team building', 'maryland', 'film student', 'street dancing', 'girlfriend in a coma', 'prosthetic leg', 'pawn', 'mysterious island', 'art collector', 'bull riding', 'actor name in title', 'anal sex', 'claw', 'inspector', 'order', 'beekeeping', 'insect in title', 'cleaning a swimming pool', 'male male kiss', 'talking dog', 'game tester', 'snack', 'private eye', 'half human', 'rebellious daughter', 'pipe bomb', 'gangsta rap', 'nwa', 'animator', 'bamboo cane', 'beekeeper', 'false memory', 'implanted memory', 'space colony', 'earl of oxford', 'lost in the woods', 'screaming in horror', 'meddling', 'sobbing female', 'subtitled scene', 'you can say that again joke', 'narcolepsy', 'traveling', 'train robbery', 'answering machine', 'home movie', 'lithium', 'video diary', 'argentine', 'haunted hospital', 'drill in the eye', 'eye gouging', 'hanged woman', 'peep hole', 'based on a true story', 'accidental kiss', 'eating a worm', 'worm', 'close encounter of the third kind', 'child hitman', 'self repair', 'envy', 'homosexual sex', 'male homosexuality', 'noisy neighbor', 'al qaeda', 'bomb making', 'saudi arabia', 'stabbed in the crotch', 'snowglobe', 'series', 'tv series', 'discipline', 'imperialism', 'white slavery', 'breakdown', 'mother in law', 'errand', 'wrong number', 'michelin star', 'violent behavior', 'indian restaurant', 'network marketing', "actor's picture shown in credits", 'animated credits', 'google', 'still images in end credits', 'hansel and gretel', 'same actor playing two characters simultaneously on screen', 'year 1996', 'contamination', 'corporate crime', 'environmental issue', 'working single mother', 'sheriff of nottingham', 'document', 'jurassic', 'inside joke', 'one man show', 'employee', 'laser in eyes', 'tina turner', 'secretary general', 'united nations', '27th century', 'collapsing tunnel', 'weimaraner', 'summertime', 'treehouse', 'walking on train tracks', 'charles ii', 'earl', 'automobile association of america', 'reference to tom jones the singer', 'resistance movement', 'broke', 'promoter', 'trainer', 'soul mate', 'book club', 'cheating boyfriend', 'job promotion', 'planet earth', 'woman in bra and panties', 'interior designer', 'closed ecological system', 'perpetual winter', 'population control', 'witness protection program', 'crying man', "men's clothing store", 'giant vegetable', 'pest control', 'bedroom', 'hug', 'interrupted sex', 'trumpet', 'police corruption', 'cover band', 'tribute band', 'youtube', 'food in title', 'potato', 'car damage', 'computer printer', 'female band', 'fingernail polish', 'obsessive love', 'bread', 'coarse language', 'tricked into lesbian sex', 'chumscrubber', 'glass of water', 'loss of faith', 'girl fight', 'sailing boat', 'tent', 'caught in a net', 'custody', 'fashion photographer', 'bra', 'small village', 'androgynous', 'psychological drama', 'erasmus', 'lebanese', 'loss', 'thesis', 'hitchhiking', 'modern western', 'cayman islands', 'retired actress', '22 year old', 'b.a.r.t.', 'bay area', 'scottish', 'ugandan', '22nd century', 'lyricist', 'drunk wife', 'fbi investigation', 'fire investigator', 'turpentine', 'bravery', 'medal of honor', 'military life', 'unreliable flashback', 'hanged man', 'mysterious past', 'reference to mr. rogers', 'reference to vincent van gogh', 'talking about suicide', 'animated sequence', 'judicial system', 'private investigation', 'suspected murder', 'feminist', 'visit', 'romulan', 'puppet animation', 'red hat', 'poker player', 'reference to frederic chopin', 'reference to johannes brahms', 'slaughterhouse', 'ferry', 'first daughter', 'lobby boy', 'mentor protege relationship', 'dude', 'ensemble cast', 'episodic structure', 'motel room key', 'fat', 'health', 'juice', 'obese', 'german immigrant', 'war orphan', 'grand canyon', 'casualty notification', 'casualty notification team', 'staff sergeant', 'arrogance', 'kennel', 'refrence to liza minnelli', 'quantum mechanics', 'runaway', 'diver', 'diving', 'flare', 'belgium', 'author cameo', 'two part tv movie', 'staff', 'liberty', 'space adventure', 'undercover agent', 'deathbed', 'third part', 'nigerian', 'born in vitro', 'in vitro fertilization', 'siblings', 'year 2065', 'duck', 'stuntman', 'beatnik', 'necrophilia', 'partying', 'lunch wagon', 'back from the dead', 'paranormal research', 'shaky cam', 'mongolia', 'spear', 'speared to death', 'circus vargas', 'body', 'white blood cell', 'missing file', 'staged rape', 'deforestation', 'moose', 'voice', 'female in a shower', 'woman wearing a miniskirt', 'u.s. congressman', 'immorality', 'ukrainian', 'reference to winnie mandela', 'adventurer', 'el dorado', 'high priest', 'cultural revolution', 'baptist church', 'double murder', 'nuclear explosion', 'nuclear weapon', 'nazi germany', 'track and field', 'lds film', 'mormonism', 'gravestone', 'pipeworks', 'friday', 'techno rave', 'death of animal', 'high heels', 'kneed in the crotch', 'communist dictatorship', 'exploding helicopter', 'finger bitten off', 'apology', 'brother in law sister in law relationship', 'expectant father', 'histrionic personality disorder', 'middle aged woman', 'off screen rape', 'high school romance', 'fatal attraction', 'interspecies romance', 'ragtime', 'prosthetics', 'tail', 'manmade earthquake', 'tunnel boring machine', 'christ figure', 'messiah', 'saviour', 'social commentary', 'leo tolstoy', 'construction site', 'man without a country', 'meal', 'overeating', 'male in bathtub', 'seductive fembot', 'beer pong', 'mischief', 'no dialog', 'sign language', 'player', 'tennis player', 'wimbledon', 'corrupt judge', 'tax evasion', 'white suit', 'year 1930', 'aids', 'homophobia', '40th birthday', 'reference to c. thomas howell', 'cards', 'tarot card', 'human race', 'religious faith', 'child murderer', 'art forgery', 'picture framer', 'substance d', 'conversations', 'record store', 'thirty something', 'christmas season', 'release from prison', 'last man standing', 'hiding in a closet', 'cyclops', 'fortress', 'martinique', 'banishment', 'sorcery', 'storm at sea', 'romantic friendship', 'john wilkes booth', 'lincoln assassination', 'vegan', 'feces in a swimming pool', 'female vomiting', 'steel helmet', 'u.s. marine', 'neighbor neighbor relationship', 'remote control car', 'andy kaufman', 'actress name in title', 'bushwhacker', 'based on stage musical based on stage play', '20th century', 'association', 'fifa', 'movie business', 'death row inmate', 'southern u.s.', 'diminution', 'physical flexability', 'ticket', 'at work', 'vermin', 'high school prom', 'prom king', 'shanghai', 'sex in an airplane', 'connection', 'african national congress', 'film starts with text', '12 step program', 'looting', 'nightclub singer', 'breakup', 'outrunning explosion', 'one armed man', 'equal pay', 'shakespeare in modern dress', "shakespeare's coriolanus", 'kite', 'hook and chain', 'ghost hunting', 'cosmetics', 'eiffel tower', 'ritual', 'stew', 'ballroom dancing', 'poise', 'self confidence', 'may december romance', 'rockefeller center', 'surveillance camera', 'war on terror', 'harmonium', 'pudding', 'bilingual', 'three women', 'luberon', 'mystery writer', 'i.d.', 'totalitarian', 'hot shot', 'stock car driver', 'mad max', 'bullet time', 'knife throwing', 'throwing a knife', 'ballroom', 'dance teacher', 'salford', 'armpit fetish', 'electric torture', 'japanese american', 'tree house', 'rapture', 'heart ripped out', 'reference to michael jackson', 'first person title', 'reference to louis de funes', 'amish', 'short order cook', 'clock tower', 'delorean', 'news broadcast', 'tv broadcast', 'dead teenager', 'explosives expert', 'snorting', 'postmodern', 'saint', 'spanish civil war', 'starts with narration', 'alternative reality', 'multiple outcomes', 'twist of fate', 'shot gun', 'bechdel test failed', 'repeated event', 'dwegons', 'judicial', 'judiciary', 'table broken in fight', 'baseball team', 'actor director writer', 'tramp', 'essex', 'little sister', 'pallbearer', 'nail gun', 'catastrophe', 'puerto rican', 'cancun mexico', 'lti txii', 'peugeot 206', 'rover 3 5 litre.coupe p5b', 'skeptic', 'tailing', 'close up of eyes', 'quirkiness', 'comic', 'jay and silent bob', 'plastic surgery', 'pathology', 'bigotry', 'abu dhabi', 'digimon', 'extradition', 'poland', 'miscarriage', 'pact', 'colombian drug cartel', 'reference to ben affleck', 'reference to brad pitt', 'reference to george clooney', 'reference to jack nicholson', 'gramophone', 'missing children', 'flower girl', 'antique shop', 'asian dragon', 'korean', 'falling down an elevator shaft', 'oleander', 'faun', 'oven', 'stove', 'thanksgiving dinner', 'human dragon relationship', 'dark force', 'entity', 'evil force', 'paranormal phenomenon', 'adaptation directed by original author', 'thor', 'dairy farm', 'rustler', 'blackheart', 'mephistopheles', 'protest march', 'literary agent', 'chained', 'cellphone', 'martial arts school', 'wing chun', 'concert film', 'emcee', 'mohawk', 'scent', 'sense of smell', 'smell', 'bottle', 'jogging', 'seeking love', 'single guy', 'tv host', 'women rivals for man', 'hiv positive', 'marshall university', 'female fighter', 'idler', 'poorhouse', 'delusional depression', 'mysterious girl', 'cigarette lighter', 'armed forces', 'political corruption', 'shady', 'war profiteer', 'ghostface', 'east africa', 'hunter gatherer', 'indigenous', 'rift valley', 'swahili', 'bb gun', 'tongue stuck on a pole', 'curtain', 'xeroderma pigmentosum', 'collision course', 'ship run aground', 'narc', 'film director', 'rogue', 'awkward boy', 'shy kid', 'water park', 'talking to self in mirror', 'narrated by a girl', 'younger version of character', 'teenage abduction', 'art class', 'nerve gas', 'video message', 'navy seal', 'popcorn', 'outnumbered', 'bailiff', 'nazi hunter', 'paraguay', 'space shuttle transported on a 747', 'camelot', 'excalibur', 'battle of gallipoli', 'distant past', 'nobleman', 'trapped in the past', 'african american stereotype', 'rent', 'threatening telephone call', 'wine tasting', 'belief', 'passive aggression', 'shot through the head', 'kung fu master', 'crossdresser', 'long tracking shot', 'psychology', 'death squad', 'mass killing', 'musical number', 'reference to guantanamo bay', 'bobble head doll', 'rejuvenation', 'edinburgh scotland', 'closeted lesbian', 'islamophobia', 'propaganda', 'feather', 'bloody water', 'piranha', 'female bounty hunter', 'child fugitive', 'martial art', 'silk road', 'mineral', 'hoodoo', 'amateur radio', 'ham radio', 'based on cult tv series', 'exercise machine', 'midnight movie', 'year 2015', 'bad day', 'kosher', 'vhs', 'king arthur', 'lancelot', 'bomb squad', 'pushed out a window', 'smothered with a pillow', 'telescope', 'gargoyle', 'curb stomping', "son dislikes mother's boyfriend", 'white supremacist', 'first of series', 'bruges', 'alternative comedy', 'buddy comedy', 'gun smuggling', 'anthropomorphism', 'tomato', 'ambulance driver', 'magazine editor', 'iraqi', 'political military conspiracy', 'weapon of mass destruction', 'flying through a thunderstorm', 'sex in a pool', 'knocked out', 'online game', 'fixer', 'valor', 'listener', 'radio talk show', 'in love with an inanimate object', 'sex doll', 'appearing from water', 'college tuition', 'state fair', 'breaking the fourth wall by talking to the audience', 'singapore', 'garden', 'unity', 'female reporter', 'san francisco giants']
  arv=len(keywords)
  keycolumns={'movie_title':[]}
  for index, movie in processedDataFrame.iterrows():
    movieKeywords=[0]*arv;
    currentKeyword=""
    keycolumns['movie_title'].append(movie['movie_title'])
    for i in keywords:
      j=0
      currentKeyword=i
      keycolumns.update({currentKeyword:[0]*processedDataFrame.shape[0]})
      onvei=0
      if(type(movie['plot_keywords'])==float):
        continue;
      elif i in movie['plot_keywords'].split("|"):
        onvei=1
        movieKeywords[j]=1
        keycolumns[currentKeyword][j]=onvei
      j+=1
  df2=pd.DataFrame(keycolumns)
  processedDataFrame=processedDataFrame.merge(df2, on='movie_title', how='inner')
  processedDataFrame = processedDataFrame.drop('plot_keywords', axis=1)
  processedDataFrame = processedDataFrame.drop('movie_title', axis=1)
  #getting rid of nan values
  for i in processedDataFrame.columns:
    mean_value = processedDataFrame[i].mean()
    processedDataFrame[i].fillna(value=mean_value, inplace=True)



  return processedDataFrame, andmestikud

In [ ]:
newDataFrame, andmestikud = preprocessRating(filmid.copy())

4538


In [ ]:
#for i in newDataFrame.columns:
  #mean_value = newDataFrame[i].mean()
  #newDataFrame[i].fillna(value=mean_value, inplace=True)

TypeError: Could not convert RPGPGPG-13PG-13PGRRPGPG-13RRPG-13RPG-13RTV-PGRRRGPGPG-13RRPG-13RRRRRRRRRRRRRRRRPGPG-13RRRRPG-13RTV-14RPGApprovedRXXXXXXXXXXXXXXXXPG-13PGPG-13PG-13PG-13PG-13PG-13PGRApprovedApprovedApprovedApprovedApprovedApprovedApprovedApprovedApprovedApprovedApprovedApprovedApprovedApprovedApprovedApprovedPGPG-13PGPG-13RRPGRPGRRRPGPG-13RNC-17Not RatedGPG-13PGRPGPGApprovedPG-13RRPG-13PG-13RRPG-13RRRRRPG-13PGRPG-13RRPGRRPG-13PGRPGGRRRPGPGApprovedRPG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13RRRRRPGPG-13PGPG-13RPGPG-13PGPG-13TV-GPGRRRRRRRRRRRRRRRRPGRPG-13PGRPGPG-13RPG-13PG-13RPG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13PG-13ApprovedPG-13PG-13TV-14RPGPG-13RRPGUnratedPG-13RRRPGPGPG-13PG-13PG-13RRPGGRPGPGApprovedRPGPGRPG-13TV-PGRPG-13PG-13PG-13ApprovedPGRPG-13PG-13RPG-13PGApprovedRPGPG-13PGPGPGPGPGPGPGPGPGPGPGPGPGPGPGPGPG-13RPG-13RPG-13RPG-13PG-13PG-13PGRPGPG-13RPGRRNot RatedPGRPG-13PGRRPG-13PG-13ApprovedPG-

In [ ]:
print(newDataFrame.columns)

Index(['color_x', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'actor_1_name',
       ...
       'sex doll', 'appearing from water', 'college tuition', 'state fair',
       'breaking the fourth wall by talking to the audience', 'singapore',
       'garden', 'unity', 'female reporter', 'san francisco giants'],
      dtype='object', length=7692)


In [ ]:
newDataFrame['content_rating'].values
filmid['content_rating'].values

array(['R', 'PG', 'PG', ..., 'R', 'PG-13', 'R'], dtype=object)

In [ ]:
filmid.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [ ]:
filmid['genres']
genres=set()
for i in filmid.genres:
  for j in i.split("|"):
    genres.add(j)

In [ ]:
print(genres)
genreLength=0
for i in genres:
  genreLength+=1
print(genreLength)

{'Documentary', 'Game-Show', 'Sport', 'Mystery', 'Drama', 'Adventure', 'Action', 'Comedy', 'Musical', 'Reality-TV', 'Short', 'History', 'News', 'Film-Noir', 'Horror', 'Romance', 'Crime', 'Family', 'Biography', 'Animation', 'Sci-Fi', 'Fantasy', 'Western', 'Thriller', 'War', 'Music'}
26


In [ ]:
genrecolumns=dict()
for movie in filmid.genres:
  movieGenres=[0]*26;
  j=0
  currentGenre=""
  for i in genres:
    currentGenre=i
    genrecolumns.update({currentGenre:[0]*len(filmid)})
    onvei=0
    if i in movie.split("|"):
      onvei=1
      movieGenres[j]=1
      genrecolumns
    genrecolumns[currentGenre][j]=onvei
  j+=1
print(len(genrecolumns))
print(genrecolumns['Sci-Fi'][1])

26
0




### 2.2. Treeni mudel

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def trainRating(filmiDataFrame):
    '''Õpib sisendandmete põhjal mudeli, mis oskab ennustada filmi reitingu.
       Võib kasutada väliseid funktsioone ja muud endale sobivat.
       Sisendiks saab treeningandmete DataFrame'i.
       Tagastab treenitud mudeli.'''

    treeningandmed, andmestikud = preprocessRating(filmiDataFrame)
    mudel = None
    X = treeningandmed.drop('content_rating', axis=1)
    y = treeningandmed['content_rating']
    ########
    # ToDo
    ########
    #ratings = {'TV-Y': 0,'TV-Y7': 1,'G': 2,'TV-G': 3,'Approved': 4,'PG': 5,'TV-PG': 6,'GP': 7,'PG-13': 8,'TV-14': 9,'M': 10,'R': 11,'NC-17': 12,'X': 13,'Not Rated': 14,'Passed': 15,'Unrated': 16,'TV-MA': 17}
    mudel = KNeighborsClassifier()
    mudel.fit(X, y)
    return mudel, andmestikud

In [ ]:
mudel, andmestikud= trainRating(filmid.copy())

4538


In [ ]:
j=0
rating=[]
number_rating=[]
for i in filmid['content_rating']:
  if i not in rating:
    rating.append(i)
    number_rating.append(j)
    j+=1
print(rating)
print(number_rating)

['R', 'PG', 'PG-13', nan, 'TV-PG', 'G', 'TV-14', 'Approved', 'X', 'NC-17', 'Not Rated', 'TV-G', 'Unrated', 'TV-MA', 'GP', 'TV-Y', 'Passed', 'TV-Y7', 'M']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


### 2.3. Ennustamine

Meeldetuletus: andmete eeltöötlus tuleb teha ka testhulgale. Selleks võib kasutada abifunktsioone. Siia võib lisada abiplokke.

In [ ]:
def predictRating(mudel,andmestikud, testhulk):
    '''Saab sisendiks treenitud mudeli, eeltöötluseks vajalikud andmed
       ja terve testhulga algsel kujul (ilma reitinguta ja eeltöötluseta).
       Tagastab ennustatud reitingute järjendi.'''
    # Filmireitingud ('R', 'PG-13' jne)
    reitingud = ['TV-Y','TV-Y7','G','TV-G','Approved','PG','TV-PG','GP','PG-13','TV-14','M','R','NC-17','X','Not Rated','Passed','Unrated','TV-MA']
    # Eeltöötlus
    test_andmed = preprocessRating(testhulk)

    # Ennustamine
    #try:
      #X_test = test_andmed.drop('content_rating', axis=1)
    #except:
      #print('no column content_rating')
    X_test=test_andmed[0]
    ennustused = mudel.predict(X_test)

    # Tagastamine

    return ennustused

### 2.4. Testimine  

Siin on etteantud testplokk, mis näitab, kuidas see kõik koos toimida võiks. Oluline on see, et see kõik oleks kasutatav erinevatel andmehulkadel, näiteks varem nägemata failil.  Jah, muidugi võib seda töö käigus muuta - kontrollimisel eeldatakse aga, et see **algne testplokk ka töötab**.

Üldiselt on alguses mõttekas väiksema andmehulgaga katsetada, kui süntaksi- ja muud vead on välja roogitud, võib kogu andmehulga ette võtta.  

**NB!** Ärge eeldage, et *testandmed* väärtustamisel saab midagi juba töödeldut. Testandmete eeltöötlus tuleks ära teha ennustusmeetodi raames või mõne abifunktsiooniga enne seda. Tavaelus tulevad testandmed teile uue ja iseseisvana ette.

In [ ]:
# Eeldades, et 'filmid2' on äsja failist sisseloetud, 28-tulbaline ja muidu muutmata kujul DataFrame
filmid2 = pd.read_csv(faili_asukoht, delimiter=",",index_col=0)
# Viska välja puuduvate reitingutega read - võite arvestada, et kodutöö kontrollimisel ennustatav väärtus puudu ei ole.
filmid2 = filmid2.dropna(subset=['content_rating'],axis=0)


# Võta testimiseks esimesed 10 kirjet, ilma ennustatava väärtuseta
# Kodutöö hindamisel loetakse testandmed eraldi failist, treenimiseks kasutatakse kogu treeninghulka
# Eeldades, et 'filmid2' on äsja failist sisseloetud, 28-tulbaline ja muidu muutmata kujul DataFrame
filmid2 = pd.read_csv(faili_asukoht, delimiter=",",index_col=0)
# Viska välja puuduvate reitingutega read - võite arvestada, et kodutöö kontrollimisel ennustatav väärtus puudu ei ole.
filmid2 = filmid2.dropna(subset=['content_rating'],axis=0)
testandmed = filmid2[:100]


# Vali treeninghulk - jah, seda võib arenduse käigus muuta.
mudel, andmestikud = trainRating(filmid2[100:])

# Tunnused, mille põhjal ennustada
testandmed_x = testandmed.drop('content_rating', axis=1)
# Tunnus, mida ennustada
testandmed_y = testandmed['content_rating']


# Ennustatud reitingud
ennustused = []
ennustused = predictRating(mudel,andmestikud,testandmed_x)

# Arvuta ennustuste ja õigete vastuste põhjal ennustamise täpsus
print("Täpsus", accuracy_score(ennustused, testandmed_y))
#Sain täpsuseks oma andmetega 0.45

4160
100
no column content rating
Täpsus 0.45


## 3. Filmiskooride ennustamine  

**2 punkti** saab mudeli eest, mis ennustab filmi IMDB skoori. Kuna see on pidev väärtus, siis kasutame regressiooni ja mõõdikuna keskmist ruutviga (MSE).


### 3.1. Eeltöötlus

In [8]:
def preprocessScore(dataFrame, mode="train"):
  '''Tegeleb sisse saadud DataFrame'i töötlemisega.
  '''
  # võimalikud testandmete eeltöötluseks vajalikud mudelid-teadmised (nt keskmised väärtused, one-hot tulpade nimed vms)
  andmestikud = []
  processedDataFrame=dataFrame
  # eeltöötluse tulemus - muudetud DataFrame
  try:
    processedDataFrame = processedDataFrame.drop('movie_imdb_link', axis=1)
  except:
    print('no column movie_imdb_link')
  #color tegemine 1 ja 0ks
  try:
    processedDataFrame['color'] = processedDataFrame['color'].map({'Color': 1, 'Black and White': 0})
  except:
    print('no column color')
  #teeme numbriliseks osad columnid, selleks et onehot vectorite arv ei muutuks liiga suureks
  directors=dict()
  for i in processedDataFrame['director_name'].values:
    if i not in directors.keys():
      directors.update({i:len(directors.keys())})
  processedDataFrame['director_name'] = processedDataFrame['director_name'].map(directors)
  directors=dict()
  for i in processedDataFrame['actor_2_name'].values:
    if i not in directors.keys():
      directors.update({i:len(directors.keys())})
  processedDataFrame['actor_2_name'] = processedDataFrame['actor_2_name'].map(directors)
  directors=dict()
  for i in processedDataFrame['actor_3_name'].values:
    if i not in directors.keys():
      directors.update({i:len(directors.keys())})
  processedDataFrame['actor_3_name'] = processedDataFrame['actor_3_name'].map(directors)
  directors=dict()
  for i in processedDataFrame['actor_1_name'].values:
    if i not in directors.keys():
      directors.update({i:len(directors.keys())})
  processedDataFrame['actor_1_name'] = processedDataFrame['actor_1_name'].map(directors)
  processedDataFrame['language']=numerate(processedDataFrame,'language')
  processedDataFrame['country']=numerate(processedDataFrame,'country')
  #Teeme onehot vektorid filmide shanritele
  genres=set()
  genrecolumns=dict()

  for i in processedDataFrame.genres:
    for j in i.split("|"):
      genres.add(j)
  genrecolumns.update({'movie_title':list()})
  print(processedDataFrame.shape[0])
  genres=['War', 'Music', 'Comedy', 'Documentary', 'Film-Noir', 'Drama', 'Action', 'Biography', 'Game-Show', 'Sci-Fi', 'Crime', 'Sport', 'Fantasy', 'Short', 'News', 'Adventure', 'Thriller', 'Western', 'Mystery', 'History', 'Romance', 'Animation', 'Musical', 'Reality-TV', 'Horror', 'Family']
  for genre in genres:
    genrecolumns.update({genre:[]})
  for index, movie in processedDataFrame.iterrows():
    movieGenres=[0]*len(genrecolumns);
    currentGenre=""
    genrecolumns['movie_title'].append(movie['movie_title'])
    j=0
    for index2, currentGenre in enumerate(genres):
      if currentGenre in movie['genres'].split("|"):
        movieGenres.append(1)
        genrecolumns[currentGenre].append(1)
      else:
        movieGenres.append(0)
        genrecolumns[currentGenre].append(0)
      j+=1
  df2=pd.DataFrame(genrecolumns)
  processedDataFrame=processedDataFrame.merge(df2, on='movie_title', how='inner')
  processedDataFrame = processedDataFrame.drop('genres', axis=1)

  keycolumns=dict()
  #Teeme onehot vektorid filmide võtmesõnadele
  keywords=set()
  keywordsBefore=dict()
  #for i in processedDataFrame.plot_keywords:
    #try:
      #for j in i.split("|"):
        #if j in keywordsBefore.keys():
          #keywordsBefore[j]=keywordsBefore[j]+1
        #else:
          #keywordsBefore[j]=1
    #except:
      #i="None"
  #for i in keywordsBefore.keys():
    #if keywordsBefore[i]>5:
      #keywords.add(i)
  #keycolumns.update({'movie_title':list()})
  #print(len(keywords))
  keywords = ['bomb', 'bus', 'elevator', 'explosive', 'rescue', 'acorn', 'herd', 'iceberg', 'ocean', 'pirate', 'bad', 'dragon', 'grandfather', 'orb', 'warrior', 'apartment', 'bedtime story', 'eagle', 'stuttering', 'swimming pool', 'autopsy', 'lantern', 'planet', 'ring', 'test pilot', 'arranged marriage', 'bride', 'corpse', 'groom', 'woods', 'actor', 'detective sergeant', 'hotel', 'nightclub', 'thief', 'blood', 'publisher', 'publishing house', 'werewolf', 'wolf', '1990s', 'brother brother relationship', 'nantucket island', 'one word title', 'sister sister relationship', '2010s', 'ape', 'ballet', 'psychic', 'supernatural', 'artist', 'drug', 'tree sap', 'vampire', 'amusement park', 'on the road', 'zombie', 'zombie apocalypse', 'zombie spoof', 'countess', 'game reality crossover', 'ghost', 'group of friends', 'tower', 'airline', 'airplane', 'bathroom', 'sex', 'toilet', 'church', 'college', 'color in title', 'philosophy', 'texas', 'basketball', 'burglary', 'police', 'robbery', 'teenage girl', 'boss', 'holiday', 'letter', 'north pole', 'santa', 'adulterous wife', 'bipolar disorder', 'obsessive compulsive personality disorder', 'unhappy marriage', 'widow', 'dam', 'flood', 'money', 'sheriff', 'antarctica', 'geologist', 'investigation', 'u.s. marshal', 'winter', 'immigration', 'jewish', 'pogrom', 'tradition', 'tradition versus modernity', 'avant garde', 'british pop', 'character name in title', 'musician', 'number in title', 'girl', 'maori', 'new zealand', 'whale', '19th century', 'colombia', 'love', 'marriage', '1950s', 'archeologist', 'korea', 'korean war', 'soldier', 'author', 'disney', 'loss of father', 'reference to walt disney', 'suppression of emotion', 'evil', 'killer', 'murder', 'psychopath', 'serial killer', 'blue', 'new york', 'new york city', 'no opening credits', 'smurf', 'based on tv series', 'fictional band', 'girl band', 'music industry', 'pop band', 'alien', 'drug lord', 'gang', 'hunter', 'predator', 'nevada', 'pledge', 'porcupine', 'search', 'arms dealer', 'atf', 'bail', 'multiple perspectives', 'detective', 'detective novel', 'fantasy sequence', 'hallucination', 'writer', 'blockbuster', 'bumblebee the character', 'semi truck and trailer', 'truck', 'colonel', 'lieutenant', 'nuclear warhead', 'team', 'washington d.c.', 'black comedy', 'bully', 'friend', 'house', 'practical joke', 'buried alive', 'coffin', 'iraq', 'one person', 'single set production', 'athlete', 'jew', 'missionary', 'olympic games', 'race', 'apache', 'apache indian', 'confederate', 'mexico', 'scout', 'biblical plague', 'louisiana', 'plague', 'small town', 'freddy krueger', 'lucid dream', 'slasher', 'sleep deprivation', 'bloopers during credits', 'bucket of popcorn', 'chastity belt', 'guru', 'toronto maple leafs', 'dying planet', "regaining one's memory", 'sacrifice', 'stabbed to death', 'bank', 'bank clerk', 'gangster', 'loki', 'mask', 'artificial intelligence', 'box office flop', 'fighter jet', 'navy', 'pilot', 'car accident', 'japan', 'newlywed', 'photograph', 'photographer', 'butchery', 'catholic', 'catholic school', '1970s', 'afterlife', 'heaven', 'pedophile', 'rape', 'bare chested male bondage', 'european', 'female frontal nudity', 'topless woman', 'war paint', 'boston', 'mole', 'undercover', 'undercover cop', 'captain', 'expedition', 'sea', 'submarine', 'battle', 'creature', 'hyena', 'monkey', 'village', 'airport', "asperger's syndrome", 'autism', 'muslim', 'racial profiling', 'astronaut', 'mars', 'mars the planet', 'outer space', 'zero gravity', 'bangladesh', 'buckingham palace', 'september 11 2001', 'title directed by female', 'austrian', 'breasts', 'cholera', 'provence', 'destiny', 'imagination', 'sex scene', 'crown', 'dream', 'king neptune', 'plankton', 'cup', 'derby county', 'flashback', 'football', 'leeds united', 'latter day saints', 'pickup truck', 'video camera', 'blackmail', 'panties pulled down', 'prostitution', 'bar', 'bullet', 'corruption', 'lawyer', 'professor', 'research', 'sexuality', 'student', 'based on film', 'crime family', 'remake', 'southern california', 'boy', 'brother', 'mafia', 'senator', 'ambush', 'assassin', 'priest', 'prostitute', 'sweden', 'absurd humor', 'egg', 'gross out humor', 'lesbian', 'hong kong', 'missing person', 'missing son', 'best friend', 'car race', 'conspiracy', 'gadget car', 'spy', 'cardinal richelieu', 'critically bashed', 'duel', 'musketeer', 'american politics', 'iraq war', 'occupation', 'white house', 'east germany', 'secret police', 'stasi', 'bicyclist', 'doll', 'star wars reference', 'voice impersonation', 'arab', 'bare chested male', 'foreign legion', 'french foreign legion', 'morocco', 'banker', 'female protagonist', 'live action and animation', 'magic', 'nanny', 'cultural conflict', 'politics', 'president', 'slavery', 'u.s. president', 'adoption', 'divorce', 'family relationships', 'farce', 'wedding', 'camping', 'forest', 'lake', 'stabbed with glass', 'vomiting', 'campaign', 'election', 'presidential election', 'satire', 'danger', 'drug cartel', 'female cop', 'female police officer', 'fugitive', 'womanizer', 'barefoot', 'endorsement', 'surfboard', 'virtual reality', 'waves', 'death', 'doctor', 'grave', 'patient', 'venezuela', 'blood splatter', 'camera shot from inside human body', 'combat', 'die hard scenario', 'fbi', 'convict', 'karaoke', 'middle america', 'salesman', 'singer', 'new mexico', 'prison', 'smuggling', 'theft', 'hostage', 'shot in the chest', 'shot in the heart', 'outlaw', 'rancher', 'small western town', 'stagecoach', 'train', '1880s', 'cowboy', 'low budget film', 'shoot out', 'wild west', 'chase', 'clue', 'movie star', 'alligator', 'golf', 'panda', 'stunt', 'area 51', 'military', 'spacecraft', 'spaceship', 'hip hop', 'liberal', 'truth', 'game', 'showdown', 'bus driver', 'money scheme', 'poverty', 'sewer', 'lynchian', 'reality', 'police officer', 'police officer taken hostage', 'communications', 'computer', 'genius', 'job', 'athletic training', 'football practice', 'funeral', 'gymnasium', 'man crying', 'christian film', 'hymn', 'religion', 'slave', 'fuel', 'rocket', 'space travel', 'chipmunk', 'cruise ship', 'football ball', 'overboard', 'tennis ball', 'based on true story', 'cia', 'cocaine', 'journalist', 'smear campaign', 'female pubic hair', 'girl in panties', 'leg spreading', 'public nudity', 'sex in bed', 'prank gone wrong', 'scantily clad female', 'shotgun', 'sorority house', 'stabbed in the neck', 'hawaii', 'hawaiian', 'historically inaccurate', 'princess', 'tearing up a letter', '1800s', 'bird', 'blue bird of happiness', 'foreign language adaptation', 'past', 'hand job', 'hugging', 'long hair', 'two men one woman', 'alien invasion', 'iron man', 'courtroom', 'dysfunctional family', 'indiana', 'trial', 'cancer', 'survival rate', 'therapist', 'lapd', 'mexican', 'violence', 'package', 'slow motion scene', 'topless female nudity', 'wrong address', 'bloody body of child', 'dead boy', 'monster', 'afghanistan', 'death of marine', 'dog', 'dog adoption', 'man dog relationship', 'female nudity', 'male nudity', 'parody', 'reference to myspace', 'sequel', 'pakistan', 'road trip', 'adultery', 'extramarital affair', 'infidelity', 'unfaithfulness', 'american president', 'cbs', 'expose', 'fear', 'paranoia', 'reporter', 'buddy movie', 'general', 'inventor', 'steampunk', 'utah', 'pyramid', 'satellite', 'connecticut', 'witch', 'witchcraft', 'basketball player', 'boarding school', "shakespeare's othello", 'african american', 'old age', 'bear', 'grizzly', 'wilderness', 'friendship', 'frontier', 'honor', 'outpost', 'love across time', 'parallel time', 'dysfunctional marriage', 'sex offender', 'suburb', 'breaking the fourth wall', 'high school friends', 'imperative in title', 'skateboard', 'amateur detective', 'boat builder', 'stepfather', 'witness', 'fame', 'fashion', 'gay lead character', 'mockumentary', 'crying woman', 'plot', 'suspect', 'waitress', 'younger man older woman relationship', 'defense lawyer', 'marine', 'villager', 'agent', 'federal bureau of investigation', 'gay relationship', 'secret', 'emaciation', 'plane crash', 'prisoner of war', 'torture', 'beating', 'muscleman', 'tied up', 'tough guy', 'engineer', 'ethics', 'mountain', 'railroad', 'south america', 'journalism', 'news anchor', 'newscast', 'newscaster', 'tv journalism', 'abbreviation in title', 'sudden wealth', 'unknowing heir', 'music business', 'record executive', 'record label', 'black september', 'israel', 'munich olympics', 'cannibal', 'evil brother', 'giant', 'london england', 'queen', 'cadillac', 'disguise', 'secret agent', 'novelist', "writer's block", 'writing', 'engagement', 'maid', 'song', 'california', 'car', 'drag racing', 'high school', "rock 'n' roll", 'drink', 'manager', 'partner', 'pool hall', 'black hole', 'father daughter relationship', 'saving the world', 'wormhole', 'canadian armed forces', 'canadian soldier', 'directed by star', 'veteran', 'gang rape', 'hmong', 'teenager', 'widower', 'bathory', 'legend', 'virgin blood', 'curse', 'tourism', 'travel', 'england', 'prejudice', 'upper class', 'working class', 'cellular memory', 'eye', 'eye transplant', 'seeing dead people', 'violinist', 'name calling', 'penis slur', 'wyatt earp', 'count', 'escape', 'island', 'revenge', 'sailor', 'latina', 'south central los angeles', 'exploitation', 'four word title', 'hallucinogen', 'telepathy', 'explosion', 'klingon', 'moon', 'peace', 'farm', 'hannah montana', 'horseback riding', 'overalls', 'japanese soldier', 'physical torture', 'pow camp', 'reference to adolf hitler', 'baby', 'christmas spirit', 'elf', 'anti semitism', 'drugs', 'highway', 'state trooper', 'vermont', 'japanese tourist', 'secret service', 'wizard of oz', 'kid', 'oxygen', 'rugrats', 'trapped underwater', 'underwater', 'chamberlain', 'theatre', 'bangkok', 'thailand', 'digit in title', 'human form', 'number 3 in title', 'talking on a rooftop', 'terrorism', 'told in flashback', 'wearing a sound wire', 'coach', 'football player', 'misfits', 'strike', 'sumo wrestler', 'betrayal', 'final showdown', 'on the run', 'babysitting', 'cell phone', 'nightmare', 'egypt', 'pharaoh', 'miniature person', 'shrinking', 'sword and fantasy', 'temple', 'based on real person', 'female inventor', 'mop', 'sales pitch', 'woman holding a shotgun', 'analyst', 'nuclear bomb', 'nuclear weapons', 'russia', 'king', 'king george vi', 'speech', 'speech therapist', 'throne', 'birthday', 'clone', 'cloning', 'character says i have a bad feeling about this', 'hand to hand combat', 'seven word title', 'wilhelm scream', 'criminal', 'maine', 'only son', 'school choir', 'summer', 'cat', 'first part', 'lasagna', 'veterinarian', 'iran', 'islamic revolution', 'revolution', 'circus', 'magician', 'oz', 'american football', 'football star', 'high school football', 'left tackle', 'alcoholism', 'little girl', 'orphan', 'troubled marriage', 'american south', 'baseball', 'farmland', 'motel', 'diner', 'private school', 'school', 'android', 'ex cop', 'virtual character come to life', 'cartoon on tv', 'second part', 'videotape', 'education', 'factory', 'mexican american', 'religious icon', 'backup singer', 'band', 'bechdel test passed', 'concert footage', 'pushed off a cliff', 'rat', 'secret passage', "alzheimer's disease", 'book', 'golden fleece', 'half brother', 'magical tree', 'tree', 'art teacher', 'homework', 'principal', 'ballerina', 'madness', 'ice', 'ice age', 'sleeping', 'sloth', 'memory', 'short term memory', 'telephone', 'oceanographer', 'shark', 'boyfriend girlfriend relationship', 'erection', 'arab israeli conflict', 'media', 'middle east', 'palestine', 'dissociative identity disorder', 'limousine', 'multiple personality', 'rhode island', '1980s', 'c 130 hercules', 'panzer', 'san francisco california', 'hero', 'masked hero', 'texas ranger', 'commitment', 'pizza', 'title appears in writing', "woman wearing only a man's shirt", 'jason voorhees', 'massacre', 'slaughter', 'telekinesis', 'alaska', 'deportation', 'father son relationship', 'male objectification', 'sitka alaska', 'abortion', 'anti choice', 'christian preach', 'malpractice', 'unprofessional conduct', '2nd century', 'emperor', 'european history', 'loosely based on historical events', 'roman empire', 'captain nemo', 'dorian gray', 'invisible man', 'dance', 'emperor penguin', 'penguin', 'bathtub', 'eaten alive', 'ferry boat', 'lobster trap', 'sprinkler system', '1960s', 'car hitting an animal', 'folk singer', 'hold up', 'stockholm syndrome', 'swat team', 'vegetarian', 'mob', 'mobster', 'montana', 'coma', 'sword', 'cheating wife', 'film camera', 'hasselblad camera', 'leica camera', 'medium format camera', 'flip phone', 'rape threat', 'vulnerable', 'dog movie', 'golden retriever', 'atlantis', 'crew', 'journal', 'museum', 'anti war', 'interview', 'war on terrorism', '1930s', 'city', 'invisibility', 'the shadow', 'male full back nudity', 'practicing a suicide', 'suicide contemplation', 'anthology', 'heavy metal', 'secretary', 'segments', 'earth', 'giant robot', 'scientist', 'face slap', 'loss of virginity', 'pierced nipple', 'ambassador', 'cannes', 'identity', 'suicide', 'heist', 'pretending to be pregnant', 'toy', 'wish', 'fight', 'mina', 'terrorist', 'exorcism', 'sin eater', 'ufo', 'head coach', 'deputy', 'childhood', 'love triangle', 'organ donation', 'air traffic controller', 'spoof', 'embassy', 'gym', 'memoir', 'lobbyist', 'state dinner', 'celtic mythology', 'cottage', 'fisherman', 'irish', 'selkie', 'demon', 'fbi agent', 'religious fanatic', 'neighborhood watch', 'ohio', 'civil war', 'post civil war', 'reference to friedrich schiller', 'u.s. civil war', 'prisoner', 'reverend', 'terminal illness', 'two word title', 'acceptance', 'coming out to family', 'gay', 'large family', '40 year old', 'cheating on girlfriend', 'collection', 'porno movie', 'earth viewed from space', 'race against time', 'race car driver', 'title at the end', 'death of sister', 'dystopia', 'execution', 'rebellion', 'desert island', 'non statutory female on male rape', 'promotion', 'sexual harassment', 'angkor wat', 'human trafficking', 'sex slavery', 'sex tourism', 'brownstone', 'burglar', 'panic', 'panic room', 'air marshal', 'passenger', 'transatlantic flight', 'based on video game', 'fictional language', 'fictional war', 'orc', 'sword and sorcery', 'day care', 'teddy bear', 'jester', 'kansas', 'tornado', 'suspicion', 'cavalry', 'stallion', 'wild stallion', 'blindness', 'cave', 'spirit', 'wolf cub', 'human versus dinosaur', 'lizard', 'primate', 'time travel', 'tyrannosaurus rex', 'ira', 'limerick', 'freeway', 'trailer park', 'contract killer', 'murder of a girl', 'serial murderer', 'abusive husband', 'chick flick', 'north carolina', 'main characters killed off', 'agoraphobia', 'copycat murder', 'psychologist', 'hairy chest', 'social worker', 'spinning in chair', 'cunnilingus', 'drug trafficking', 'no panties', 'shootout', 'dubstep', 'tied feet', 'based on short film', 'based on short story', 'latino', 'life or death', 'attic', 'surrealism', 'yuppie', 'china', 'faith', 'panama', 'photography', 'suspense', 'dock', 'longshoreman', 'union', 'experiment', 'human versus shark', 'underwater headquarters', 'french revolution', 'old testament', 'part of an unfinished series', 'reference to john wayne', 'roman emperor', 'horse', 'horse racing', 'jockey', 'limp', 'seabiscuit', 'classified ad', 'intern', 'magazine', 'time machine', 'hindu', 'india', 'sikh', 'protection', 'russian', 'sergeant', 'animal name in title', 'ape abducts a woman', 'gorilla', 'king kong', 'bare breasts', 'bra and panties', 'nudity', 'syringe', 'alliance', 'chinese', 'strategy', 'conflicted hero', 'mission', 'rescue mission', 'political cover up', 'political satire', 'carlsbad california', 'security camera', 'fire', 'homicide', 'rubble', 'trapped', 'misunderstanding', 'obese woman', 'overweight woman', 'gay kiss', 'homoeroticism', 'lesbian kiss', 'transsexual', 'code', 'false accusation', 'megalomaniac', 'torah', 'aristocrat', 'english', 'painter', 'painting', 'horse ranch', 'ranch', 'wild mustang', 'chef', 'italy', 'restaurant', 'epic', 'greece', 'spartan', 'career', 'casino', 'judge', 'stock', 'stock broker', 'cow', 'farmer', 'howl', 'howling at the moon', 'cult film', 'flash gordon', 'mind reading', 'meaning of life', 'moral challenge', 'morality', 'search for meaning', 'ten commandments', 'atlanta georgia', 'competition', 'dancer', 'fraternity', 'continental congress', 'debate', 'declaration of independence', 'massachusetts', 'pennsylvania', 'biker', 'biker gang', 'mississippi', 'cat burglar', 'female lead', 'future', 'mixed martial arts', 'post apocalypse', 'female full frontal nudity', 'gladiator', 'group sex', 'slave girl', 'international cooperation', 'left for dead', 'nasa', 'space habitat', 'cabin in the woods', 'drinking acid', 'introvert', 'truth or dare', 'cover up', 'newspaper', 'sexual abuse', 'beach', 'party', 'resort', 'based on novel', 'imperial japan', 'japanese army', 'world war two', 'boston marathon', 'marathon', 'miracles', 'running', 'missing', 'accidental death', 'dead woman', 'dead woman with eyes open', 'death of mistress', 'cybercrime', 'hacker', 'film within a film', 'filmmaking', 'guerrilla filmmaking', 'making of a movie', 'scientology parody', 'disappearance', 'hearing voices', 'hypnosis', 'post hypnotic suggestion', 'seeing a ghost', 'africa', 'capture', 'jungle', 'tarzan', 'aphrodisiac', 'court jester', 'written and directed by cast member', 'black panther', 'hells angels', 'racist', 'racist cop', 'ballot', 'vote', 'voting', 'orgy', 'pianist', 'sexual fantasy', 'hair', 'taliban', 'aunt', 'death of pet', 'boarding house', 'great depression', 'mistress', 'schizophrenic', 'heroin', 'questioned by police', 'bounty hunter', 'necromonger', 'female star appears nude', 'pants falling down', 'samurai', 'white rat', 'woman in a towel', 'bouquet of roses', 'bright light', 'cardiopulmonary resuscitation', 'white rose', 'charles dickens', 'christmas', 'christmas eve', 'scrooge', 'alice in wonderland', 'mistaking reality for dream', 'shrinking potion', 'colorado', 'niece', 'female neighbor', 'trip', 'emir', 'map', 'sports agent', 'training', 'desert', 'tragic event', 'blackbeard', 'bigfoot', 'sasquatch', 'tourist', 'independence day', 'lady penelope', 'the hood', 'scene of the crime', 'therapy', 'alabama', 'divorce papers', 'husband wife relationship', 'john keats', 'passion', 'poet', 'domestic abuse', 'public humiliation', 'sex symbol', 'suicide attempt', 'coming of age', 'sidekick', 'superpower', 'art deco', 'bible quote', 'silent film', 'worker', 'nubile woman', 'policeman', 'brain tumor', 'psionic power', 'pity', 'readjusting to civilian life', 'world war two veteran', 'corporation', 'ninja', 'robocop', 'exercise bicycle', 'face in food', 'police brutality', 'inheritance', 'uncle', 'wine', 'boxing', 'heavyweight champion', 'iceman', 'mail carrier', 'poetry', 'postal worker', 'choking', 'hospital', 'scam', 'daughter', 'gun', 'sociopath', 'cro magnon', 'dialogue driven storyline', 'single setting', 'storytelling', 'blind date', 'crossword puzzle', 'eccentric', 'fall', 'mine shaft', 'broken leg', 'urination', 'physical abuse', 'radio flyer', 'stepfather stepson relationship', 'halloween', 'killer robot', 'wrestler costume', 'bum', 'mutant', 'street', 'body swap', 'camera shot of feet', 'foot closeup', 'reference to keith richards', 'reference to the rolling stones', 'gay best friend', 'gay man', 'gay marriage proposal', 'south africa', 'homosexual', 'homosexuality', 'hustler', 'male rear nudity', '3d in title', 'animal in title', 'dolphin', 'manatee', 'fake commercial', 'fake newscast', 'fake trailer', 'amazon', 'anaconda', 'boat', 'snake', 'breeding', 'caretaker', 'evacuation', 'orphanage', 'butler', 'civil rights', 'andy warhol', 'art', 'street artist', 'bordeaux', 'mayor', 'pamphlet', 'political thriller', 'long distance runner', 'olympics', 'oregon', 'runner', 'watching television', 'friendship between men', 'hollywood california', 'sitcom', 'female rear nudity', 'lesbian cunnilingus', 'rear entry sex', 'justice', 'nazi', 'body count', 'dictator', 'jumping from an airplane', 'knife fight', 'one man army', 'manipulation', 'simulation', 'anime', 'castle', 'masturbation', 'conspiracy theory', 'kennedy assassination', 'mock documentary', 'bank heist', 'bank robbery', 'heist gone wrong', 'pizza delivery boy', 'unlikely criminal', 'robot', 'robot as menace', 'survival', 'undead', 'roommate', 'thong panties', 'upskirt', 'story within the story', 'talking animal', 'bank manager', 'criminal gang', 'mob hit', 'opera', 'repeating history', 'atf agent', 'ball gag', 'cult leader', 'pastor', 'pretending to be dead', 'confederacy', 'prequel', 'stonewall jackson', 'war hero', '17th century', 'based on fairy tale', 'flower', 'box office hit', 'materialism', 'snowflake', 'defection', 'soviet submarine', 'boston massachusetts', 'true crime', 'falling in love', 'glowing eyes', 'insult', 'three word title', 'boredom', 'luxury hotel', 'dvd', 'girlfriend', 'loan', 'self help', 'makeover', 'matchmaker', 'popularity', 'drifter', 'hope', 'postman', 'usa', 'mental institution', 'post traumatic stress', 'psychiatrist', 'voyeur', 'neo nazi', 'punk band', 'skinhead', 'fake documentary', 'hidden camera', 'narrative', 'reality show', 'austria', 'cake', 'france', 'versailles', 'filming a wedding', 'lingerie slip', 'bitch', 'popular girl', 'prom', 'alamut', 'dagger', 'persia', 'cowboy boots', 'dancing', 'high school dance', 'black humor', 'dentist', 'hitman', 'family home', 'piano', 'skyscraper', 'tightrope walker', 'russian spy', 'strong female lead', 'concert', 'live concert', 'pop group', 'spice girl', 'business', 'crime scene cleanup', 'male frontal nudity', 'tv mini series', 'bosnian war', 'armor', 'human bomb', 'missile attack', 'fighter plane', 'jet fighter', 'refueling', 'soviet union', 'bachelor', 'character says i love you', 'flirting', 'social satire', 'mother son relationship', 'rabbit', 'winnie the pooh', 'don juan', 'retirement', 'cotton', 'flogging', 'los angeles california', 'one night stand', 'single mother', 'white panties', '16 year old', 'tinnitus', 'kung fu', 'martial artist', 'martial arts', 'sister', 'ballistics', 'food', 'raccoon', 'turtle', 'gold', 'mining', 'prospector', 'gay interest', 'critically acclaimed', 'juarez mexico', 'u.s. mexico border', 'war on drugs', 'desperation', 'fog', 'giant insect', 'survival horror', 'balladeer', 'bachelor party', 'activist', 'mystery killer', 'audition', 'deaf', 'singing', 'beach house', 'blonde bombshell', 'cheating husband', 'wife meets mistress', 'hockey team', 'respect', 'chariot race', 'jerusalem', 'dialogue driven', 'jury', 'cabin', 'going in circles', 'pac man', 'stranger', 'cooking', 'guardian', 'kitchen', 'neo noir', 'father daughter incest', 'mental hospital', 'fellatio', 'isolated house', 'overprotective father', 'social isolation', 'unsimulated sex', 'animal', 'donkey', 'lamb', 'pig', 'rooster', 'alien space craft', 'giantess', 'gore', 'country singer', 'music group', 'air raid', 'beer', 'strip club', 'stripper', 'limousine driver', 'cartoon cat', 'puss in boots', '15th century', 'cathedral', 'gypsy', 'paris', 'victor hugo', 'bodyguard', 'death of husband', 'male nanny', 'wrestling match', 'abandoned building', 'chernobyl disaster', 'exploration', 'tour guide', 'van', 'act of kindness', 'neighbor', 'vacation', 'anger management', 'argument', 'irony', 'sarcasm', 'actress', 'assistant director', 'wardrobe', 'pop singer', 'secret service agent', 'heroine', 'kindred spirit', 'rural setting', '2000s', 'golf caddy', 'golf course', 'golfer', 'question mark in title', 'alcohol', 'alcoholic drink', 'drunken man', 'american dream', 'convertible', 'michael myers', 'digging', 'escape attempt', 'tunnel', '1940s', 'child hero', 'fantasy world', 'reference to peter pan', 'army', 'avatar', 'kingdom', 'tribe', 'water', 'crystal meth', 'search for father', 'male pubic hair', 'mall', 'police detective', 'pubic hair', 'security guard', 'gunfight', 'hong kong police', 'police escort attacked', 'denmark', 'wristwatch', 'independent film', 'french', 'police detective hero', 'cage', 'year 2029', 'camp', 'governor', 'senate', 'u.s. senate', 'robot human relationship', 'turing test', 'video surveillance', 'delivery boy', 'dinner', 'punishment', 'attempted murder', 'based on comic book', 'escape from prison', 'lost at sea', 'wealth', 'australia', 'body paint', 'kangaroo', 'camera focus on female butt', 'gay bar', 'millionaire', 'singing in a car', 'corrupt', 'motor scooter', 'newborn baby', 'scooter', 'single', 'army rangers', 'abusive mother', 'adopted daughter', 'borderline personality disorder', 'perfection', 'sadistic cruelty', 'premarital sex', 'sex standing up', 'autumn', 'four seasons', "looking at one's self in a mirror", 'unhappiness', 'journey', 'tiger', 'bdsm', 'employer employee relationship', 'based on stage musical', 'cinderella', 'little red riding hood', 'chauffeur', 'disorganized crime', 'ransom', 'slapstick', 'broadway play', 'replacement', 'stage', 'acronym in title', 'period in title', 'title ends with period', 'cane', 'iowa', 'lawn mower', 'tractor', 'wisconsin', 'santa claus', 'son', 'teacher', 'congressman', 'cook', 'birthday party', 'high school senior', 'millennial generation', 'disposing of a dead body', 'high school student', 'teenage boy', 'stillborn child', 'flat tire', 'gay activist', 'gay pickup', 'gay rights', 'cigarette smoking', 'hand on butt', 'niagara falls', 'plastic bubble', 'dental braces', 'dental headgear', 'former child star', 'panties', 'gambling', 'novel', 'stenographer', 'deer', 'grizzly bear', 'employee employer relationship', 'ship', 'wake', 'italian', 'kidnapping a child', 'reunion', 'commandant', 'russian mafia', 'security system', 'racism', 'separation from family', 'social injustice', 'whipping', 'abusive father', 'coincidence', 'fictional game show', 'fictional quiz show', 'multiple storyline', 'noodles', 'salary', 'illuminati', 'planetary alignment', 'time', 'tomb', 'tomb raider', 'antique', 'cockney accent', 'hatchet', 'drinking', 'female dressed as male', 'nebraska', 'transgender', 'boxer', 'diamond', 'carnage', 'purple panties', 'sexual attraction', 'shaved labia', 'fairy tale', 'true love', 'hollywood', 'scandal', 'diorama', 'idiot', 'irs', 'mouse', 'aspiring actor', 'pimp', 'suitcase full of money', 'boxing trainer', 'women boxing', '23 year time span', 'loss of loved one', 'male female friendship', 'british secret service', 'official james bond series', 'terrorist plot', 'blond boy', 'chlamydia', 'list', 'young couple', 'catholic orphanage', 'long lost siblings', 'twins', 'boob job', 'breast enlargement', 'breast surgery', 'cheating on a test', 'ejaculation', 'disarming someone', 'qing dynasty', 'stabbed with a bayonet', 'war violence', 'skinny dipping', 'humanoid robot', 'man versus machine', 'prosthetic limb', 'robot as pathos', 'robot vs. robot', 'nsa', 'nsa agent', 'prague', 'loyalty', 'genghis khan', 'starving', 'fighting', 'out of body experience', 'reference to the exploited', 'druid', 'eclipse', 'rune', 'scalping', 'warlock', 'convenience store', 'punctuation in title', 'interrogation', 'chaos', 'riot', 'shrine', 'inspiration', 'stage play', 'portal', 'teleportation', 'troubled production', 'anti hero', 'greed', 'evil wizard', 'quidditch', 'wizard', 'mechanic', 'greeting card', 'office', '1920s', 'ambiguity', 'childhood friend', 'speakeasy', 'reference to batman', 'valentine', 'lolita', 'nymphet', 'older man young girl relationship', 'latin', 'pirate ship', 'treasure', 'treasure map', 'greenland', 'u.s. soldier', 'apartment building', 'blood sample', 'crucifix', 'ussr', 'binoculars', 'electronic tag', 'house arrest', 'watching someone', 'coal', 'irish immigrant', 'night', 'latin music', 'on tour', 'tejano', 'fort', 'native american', 'trade', 'escort', 'male escort', 'wedding date', 'animal costume', 'break up', 'fatherhood', 'rapper', 'talking baby', 'camera focus on a female butt', 'hairy buttocks', 'seal the animal', 'wedding planner', 'long distance relationship', 'porn actor in mainstream movie', 'cigarette butt', 'drone', 'funeral procession', 'nellis air force base', 'college campus', 'dance craze', 'depression', 'personal ad', 'catfight', 'dominatrix', 'spanking', 'special forces', 'american', 'driver', 'custody battle', 'gay friend', 'assassination', 'blood brother', 'dynasty', 'stabbing a woman', 'exploding truck', 'machine', 'skynet', 'accidental killing', 'drug humor', 'profanity', 'sexual humor', 'stand up comedy', 'directorial debut', 'nurse', 'old age home', '13 year old', '13th birthday', '30 year old', 'year 1987', '18 wheeler', 'businessman', 'historical fiction', 'kuwait', 'magic kit', 'stage magician', 'vegas', 'debutante', 'green beret', 'insanity', 'vietnam', 'hoax', 'mafia boss', 'mafiosi', 'mafioso', 'lifting someone into the air', 'paranormal phenomena', 'voice over letter', 'year 1818', 'little league', 'con artist', 'con man', 'deception', 'rivalry', 'seduction', 'shoe salesman', 'written by director', 'karate', 'apartheid', 'nation', 'nelson mandela', 'rugby', 'mob boss', 'thug', 'alcatraz', 'blanket', 'sesame street', 'damsel in distress', 'inspired by true events', 'minnesota', 'year 1990', 'psychic vision', 'staff physician', 'dying words', 'guitar', 'art thief', 'insurance', 'master thief', 'reference to iron maiden', 'candle', 'end of the world', 'record', 'dea', 'female impersonation', 'gender disguise', 'obesity', 'competition dancing', 'location in title', 'eugene oregon', 'obsession', 'employment agency', 'stage fright', 'childbirth', 'nude protest', 'nudist', 'floppy disk', 'modem', 'rival', 'repossession', 'surgery', 'bar mitzvah', 'marital breakup', 'nose job', 'rabbi', 'genital dismemberment', 'mulching someone', 'tooth fairy', 'woodchipper', 'michigan', 'child spy', 'notebook', 'apple computer', '3 dimensional', 'englishman abroad', 'reference to the doors', 'greek', 'manifestation', 'sphere', 'abduction', 'martial law', 'machete', 'army of the dead', 'necronomicon', 'reluctant hero', 'skeleton soldier', 'end credits roll call', 'sniper', 'ancient greece', 'forbidden love', 'hercules', 'chopping down a tree', 'hanging upside down', 'shopkeeper', 'vampire hunter', 'fair trade', 'non profit', 'rockumentary', 'danish', 'double agent', 'nazi occupation', 'resistance', 'ireland', 'sheep', '18th birthday', 'community center', 'resurrection', 'great white shark', 'killing an animal', 'british horror', 'pub', 'record collection', 'child', 'comic book', 'murderer', 'full frontal female nudity', 'menage a trois', 'caper', 'computer genius', 'security chief', 'fountain of youth', 'immortality', 'young man', '13 year olds', 'adolescence', 'peer pressure', 'werewolf bite', 'championship', 'harvard', 'physics', 'zamboni', 'jewel', 'b movie', 'masked killer', 'trick or treat', 'anarchist', 'greedy institution', 'prehistoric times', 'primitive', 'trio', 'crushed by a car', 'one against many', 'currency', 'film set', 'movie set', 'slow motion', 'urban legend', 'eyeball run over by car', 'latex gloves', 'suspension bridge', 'dead husband', 'forename as title', 'rodeo cowboy', 'falling through a glass roof', 'hitchcockian', 'prism', 'rented room', 'scrofula', 'sketching', 'sunrise', 'falling into water', 'medical doctor', 'roof leak', 'swimming in the ocean', 'beast', 'beastmaster', 'psychotronic', 'crocodile', 'garbage can', 'knife', 'paramount pictures', 'city symphony', 'diary', 'essay', 'surveillance', 'brutality', 'trap', 'amnesia', 'aquarium', 'fish', 'fish tank', 'gigolo', 'norwegian', 'generation y', 'strong female character', 'scepter', 'child custody', 'disability', 'mentally impaired man', 'single father', 'apocalypse', 'mercenary', 'urban setting', 'death row', 'inmate', 'oklahoma city', 'sexual promiscuity', 'innocence', 'negotiator', 'major', 'arab stereotype', 'broadway', 'demonic possession', 'exorcist', 'loss of innocence', 'goose', 'alter ego', 'teen movie', 'christmas party', 'held at gunpoint', 'arm tattoo', 'cauterizing a wound', 'dancing on a table', 'dinner party', 'book strap', 'racial discrimination', 'racial tension', 'stereotype', 'cider', 'fox', 'tale', 'mysterious villain', 'police chase', 'shooting range', 'police partner', 'serial murder', 'seven deadly sins', 'bartender', "new year's eve", 'punk', 'armenian', 'armenian genocide', 'customs agent', 'genocide', 'prayer', 'brontosaurus', 'classical music', 'dinosaur', 'dinosaur feature', 'earthquake', 'bus stop', 'community college', 'speech class', 'porn star', 'advice', 'black romance', 'reading a book', 'baseball movie', 'little league baseball', 'fantastic four', 'space', 'superheroine', 'business card', 'narcissism', 'child in peril', 'electroshock therapy', 'memory loss', 'faberge egg', 'trust', 'ancient astronaut', 'famous line', 'message from outer space', 'monolith', 'star child', 'graduation', 'graduation speech', 'tv studio', 'unemployment', 'bunker', 'car crash', 'kidnapping', 'minimal cast', 'modern silent movie', 'silent movie', 'coke', 'meeting future self', 'reference to spongebob squarepants', 'sony', 'bet', 'prom queen', 'competitiveness', 'creator', 'entrepreneur', 'facebook', 'intellectual property', 'bridge', 'midget', 'sideshow', 'british', 'indian', 'repair shop', 'nun', 'cheerfulness', 'garden gnome', 'magical realism', 'hyperthymesia', 'iguanodon', 'lemur', 'meteor', 'nesting grounds', 'single parent', 'supermarket', 'paris france', 'villain', 'brooklyn new york city', 'found footage', 'night vision', 'statue of liberty new york city', 'subway tunnel', 'dwarf', 'hobbit', 'gentleman', 'georgian', 'billionaire', 'playboy', 'trust fund baby', 'serum', 'dance competition', 'dance lesson', 'gang violence', 'alopecia', 'belfast northern ireland', 'irish republican army', 'scalper', 'robber', 'friendship between women', 'norway', 'steve', 'napoleonic wars', 'pursuit', 'royal navy', 'zorro', 'manhattan new york city', 'crop circle', 'intelligence', 'paranormal', 'phenomenon', 'universe', 'children', "children's tv show", 'adult humor', 'process server', 'roach', 'stoner', 'abandoned house', 'swing', 'forced to strip', 'porn actress', 'jackal', 'cop', 'bigger dreams', 'concert promoter', 'hotel maid', 'pep talk', 'rap concert', 'old west', 'stealing from the rich', 'filipino', 'guerrilla army', 'las vegas nevada', 'mexican border', 'embarrassing male nudity', 'embarrassing nudity', 'truck driver', 'contagion', 'cure', 'infection', 'virus', 'cthulhu', 'reputation', 'cruise', 'drag', 'female armpit hair', 'sorority', 'oil', 'tycoon', 'dating', 'divorcee', 'ex husband', 'ex wife', 'container', 'shipping container', 'storm', 'celebration', 'locust', 'repressed memory', 'cave painting', 'medical scanner', 'pregnant with an alien fetus', 'suspended animation', 'environment', 'gas chromatograph', 'nature', 'cult', 'gas station', 'supernatural ability', 'slacker', 'grief', 'hiking', 'loss of mother', 'mother daughter relationship', 'jujitsu', 'kickboxing', 'organ', 'ancient egypt', 'bracelet', 'scorpion', 'attacked by a plant', 'hill', 'wellesley college', 'car falling off a cliff', 'star died before release', 'christian', 'lesbian sex', 'oral sex', 'pregnant schoolgirl', 'adopted son', 'blog', 'clitoris', 'deep throat', 'porn industry', 'scissors', 'fire breathing dragon', 'militia', 'ex boyfriend ex girlfriend relationship', 'media tycoon', 'world war three', 'bookstore', 'simultaneity', 'angel', 'bethlehem', 'nazareth', 'virgin', 'chain', 'mysterious killer', 'warning', 'academic probation', 'school expulsion', 'smashing a guitar', 'bag of money', 'found money', 'screenplay adapted by author', 'snow', 'gay slur', 'hatred', 'celebrity criminal', 'public enemy number one', 'vegetable', 'waiter', 'arctic circle', 'volunteer', 'city name in title', 'place name in title', 'florida', 'toy store', 'canal', 'tailor', 'preacher', 'lesbianism', 'watching pornography', 'concierge', 'stalking', 'adult actor playing minor', 'politician', 'progeria', 'reference to spiderman', 'crane the bird', 'hoover dam', 'police department', 'rainy night', 'towel', 'chick', 'jail', 'monster movie', 'paper mill', 'pollution', 'dyed hair', 'fake tan', 'rock', 'twin', 'two headed person', 'couple', 'tropical', 'voyeurism', 'interracial couple', 'jesuit', 'pope', 'ss', 'helmet', 'medieval times', 'biological father', 'dad', 'product placement', 'pregnancy', 'question in title', 'crash', 'pistol', 'first lady', 'framed', 'crying during sex', 'humor', 'man with glasses', 'liverpool', 'protest', 'unrequited love', 'corrupt cop', 'murder of a police officer', 'hitler', 'no exit', 'war ruins', 'loss of memory', 'screwball', 'self discovery', 'social differences', 'abdication', 'forbidden city', 'republic', 'coastal town', 'iceland', 'snowstorm', 'winter storm', 'apprentice', 'orchestra', 'gloves', 'atrocity', 'central africa', 'ethnic warfare', 'united nations peace keeping force', 'united nations soldier', 'quest', 'the bronx', 'close up of breasts', 'power plant', 'sabotage', 'store clerk', 'video store', 'family business', 'jealousy', 'spiritual', 'good deed', 'protective male', 'prefect', 'masked man', 'moving', 'tokyo japan', 'toxic avenger', 'troma', 'farming', 'flesh eating', 'food industry', 'gluttony', 'electrocuted in bathtub', 'attack', 'pearl harbor', 'u.s. navy', "character says now that you're on the case i hope we're going to have some gratuitous sex and violence", 'cruise missile', 'missile', 'heiress', 'cia agent', 'consulate', 'safe house', "coup d'etat", 'mutiny', 'madagascar', 'zoo', 'haunted house', 'implied incest', 'squeegee', 'tied to a chair', 'mozart', 'musical prodigy', 'royal court', 'talent', 'airport security', 'confidence', 'iphone', 'internet', 'make up', 'publicist', 'purim', 'african', 'dialect', 'interpreter', 'threat', 'brains', 'exploding head', 'swarm behavior', 'spider', 'tarantula', 'super villain', 'tentacle', 'clown', 'elephant trainer', 'trapeze', 'trapeze artist', 'dynamite', 'ku klux klan', 'n word', 'atheist', 'female masturbation', 'lucifer', 'butcher', 'mute character', 'subway', 'alcoholic', 'kazakhstan', 'nomad', 'bare butt', 'defecation', 'father in law son in law relationship', 'sexual innuendo in title', 'dissection', 'medical school', 'medical student', 'benzedrine', 'cactus', 'poem', 'year 1949', 'highlander', 'illusion', 'immortal', 'playing with gun', 'working class family', 'archipelago', 'guillotine', 'redemption', 'action heroine', 'falling from height', 'mind control', 'trainhopping', 'truth serum', 'based on memoir', 'cambridge university', "lou gehrig's disease", 'physicist', 'hazmat suit', 'based on sketch comedy', 'evil corporation', 'dump', 'show', 'spectator', 'tire', 'jackpot', 'aborigine', 'colonialism', 'fence', 'half caste', 'class', 'cornwall connecticut', 'pawtucket rhode island', 'mormon', 'grindhouse', 'suffering', 'wales', 'bible', 'key', 'slum', 'wallet', 'blind prophet', 'sirens', 'texan', 'spitting', 'sushi', 'cold war', 'address as title', 'parents', 'horror movie remake', 'undead sex', 'undead sexuality', 'club', 'study', 'talmud', 'yeshiva', 'bat', 'hawaiian shirt', 'pregnant', 'gold rush', 'mirror', 'abuse', 'pickpocket', 'estranged wife', 'medication', 'graduate student', 'sprite', 'uncle nephew relationship', 'disfigurement', 'harpooner', 'motorboat', 'sequel to cult film', 'collector', 'ghost hunter', 'glass house', 'ostrich', 'waterhole', 'wildebeest', 'zebra', 'kissing in an elevator', 'mythical hero', 'silent protagonist', 'car bomb', 'card cheat', 'cash', 'gambling syndicate', 'economy', 'individual versus society', 'christianity', 'reference to justin bieber', 'reference to taylor swift', 'worship', 'abusive stepfather', 'attempted rape', 'food shortage', 'starving child', 'desire', 'resort town', 'sales', 'dance crew', 'mortgage', 'evil man', 'absent father', 'asian american', 'escaped convict', 'motorcycle', 'male full frontal nudity', 'based on cult comic book', 'home alone', 'talking cat', 'meeting', 'statistics', 'voice over', 'play', 'playwright', 'epidemic', 'paradox', 'freemason', 'jack the ripper', 'opium', 'victorian era', 'new york cosmos', 'professional sports', 'soccer', 'soccer team', 'northern ireland', 'consciousness', 'power outage', 'technology', 'jurassic park', 'paleontologist', 'based on cult favorite', 'disease', 'female hero', 'breaking and entering', 'gallows', 'hanging', 'salmon', 'high school teacher', 'teacher inspires students', 'teaching', 'changing the future', 'invention', 'nonlinear timeline', 'compulsive liar', 'fired from a job', 'gift', 'substance abuse', 'lottery', 'amputee', 'anti romantic comedy', 'sex comedy', 'comedian', 'craigslist', 'stand up comedian', 'climate', 'global warming', 'science', 'colosseum', 'homeless man', 'tombstone', 'surfing', 'surfing contest', 'wave', 'bail bond', 'jeans', 'shot in the butt', 'mennonite', 'shower', 'punched in the nose', 'rafting', 'river', 'white water rafting', 'cuba', 'cobra', 'medallion', 'waterfall', 'consensual sex', 'surfer', 'humiliation', 'man child', 'moped', 'coven', 'occult', 'outcast', 'spell', 'cookie', 'microfilm', 'cheerleader', 'valedictorian', 'recession', 'record company', 'rock star', 'plant', 'soil', 'life insurance policy', 'mother', 'recluse', 'shy', 'famous score', 'vietnam war', 'perversion', 'chainsaw', 'mansion', 'battle of agincourt', 'shakespeare play', 'sword and shield', 'bachelorette party', 'drug use', 'overdose', 'wedding dress', 'dalai lama', 'himalaya', 'tibet', 'arcade', 'candy', 'glitch', 'medal', 'lebanon', 'pittsburgh steelers', 'baltimore colts', 'kicked in the crotch', 'tough girl', 'lost world', 'squirrel', 'weasel', 'route 66', 'tow truck', 'contract', 'devil', 'soul', 'holy land', 'knight templar', 'monastery', 'radical transformation', 'dc comics', 'nuclear disarmament', 'superman', 'call in', 'date', 'hot air balloon', 'show producer', 'ant', 'grasshopper', 'overhearing', 'geek', 'reference to janet jackson', 'drug dealing', 'car chase', 'plot twist', 'police van', 'rear ending a car', 'cheerleading', 'teen comedy', 'teenage romance', 'community', 'fem bot', 'tv producer', 'abortion clinic', 'clinic', 'coyote', 'quarterback', 'immigrant', 'eighteen wheeler', 'illegal street racing', 'trucker', 'hypochondriac', 'television', 'college girl', 'midlife crisis', 'pittsburgh pennsylvania', 'courtship', 'quaker', 'health club', 'superhero', 'toxic waste', 'baltimore orioles', 'boston red sox', 'ups', 'world series', 'kissing', 'arctic', 'rhedosaurus', 'british prime minister', 'female prime minister', 'prime minister', 'black militant', 'frustration', 'pizza parlor', 'pizzeria', 'city by night', 'driving at night', 'taxi', 'family vacation', 'national lampoon series', 'clinical trial', 'neuropharmacology', 'side effect', 'nerd', 'animal rights', 'congresswoman', 'doorman', 'transvestism', 'basement', 'library', 'track', 'nihilism', 'rug', 'white russian', 'october', "child's point of view", 'real estate', 'washing machine', 'contest', 'refusing to eat', 'starving oneself', 'brother sister relationship', 'shrimper', 'south carolina', 'loneliness', 'playing against type', 'retrograde narrative', 'lava', 'volcano', 'caramel', 'suitor', 'f word', 'umbrella', 'attorney', 'cat and mouse', 'pan am', 'boogeyman', 'hometown', 'workhouse', 'godparent', 'pacific palisades', 'rodeo', 'pandemic', 'lesbian mother', 'lesbian parent', 'lesbian relationship', 'lesbian romance', 'child abuse', 'basketball coach', 'box', 'yard sale', 'marigold', 'retiree', 'bad luck', 'hole', 'fate', 'federation', 'cannibalism', 'condom', 'hook for hand', 'satanic ritual', 'driving a car down stairs', 'duct tape gag', 'held captive', 'sailing', 'parole', 'parole officer', 'puppy', 'final concert', 'thanksgiving', 'breast cancer', 'comma in title', 'detroit michigan', 'firefighter', 'master of disguise', 'football team', 'gang leader', 'italian woman', 'period piece', 'purgatory', 'bee', 'dream sequence', 'scatological humor', 'squashed bug', 'half sister', 'medicine', 'court', 'intrigue', 'spain', 'movie producer', 'sting operation', '1969 dodge charger', 'bikini', 'misogynist', 'blindfolded woman', 'control', 'male dominance', 'sex game', 'sex in public', 'heir', 'falcon', 'accountant', 'tobacco industry', 'subjective camera', 'teen angst', 'troubled teen', 'fired from the job', 'person in car trunk', 'racial impersonation', 'south asian', 'street gang', 'barn owl', 'owl', 'ruler', 'power', 'driving range', 'tournament', 'winnebago', 'hinduism', 'marriage engagement', 'caught having sex', 'telephone call', 'voice mail', 'unplanned pregnancy', 'website', 'desert planet', 'dune', 'giant worm', 'spice', 'exodus', 'moses', 'actor playing multiple roles', 'axe', 'dark comedy', 'sadism', 'handheld camera', 'impostor', 'rap', 'rock paper scissors', 'august', 'patriarch', 'bank robber', 'crime spree', 'good versus evil', 'holcomb kansas', 'truman capote', 'gay republican', 'bare midriff', 'cemetery', 'farmhouse', 'radiation', 'running out of gas', 'premonition', 'roller coaster', 'tough cop', 'spin off', 'conclave', 'reference to bernini', 'reference to raphael', 'bobsled', 'jamaica', 'kansas city', 'female bassist', 'female drummer', 'female guitarist', 'female singer', 'portland oregon', 'barricade', 'innkeeper', 'cigar smoking', 'tombstone arizona', 'street hustler', 'fake family', 'marijuana', 'drug culture', 'filthy toilet', 'heroin addict', 'heroin withdrawal', '2020s', 'near future', 'one word series title', 'year 2027', 'mythology', 'prince', 'trojan', 'troy', 'nude nun', 'russian nobleman', 'showcase', 'female stockinged feet', 'female stockinged soles', 'nude pantyhose', 'pantyhose', 'anger', 'family reunion', 'father son reunion', 'movie making', 'maximum security prison', 'addiction', 'diet pill', 'drug addiction', 'fast motion scene', 'ex partner', 'solar system', 'starship', 'year 2047', 'bisexual', 'explicit sex', 'knight', 'mentor', 'foreboding', 'hideout', 'aunt niece relationship', 'building', 'returning character killed off', 'six word title', 'writer director', 'class conflict', 'edwardian era', 'sexual awakening', 'hiding', 'device', 'disaster', 'snow white', 'estranged father', 'genetics', 'lie', 'screenwriter', 'vulgarity', 'high school basketball', 'dying', 'to do list', 'twenty something', 'underwear', 'breakfast cereal', 'department store ride', 'face painting', 'sponge bob square pants slippers', 'phantom', 'skull', 'based on poem', 'the huns', 'reference to kurt cobain', 'year 1994', 'obscenity', 'vacant lot', 'barricading a door', 'injection in leg', 'taser', 'presumed dead', 'age difference', '3d', 'cossack', 'warden', 'recording', 'saxophone', 'goblet', 'assistant', 'frankenstein', 'costume drama', 'final battle', 'napoleon', 'napoleon bonaparte', 'arizona', 'dementia', 'married man', 'sun city arizona', 'distress signal', 'family in danger', 'stealing a car', 'mortal kombat', 'disobeying orders', 'evangelist', 'revival', 'traveling salesman', 'chicken', 'locker room', 'roller skates', 'tubesocks', 'chosen one', 'enchanted object', 'radio', 'highwayman', 'nell gwynne', 'lap dance', 'lap dancing', 'striptease', 'labyrinth', 'maze', 'red light', 'debauchery', 'lust', 'palace', 'saddam hussein', 'gossip', 'cult tv', 'sandman', 'spider man', 'symbiote', 'venom', 'anchorman', 'newsroom', 'tv station', 'new home', 'rainbow', 'reference to avenged sevenfold', 'reference to slayer', 'switching lights on and off', 'heart surgeon', 'pop chart', 'gambler', 'claw fight', 'commando', 'wolverine the character', 'country club', 'summer vacation', '1000000 b.c.', 'lawn mowing', 'stone age', 'caddy', 'gopher', 'loss of husband', 'suntan pantyhose', 'male star appears nude', 'fashion magazine editor', 'abuse of power', 'prison guard', 'race relations', 'social problem', 'acting', 'briefs', 'reference to cuba gooding jr', 'reference to netflix', 'written by star', 'female removes her clothes', 'fondling', 'drug deal', 'sex on stairs', 'mad scientist', 'actress shares first name with character', 'car wash', 'd.j.', 'internship', 'stockbroker', 'ghetto', 'dark horse comics', 'reference to david letterman', 'robert crumb', 'dancing sheep', 'fetish', 'musical scene', 'racial humor', 'snake poison', 'space shuttle', 'chores', 'father figure', 'helicopter parents', 'in laws', 'mormon church', 'proposition', 'ghostbuster', 'ghostbusters', 'reboot', 'super strength', 'vision', 'manchuria', 'new york city new york', 'carl jung', 'psychoanalysis', 'sigmund freud', 'government agent', 'german', 'heavy water', 'paper bag', 'female bare foot', 'picket fence', 'baseball practice', 'gunfighter', 'poker game', 'year 1879', 'monopoly', 'brain capacity', 'fictional drug', 'taking a pill', 'libya', 'u.s. ambassador', 'irish civil war', 'irish free state', 'sinn fein', 'operating system', 'fictitious country', 'latin america', 'reference to fidel castro', 'florist', 'motivational speaker', 'self help guru', 'seminar', 'faked death', 'japanese', 'poison', 'secret headquarters', 'carnival', 'rookie cop', 'intervention', 'killer doll', 'vixen', 'lifting female in air', 'lifting male in air', 'special education', 'able to see the dead', 'surprise ending', 'classic literature', 'censorship', 'heil hitler', 'science runs amok', 'supernatural power', 'battlefield', 'drug dealer', 'neighborhood', 'based on cartoon', 'cgi film', 'computer animation', 'motorcycle gang', 'climate change', 'global', 'polar bear', 'scientific', 'death of child', 'sniper rifle', 'ice skater', 'crook', 'creature feature', 'hijacker', 'shot in the leg', 'based on the bible', 'nativity', '9 year old', 'illinois', 'new job', 'tap dancing', 'undressing', 'based on toy', 'dual personality', 'evil businessman', 'good cop bad cop', 'lego', 'moroccan', 'xenophobia', 'possession', 'courtesan', 'kama sutra', 'police shootout', 'geese', 'idaho', 'national park', 'park', 'detroit tigers', 'perfect game', 'pitcher', 'debt', 'treasure hunter', 'yacht', 'negotiation', 'deep space', 'imaginary friend', 'outback', 'town', 'lingerie', 'extraterrestrial', 'flash mob', 'target', 'mini skirt', 'marital rape', 'awol', 'demand', 'roses', 'chocolate', 'chocolate shop', 'espionage', 'secret mission', 'spy film', 'chimpanzee', 'elephant', 'shaman', 'documentary crew', 'mass suicide', 'survivor', 'tied up while barefoot', 'viking', 'main character dies', 'black cat', 'king henry viii', 'necktie', 'rock music', 'tuxedo', 'cattle', 'rejecting a marriage proposal', 'talking to a dog', 'vomiting on someone', 'lightning', 'nba', 'shoe', 'chronic traumatic encephalopathy', 'concussion', 'pathologist', 'car hitting pedestrian', 'fan', 'nineteenth century', 'reversal of fortune', 'appendicitis', 'clique', 'lip gloss', 'based on play', 'ceo', 'dubai', 'product', 'hebrew', 'hacking', 'hijacking', 'elevator girl', 'spacecraft accident', 'guardian angel', 'second chance', 'small town life', 'prison escape', 'solitary confinement', 'police officer killed', 'police officer shot in the chest', 'police officer shot in the forehead', 'police officer shot in the head', 'based on stage musical based on novel', 'alternate reality', 'girl gang', 'rambo', 'arrest', 'raid', 'europe', 'frog', 'gulag', 'prison break', 'puppet', 'twin brother and sister', 'douglas macarthur', 'inchon', 'invasion', 'elongated cry of no', 'friends become enemies', 'kicked in the butt', 'robotic midwife', 'opera singer', 'platonic marriage', 'cruelty', 'tension', 'flash photography', 'game of death', 'jigsaw', 'pig mask', 'claw crane', 'american west', 'horse riding', 'begins with narration', 'female warrior', 'inbetwequel', 'jedi knight', 'war against machines', 'holiday horror', 'stonehenge', 'music producer', 'duty', 'submarine movie', 'u boat', 'anthropomorphic animal', 'furry', 'autistic son', '1830s', 'baptism', 'intolerance', 'sibling rivalry', 'life magazine', 'female explicit nudity', 'penetration', 'con', 'grifter', 'organized crime', 'shot in the forehead', 'kick the can', 'front desk', 'apostrophe in title', 'teen sex comedy', 'krill', 'communist', 'three musketeers', 'twin brother', 'countryside', 'scar', 'golf ball', "grandmother's house", 'evil queen', 'composer', 'self esteem', 'casual sex', 'singles', 'mom', 'pta', 'reference to mad max', 'scene during end credits', 'confession', 'flickering light', 'stealing', 'traitor', 'antidote', 'llama', 'brumby', 'hammer', 'monaco', 'tony stark', 'controversy', 'doppelganger', 'group therapy', 'self mutilation', 'twin actresses for twin sisters', 'drug smuggling', 'interlinked stories', 'lifeguard', 'talent show', 'amphibian', 'frog prince', 'male nurse', 'gulf war', 'deja vu', 'time loop', 'trapped in a time loop', 'size positive', 'thin', 'wet dream', 'crime boss', 'inner city', 'parent', 'first person narration', 'prison cell search', 'wrongful imprisonment', 'shop', 'commune', 'environmentalism', 'healing power', 'marvel comics', 'mecha', 'regeneration', 'self healing', '14 year old', 'gadget', 'home invasion', 'police officer shot', 'detention', 'defense attorney', 'cartographer', 'mathematician', 'numbers', 'pattern', 'using a metal detector on a beach', 'martian', 'cleavage', 'comic violence', 'intentionally bad', 'stylized violence', 'irreverence', 'columbine', 'columbine high school killings', 'firearm', 'school shooting', 'girl group', 'fountain', 'law firm', 'high school graduation', 'gi joe', 'bow and arrow', 'rifle', 'ruins', 'missing wife', 'murder suspect', 'adult entertainment industry', 'san fernando valley california', 'bomber', 'flask', 'hell', 'the devil', 'bisexuality', 'manufacturing', 'sibling', 'el salvador', 'reagan administration', 'war journalism', 'hdtv', 'elm street', 'reservation', 'soccer coach', 'soccer mom', 'brainwashed assassin', 'breakfast', 'manitoba', 'science fiction writer', 'flying', 'gay subtext', 'machismo', 'excavation', 'relic', "character's point of view camera shot", 'fire station', 'television reporter', 'selfishness', 'turn of the century', 'smithsonian', 'terminal cancer', 'board of directors', 'industry', 'mail', 'cartoonist', 'zodiac', 'zodiac killer', 'lsd', 'missing friend', 'mk ultra', 'radio broadcast', 'castration', 'fast food', 'indian burial ground', 'bullfight', 'bullfighting', 'action hero', 'forced prostitution', 'title spoken by character', 'high society', 'florida keys', 'reference to douglas fairbanks', 'reference to jesse owens', 'reference to randolph scott', 'year 1874', 'ski resort', 'snowboard', 'snowboarding', 'gunshot', 'hunt', 'kindergarten teacher', 'australian', 'scholarship', 'bowling', 'rajah', 'sphinx', 'spleen', 'villainess', 'architect', 'superhero spoof', 'around the world', 'tattoo', 'closet', 'psychiatric hospital', 'real life sisters playing sisters', 'crepe suzette', 'mata hari spoof', 'slug', 'bellhop', 'alien friendship', 'flying car', 'loner', 'talisman', 'dream analysis', 'guilt complex', 'post traumatic stress disorder', 'car fire', 'cockroach', 'new year', 'soviet', 'live studio audience', 'christmas tree', 'fair', 'stranded', 'masked villain', 'tv', 'michael jai white', 'steve austin', 'partnership', 'vigilante', 'comic book history', 'comics', 'spawn', 'walking dead', 'birth', 'misadventure', 'suitcase', 'lycanthrope', 'shapeshifting', 'werewolf transformation', 'train wreck', 'blade the character', 'mutation', 'affair', 'altering history', 'image consultant', 'one two combo', 'blockade', 'crisis', 'cross', 'doused with gasoline', 'cross dresser', 'motion sickness', 'irony of fate', 'landlady', "kissing someone's boot", 'midsummer', 'telling someone to shut up', 'valet', 'choreographer', 'locker', 'funfair', 'luna park', 'table football', 'toy comes to life', 'wig', 'brain', 'fiance', 'laboratory', 'surgeon', 'karma', "boss' daughter", 'crush', 'house sitter', 'housesitting', 'love interest', 'opec', 'revolutionary', 'black and white', 'survivalist', '18th century', 'highlands', 'scotland', 'sexual cruelty', 'sword fight', 'skunk', 'summer camp', 'wall', 'death personified', 'fear of death', 'answer to prayer', 'breast expansion', 'pleading with god', 'walking on water', 'promiscuity', 'reference to isaac newton', 'auction', 'montreal', 'violin', 'country music', 'pink panties', 'relapse', 'songwriter', 'miracle', 'bishop', 'loan shark', 'poker', 'reference to christy turlington', 'mariachi', 'american expatriate', 'casablanca morocco', 'gestapo', 'visa', 'clerk', 'hockey', 'video', 'canuxploitation', '1910s', 'artificially created woman', 'taxi driver', 'heart transplant', 'family abandonment', 'native american protagonist', 'native american reservation', 'heroism', 'super soldier', 'world domination', 'blind', 'long take', 'leader', 'somalia', 'marital problem', 'money problems', 'nervous breakdown', 'bus trip', 'parental love', 'surprise visit', 'train ride', 'tribute', 'domestic violence', 'unwanted pregnancy', 'neurosis', 'boy scouts', 'boy scouts camp', 'young love', '1810s', 'high tech', 'resentment', 'kissing in the rain', 'minion', 'undercover spy', 'woman agent', 'death of brother', 'talking with the dead', 'aroused', 'pornography documentary', 'pornstar', 'hockey mask', 'babysitter', 'ganja', 'kumbh mela', 'little boy', 'kissing a dead body', 'repeated dialogue', 'sex in pool', 'spring break', 'frozen time', 'obese man', 'remote control', 'stopped time', 'englishman', 'leaving flowers on a grave', 'one last job', "visiting wife's grave", 'mental illness', 'shot to death', 'alternative energy', 'energy', 'green', 'flatulence', 'interracial marriage', 'k2', 'auto theft', '14th century', 'revolt', 'tyranny', 'secret society', 'riding the rails', 'corporate greed', 'natural gas', 'female chef', 'authority', 'boss subordinate relationship', 'fast food restaurant', 'gullibility', 'kissing while having sex', 'reference to princess diana', 'wedding band', 'wedding ring', 'demo tape', 'directed by cast member', 'grieving father', 'guitar player', 'blackout', 'child pornography', 'microchip', 'silicon valley', '23rd century', 'dual role', 'cattle drive', 'dominican', 'teenage pregnancy', '16th century', 'practice', 'intimate partner homicide', 'flight', 'plane', 'templar', 'manipulative behavior', 'modern day adaptation', 'narcissistic woman', 'promiscuous woman', 'survival tactics', 'audio surveillance', 'pentagon', '50s', 'illegitimate pregnancy', 'part animation', 'love at first sight', 'mumbai india', 'reference to edward scissorhands', 'sleigh', 'credit card', 'cross country', 'serendipity', 'miser', 'muppet', 'flight attendant', 'kentucky', 'shoes', 'abu ghraib', 'housemaid', 'servant', 'caveman', 'strict father', 'african american gangster', 'falling to death', "in over one's head", 'corrupt sheriff', 'flash drive', 'reference to franz beckenbauer', 'student visa', 'tropical island', 'belching', 'reference to nicki minaj', 'sneezing', 'estranged couple', 'martial arts master', 'based on comic', 'dark hero', 'image comics', 'crash survivor', 'brazilian', 'cockatoo', 'macaw', 'hearse', 'tripod', 'alcoholic father', 'stepmother', 'swaziland', 'unfaithful wife', 'greek mythology', 'mousetrap', 'stick fight', 'birthday wish', 'father disappoints child', 'workaholic', 'animate object', 'reference to macgyver', 'whispering', 'lifeboat', 'sailing ship', 'scooby doo', 'voodoo', 'actor playing himself', 'boots', 'penis', 'womb', 'chrysler building manhattan new york city', 'delusion', 'u.s. air force', 'child protagonist', 'child swearing', 'ambiguous ending', 'fight club', 'five sisters', 'isolation', 'self injury', 'cheating', 'economics', 'human nature', 'sumo wrestling', 'ares', 'hades', 'titan', 'underworld', 'zeus', 'cliff', 'abandoned', 'depot', 'newfoundland new jersey', 'hot dog bun', 'needle', 'gay pride', 'highway travel', 'road movie', 'bollywood', 'jane austen', 'convent', 'cable guy', 'foreplay', 'chinatown', 'illegal immigrant', 'triad', 'formula 1', 'sex with a nurse', 'celebrity', 'consumerism', 'media manipulation', 'merchant', 'pound of flesh', 'antibes', 'reunion island', 'man in a wheelchair', 'year 2000', 'year 2002', 'socialite', 'captain america', 'marvel cinematic universe', 'live in concert recording', 'live performance', 'saxophonist', 'virginity', "based on children's book", 'pole dancer', 'basketball movie', 'tv commercial', 'tv news', 'black magic', 'refusal to kill', 'wuxia', 'catch 22', 'kneed in the groin', 'border', 'well', 'homoerotic fight', 'homosexual subtext', 'wcw wrestling', 'basketball team', 'grateful dead', 'lithuanian', 'ellipsis in title', 'long title', 'jewish identity', 'pin', 'ruby', 'bounty', 'miner', "valentine's day", 'american civil war', 'irish american', 'german soldier', 'rage', 'west virginia', 'blogger', 'reference to the shawshank redemption', 'tasered', 'french canadian', 'hockey player', 'machinist', 'coin collector', 'u.s. army', 'fistfight', 'prison bus', 'client', 'lesson', 'mississippi river', 'fashion model', 'model', 'retired', 'hostel', 'slovakia', 'peanuts', 'red hair', 'baseball player', 'brooklyn dodgers', 'major league baseball', 'bicycle', 'new neighbor', 'drawing', 'flanders', 'vice president', 'flare gun', 'human versus vampire', 'rapist', "martha's vineyard", 'older woman younger man relationship', 'weekend', 'hulk', 'opening action scene', 'alien abduction', 'lightning bolt', 'poseidon', 'stealth', 'top secret', 'call girl', 'charlatan', 'fortune teller', 'manuscript', 'deal', 'district attorney', 'meteorologist', 'snowed in', 'snowplow', 'weather', 'ethiopia', 'marksman', 'rangefinder', 'beard', 'woman in lingerie', 'grandmother grandson relationship', 'heart attack', 'charity', 'taxi dancer', 'choir', 'leper', 'lighthouse', 'theme park', 'kicked in the face', 'punched in the face', 'woman punching a man', 'indifference', 'beauty pageant', 'pageant', 'machine gun', 'police vigilantism', 'safe deposit', 'bestiality', 'brother sister incest', 'fisting', 'hit on the head with a pipe', 'support group', 'fraud', 'army cid', 'woman in military', 'dog saved from a fire', 'growing up', 'multiple narrators', 'famous twist', 'rebel', 'snowy landscape', 'foreign', 'orgasm', 'swimming', 'medical scrubs', 'purple gloves', 'person in a car trunk', 'suitcase bomb', 'ronin', 'shogun', 'death of wife', 'dracula', 'turk', 'culture shock', 'interracial relationship', 'madea series', 'ponzi scheme', 'accused of witchcraft', 'cat people', 'leopard', 'slip the undergarment', 'federal reserve bank', 'shredder', 'african american protagonist', 'name in title', 'talk show host', 'curiosity', 'freak', 'physical deformity', 'quarantine', 'previous marriage', 'airborne troops', 'allies', 'operation market garden', 'paratrooper', 'manipulative personality', 'murder of wife', 'neglected wife', 'grandmother', 'human versus spider', 'spider feature', 'woman sheriff', 'slapstick comedy', 'deacon', 'throat slitting', 'glass', 'wager', 'gothic', 'van helsing', 'hypnotherapy', 'lost painting', 'red riding hood', 'woodsman', 'idol', 'wyoming', 'cliche', 'exchange student', 'archeological dig', 'ark of the covenant', 'cartoon tiger', 'tigger', 'appeasement', 'frontier justice', 'street shootout', 'partisan commentary', 'political propoganda', 'reenactment', 'inspired by radio program', 'pirate broadcasting', 'pirate radio', 'pirate radio station', 'canadian gothic', '12th century', 'crusader', 'drunken sex', 'mashed potatoes', 'pancakes', 'tequila', 'dying repeatedly', 'buddy', 'prequel to cult favorite', 'junkie', 'french and indian war', 'red panties', 'divorced couple', 'reference to giuseppe verdi', 'senility', 'bare chested boy', 'car explosion', 'scotland yard', 'film actress', 'nosferatu', 'evil dead', 'hometown hero', 'philadelphia pennsylvania', 'astoria', 'kindergarten', 'guantanamo', 'lance corporal', 'bonn germany', 'failed kidnapping', 'handcuffs', 'projectile', 'chemical company', 'dental assistant', 'life', 'remake by original director', 'stop motion animation', 'autistic child', 'child in danger', 'paternity', 'testicles', 'disc jockey', 'archdiocese', 'land developer', 'real estate developer', 'court martial', "no man's land", 'political conspiracy', 'sound', 'impersonation', 'see through dress', 'woman on top', 'blender', 'computer screen', 'skype', 'webcam', 'gay love', 'irish adoption board', 'harmonica', 'killing', 'woman in bathtub', '5th century', 'capri', 'roman legion', 'bond girl', 'record player', 'wings', 'colombian', 'drug mule', 'stomach', 'crucifixion', 'loincloth', 'reference to god', 'grimdark', 'mutilated corpse', 'space marine', 'ultramarines', 'warhammer 40k', 'clouseau', 'panther', 'pink', 'pink panther', 'missing brother', 'venus flytrap', 'dollar bill', 'double entendre', 'interspecies sex', 'butt', 'downward duck', 'kingfisher', 'poacher', '1520s', 'aztec', 'spanish colonization', 'spanish conquest', 'brutal cop', 'thai boxing', 'vengeance', "april fool's day", 'psycho', 'vacuum cleaner', 'conjoined twins', 'separation', 'fur coat', 'misfit', 'home office', 'released from a mental hospital', 'trauma', 'mule', 'tennis court', 'the three stooges', 'cape kennedy', 'news footage', 'reference to christopher columbus', 'trapped in space', 'housewife', 'corrupt police', 'fireworks', 'kiss', 'to be continued ending', 'usaf', 'capitalist', 'child cancer', 'falling into a hole', 'inspiring', '1850s', 'dark secret', 'heart', 'organ donor', 'private detective', 'superhero team', 'x men', 'year 1983', 'campaigning', 'candidate', 'dead body in a bathtub', 'dead body in bathtub', 'inn', 'alternate timeline', 'cyborg', 'doggystyle sex', 'rough sex', 'sodomy', 'violent sex', 'costa rica', 'online gambling', 'deal with the devil', 'narrated by character', 'evil industrialist', 'ex girlfriend', 'political leader', 'digging a grave', 'grave digging', 'investigator', 'barber', 'barbershop', 'blaxploitation', 'muse', "tiffany's", 'actress playing herself', 'american in the uk', 'jock strap', 'nintendo game boy advance', 'competing businesses', 'instant messaging', 'shop owner', 'laser gun', 'astronomer', 'comet', 'governess', 'orchestral music score', 'poltergeist', 'stairway', 'counterfeit', 'counterfeit money', 'black cop', 'hostage negotiator', 'boating accident', 'daughters', 'refugee', 'criminal profile', 'madman', 'tabloid reporter', 'tragic villain', 'drummer', 'reference to jesus christ', 'contemporary west', 'neo western', 'headbanger', 'metallica', 'roadie', 'song in title', 'standing on the roof of a car', 'the beatles', 'fable', 'old man', 'chicago illinois', 'el train', 'ephebophile', 'pederasty', 'pedophilia', 'playing hooky', 'shaving', 'pressure', 'college football', 'surrender', 'hired killer', 'bad acting', 'pulp fiction', 'resistance fighter', 'voyager golden record', 'conquest', 'greek myth', 'teeth', 'assassination attempt', 'capitalism', 'cuban american', 'cuban refugee', 'tech noir', 'ex boyfriend', 'morning after', 'prince charming', 'quarrel', 'fight to the death', 'self survival', 'protective father', 'shrew', 'extreme sports', 'comedy of manners', 'false identity', 'period drama', 'amateur film', 'sexual innuendo', 'struggling parent', 'advertising', 'biblical', 'book of esther', 'esther', 'treachery', 'fabrication', 'fake', 'bromance', 'cult figure', 'femme fatale', 'indiana jones', 'unsubtitled foreign language', 'year 1957', 'cyberpunk', 'illegality', 'snuff film', 'family dinner', 'husband wife conflict', 'husband wife kiss', 'egyptian', 'egyptian god', 'egyptian goddess', 'sharpshooter', 'the end', 'german democratic republic', 'germany', 'halfway house', 'schizophrenia', 'blue light', 'light', 'co worker', 'leukemia', 'oncology', 'sick child', 'number', 'rag doll', 'acid trip', 'gameshow', 'hot tub', 'time lord', 'weed', 'bomb shelter', 'cuban missile crisis', 'naive man', 'battleship', 'message', 'yamato', 'duchess', 'duke', 'kenya', 'tween girl', 'san diego california', 'after dark horrorfest', 'a cappella', 'anthropomorphic mouse', 'marshmallow', 'albanian', 'revenge motive', 'teenage daughter', 'freight train', 'runaway train', 'train engineer', 'wax', 'pg 13 sequel to r rated franchise', 'march', 'million man march', 'animal abuse', 'blue clay', 'diversion', 'ends with biographical notes', 'ends with historical notes', 'human branding', 'man slaps a woman', 'manhunt', 'rooftop', 'brothel', 'ex lover', 'haunted by the past', 'autobiography', 'german shepherd', 'angry spirit', 'aura', 'electronic voice phenomena', 'electronics', 'near death experience', 'mexico city', 'new jersey', 'fascist', 'sound barrier', 'space program', 'u.s. space program', 'award', 'high school reunion', 'cleopatra', 'attention deficit disorder', 'bed', 'cheetah', 'air cavalry', 'vietnamese', 'suicidal thoughts', 'movie in title', 'death of protagonist', 'follow that car', 'la dolce vita', 'corrupt politician', 'chainsaw murder', 'human monster', 'music score composed by director', 'obscene finger gesture', 'bhutan', 'buddhist', 'monk', 'weaver', 'child star', 'star', 'gun fu', 'fantasy life', 'teenage crush', 'killed in police car', 'police officer shot in the leg', 'police officer shot through the heart', 'whistleblower', 'feud', '41st century', 'space opera', 'classical composer', 'dark fantasy', 'fairy', 'punk rocker', 'televangelist', 'chorus girl', 'voice over narration', 'freedom', 'catholic guilt', 'epilepsy', 'italian american', 'bust', 'tragic hero', 'apple', 'necromancy', 'sister act', 'whistling', 'wisecrack humor', 'e mail', 'evil spirit', 'electronic music score', 'fashion industry', 'publishing', 'sherlock holmes', 'police sergeant', 'babe scientist', 'colon in title', 'five word title', 'temper', 'purity', 'stepbrother', 'temptation', 'vagina dentata', 'british agent', 'kgb', 'soviet general', 'dead girl', 'granddaughter', 'morgue', 'storage unit', 'reference to superman', 'blueberry pie', 'pie', 'art model', 'nude model', 'three sisters', 'nude woman murdered', 'trailer home', 'accident', 'electronic devices', 'gynecologist', 'little black book', 'plan', 'set up', 'fabricated facts', 'stephen glass', 'the new republic', 'expecting', 'ejected from a moving vehicle', 'gun held to head', 'shot multiple times', 'strangulation', 'killing a pet', 'madame butterfly', 'semi autobiographical', 'estrangement', 'missouri', 'mormon temple', 'bathing a baby', 'cheese', 'middle school', 'courthouse', 'private investigator', 'krug', 'dangerous friend', 'monster as victim', 'pink bra and panties', 'red bra and panties', 'gang initiation', 'gunplay', 'hazing', "shakespeare's romeo and juliet", 'battle of the sexes', 'ex husband ex wife relationship', 'punching a heavy bag', 'tied to a tree', 'collar', 'yogi bear', 'brainwashing', 'construction worker', 'sunglasses', 'lama', 'tibetan', 'labor', 'labor union', 'master', 'oklahoma', 'tennessee', 'wal mart', 'funeral home', 'hermit', 'dead children', 'missing child', 'native', 'paraplegic', 'daydream', 'snow leopard', 'catering', 'wasp', '1190s', 'archer', 'king of england', 'robin hood', 'group', 'dance contest', 'loss of sister', 'step dancing', 'miscegenation', 'showboat', 'cmnf', 'cmnf scene', 'mermaid', 'emergency room', 'huguenot', 'medicis', 'protestant', 'katana sword', 'los angeles storm drain', 'evidence', 'fingerprints', 'murder weapon', 'blue bra and panties', 'girl in bra and panties', 'killer car', 'female alien', 'peacock', 'tigress', 'black u.s. president', 'tour', 'academy award', 'dove', 'nursing', 'pigeon', 'psychological torture', 'monster sex', 'blood vomiting', 'hit squad', 'dj', 'ecstasy the drug', 'rave', 'raver', 'warehouse', 'based on supposedly true story', 'paranormal investigating', 'easter island', 'statue', 'class differences', 'nursing home', 'summer love', 'blackjack', 'card counting', 'massachusetts institute of technology', 'mumbai riots', 'marine biologist', 'plumber', 'head on a stake', 'nuclear reactor', 'partisan', 'holy grail', 'mary magdalene', 'opus dei', 'charles darwin', 'cutlass', 'queen victoria', 'road', 'stupid victim', 'autobot', 'decepticon', 'job interview', 'angel dust the drug', 'disco', 'quaalude', 'reference to princess grace of monaco', 'covert analyst', 'stock market', 'abraham', 'sodom', 'delaware', 'erotica', 'drunk', 'religion versus science', 'titanic', 'wet', 'detective inspector', 'internal affairs', 'pet name in title', 'economic hardship', 'quadriplegic', 'wheelchair', 'barbed wire', 'barbed wire fence', 'navigator', 'army veteran', 'dignity', 'euthanasia', 'howie scream', 'bastille day', 'frog leg', 'the muppets', 'foot fetish', 'houseguest', 'air traffic control', 'female bodybuilder', 'grace', 'western u.s.', 'terror', 'ancient india', 'book of mormon', 'ray gun', 'dark past', 'spaghetti western', 'thomas jefferson', 'burned to death', 'witch hunter', 'female gunfighter', 'siege', 'child labor', 'hispanic', 'migration', 'hat', 'yellow', 'revisionist history', 'robe', 'unnecessary guilt', 'ashes', 'settler', 'wagon train', 'experimental surgery', 'regression', 'mascot', 'stupidity', 'hijack', 'commando unit', 'extra chromosome', 'first person shooter', 'hotel room', 'editor', 'new york times', 'puzzle', 'flying dagger', 'government', 'parapsychologist', 'in medias res', 'reference to planet of the apes', 'sex talk', 'darkness', 'kraken', 'corporate espionage', 'core', 'natural disaster', 'pacemaker', 'hidden truth', 'misogyny', 'new friend', 'abandoned asylum', 'abandoned hospital', 'asbestos', 'lobotomy', 'homeless', 'emotional balance', 'indonesia', 'inner peace', 'losing', 'pool', 'pool player', 'electrocuted', 'university', 'slip and fall', 'squirting orgasm', 'locked in a room', 'looking at self in mirror', 'night shift', 'parallel universe', 'unlikely friendship', 'illusionist', 'action figure', 'health care', 'tennis', 'scroll', 'tibetan monk', 'boy genius', 'futuristic city', 'woman pretending to be a man', 'police chief', 'tall woman', 'female surfer', 'swimsuit', 'krampus', 'chinatown san francisco', 'vault', 'indian american', 'spelling', 'spelling bee', 'life support', 'voice recording', 'black soldier', 'military uniform', 'abandoned by mother', 'sex trade', 'teenage prostitution', 'unpaid bill', 'full frontal nudity', 'group shower', 'police academy', 'police cadet', 'caravan', 'gangsta', 'gun fight', 'kitten', 'rock band', 'lecture', 'oxford university', 'symbol', 'drug addict', 'human android relationship', 'megacorporation', 'aging', 'billings montana', 'lincoln nebraska', 'adult diaper', 'diaper', 'sawed off shotgun', 'failure', 'partial female nudity', 'science experiment', 'haunted', 'haunting', 'paranormal investigator', 'flying saucer', 'aviator', 'based on book', 'the little prince', 'stanford university', 'diabetic', 'pharmaceutical lab', 'pitching', 'attempted suicide', 'collapsing scaffold', 'overflowing bathtub', 'tooth knocked out', 'glitter', 'reference to katharine hepburn', 'reference to spencer tracy', 'assumed identity', 'bolivia', 'butch cassidy', 'desk', 'oil refinery', 'belleville', 'tour de france', 'very little dialogue', 'baby rattle', 'anthropologist', 'forensics', 'animal experimentation', 'chimera', 'ambulance', 'paramedic', 'master apprentice relationship', 'female prisoner', 'self delusion', 'year 1954', 'psychiatric ward', 'mexican american war', 'u.s. frontier', 'british actor playing american character', 'origin of hero', 'bouncer', 'private dancer', 'giant squid', "liar's dice", 'la llorona', 'weeping', 'mission control', 'heaven and hell', 'vertigo comics', 'planet of the apes', 'tragedy', 'tragic love', 'black and white segues into color', 'color', 'infanticide', 'office politics', 'photo booth', 'snobbery', 'music therapy', 'united arab emirates', 'honeymoon', 'insurance company', 'baseball card', 'barn', 'piglet', 'giant food', 'sardine', 'marlboro', "lead actor's first film", 'public access', 'ashram', 'wiley college', 'reference to william shakespeare', 'white trash', 'blow job', 'agriculture', 'emotional abuse', 'rules', 'middle class', 'dna', 'cleveland browns', 'linebacker', 'nfl', 'team owner', 'centennial', 'next door neighbor', 'death of soldier', 'war trauma', 'crematorium', 'naked dead woman', 'poison gas', 'mammoth', 'valley', 'psychosis', 'foundry', 'foundry worker', 'caterer', 'bruise', 'teenage superhero', 'flashback within a flashback', 'split screen', 'self defense', 'murdered before giving protagonist information', 'part computer animation', 'prequel to cult film', 'head butt', 'phd', 'british royal family', 'queen elizabeth i', 'subconscious', 'nuclear war', 'stabbing', 'street life', 'case', 'dry humor', 'canada', 'human becoming an animal', 'part of trilogy', 'walrus', 'bridge construction', 'professor moriarty', 'scotland yard inspector', 'fish out of water', 'older man younger woman relationship', 'leather dress', 'law partner', 'seaman', 'drought', 'playground', 'skateboarding', 'male in shower', 'conservatory', 'music school', 'ghost story', 'manson family', 'real life event', 'reference to sharon tate', 'jewish american princess', 'based on a book', 'based on a novel', 'made for tv movie', 'support', 'sketch comedy', 'gotham', 'fbi director', 'guinea pig', 'household appliance', 'talking mouse', 'headmaster', 'oxford', 'train crash', 'great barrier reef', 'short term memory loss', 'american in paris', 'embarrassment', 'revelation', 'lost', 'masseuse', 'threesome', 'bolshevik', 'russian revolution', 'prosecutor', 'carpenter', 'christmas song', 'leave of absence', 'sunflower', 'lieutenant colonel', 'alibi', 'newspaper clipping', 'profiler', 'campy', 'castle thunder', 'gulf war veteran', 'shot in the head', 'distrust', 'dodgeball', 'portrait', 'soccer game', 'bean', 'duology', "pandora's box", 'battalion', 'skateboarder', 'surf', 'goblin', 'freak accident', 'hit in the crotch', 'nosy neighbor', 'on the job injury', 'childhood home', 'human animal relationship', 'waiting', 'pot', 'smoke', 'chun li', 'crime fighter', 'political drama', 'washington post the newspaper', 'time travel romance', 'cola', 'ice cream cone', 'playboy mansion', 'african americans', 'boxing gym', 'rumble in the jungle', 'hunting', 'minister', 'cenobites', 'roof', 'tank', '1st century', 'based on web series', 'claustrophobia', 'phobia', 'football movie', 'high school sweethearts', 'sperm', 'sperm donor', 'dead man', 'killing a lion', 'killing a tiger', 'softball', 'alienation', 'conscience', 'insomnia', 'urban gothic', 'jihad', 'mindanao', 'philippines', 'squatter', 'injection', 'scalpel', 'stabbed in the mouth', 'bildungsroman', 'flying broom', 'invisibility cloak', 'wizardry', 'courier', 'delivery', 'envelope', 'jewish nazi', 'nazism', 'store', 'trying to conceive', 'yellow panties', 'mix up', 'stutter', 'opposites attract', 'block party', 'marching band', 'children adventure', 'sea goddess', 'tsunami', 'birding', 'hipster', 'narcotics', 'art of war', 'lift skirt', 'short shorts', 'broken nose', 'marriage proposal', 'sistine chapel', 'alderman', 'presidential candidate', 'language', 'male masturbation', 'central asia', 'russian empire', 'defense', 'mothership', 'arms tied overhead', 'brooklyn bridge', 'company', 'corporate takeover', 'executive', 'new boss', "diver's watch", 'coroner', 'gay son', 'mistaken identity', 'forensic', 'boxing arena', 'boxing manager', 'directed by co star', 'wrongful conviction', 'museum of natural history', 'night watchman', 'watchman', 'housekeeper', 'manor', 'service', 'dodge viper', 'the star spangled banner', 'bound and gagged', 'hearing noises', 'uninterrupted shot', 'bengali', 'reference to gogol', 'interracial kiss', 'interracial sex', 'automobile racing', 'nascar', 'street racing', 'television station', 'tv show', 'adult as child', 'janitor', 'ice cream', 'second city', 'critique of capitalism', 'investment bank', 'market economy', 'plutocracy', 'naivety', 'gods', 'aurora borealis', 'male in underwear', 'evil politician', 'director', 'psychosomatic blindness', 'self deprecating', 'studio', 'psychotronic film', 'public hearing', 'persian', 'xerxes', 'dare', 'exclamation point in title', 'insurance salesman', 'reference to britney spears', 'world tour', 'father in law', 'ex convict', 'older brother is bad influence', 'washington heights manhattan new york city', 'girl scouts', 'quitting a job', 'death of friend', 'severed arm', 'single camera', 'subtle comedy', 'theatrical producer', 'tutu', 'ingratitude', 'hairdresser', 'reference to liberace', 'armored car', 'falling out among thieves', 'planning', 'computer hacker', 'rise and fall', 'estate', 'monte carlo', 'studio ghibli', 'cameraman', 'epic battle', 'south ossetia', 'actual animal killed', 'evil sorcerer', 'warlord', 'junkyard', 'guitarist', 'obsessed fan', 'home', 'police surveillance', 'surveillance van', 'tailing a suspect', 'venice italy', 'patricide', 'developer', 'medical ethics', 'pills', 'serbian', 'black history', 'nation of islam', 'dole', 'male stripper', 'northern england', 'steel worker', 'art dealer', 'fake painting', 'macguffin', 'moustache', 'stolen painting', 'health insurance', 'long delayed release', 'nail in the head', 'bayou', 'swamp', 'pacific northwest', 'small town sheriff', 'snowmobile', 'incest', 'son murders mother', 'brawl', 'computer cracker', 'intelligence agency', 'terrorist cell', 'south orange new jersey', 'woman in underwear', 'bound with duct tape', 'flash forward', 'riddle', 'abusive relationship', 'tenderloin san francisco', 'law clerk', 'world trade center manhattan new york city', 'woodstock', 'black smoke', 'japanese military', 'japanese navy', 'sunday', 'hotel manager', 'cleveland indians', 'league', 'owner', 'clowning', 'polo', 'possum', 'abusive parent', 'child molester', 'child protection', 'sex addict', 'erectile dysfunction', 'mugger', 'mugging', 'subway robbery', '12 year time span', 'separated parents', 'art expert', 'nazi stolen art', 'monarchy', 'princess diana', 'swastika tattoo', 'high school girl', 'doc holliday', 'robbers', 'three of spades tattoo', 'weapon', 'curtain call', 'dream girl', 'satanic cult', 'ardennes forest', 'battle of the bulge', 'behind enemy lines', 'world war two in belgium', 'based on young adult novel', 'human alien', 'jazz', 'jazz music', "texas hold 'em", 'thong', 'wrestler', 'bermuda triangle', 'shipwrecked', 'travel writer', '70mm film', 'biblical epic', 'king herod', 'dead soldier', 'impersonating a police officer', 'briefcase full of money', "character repeating someone else's dialogue", 'reference to cinderella', 'genitalia', 'black and white photograph', 'drawing with blood', 'falling through ice', 'playing piano', 'red haired woman', 'boyfriend', 'nintendo ds', 'game designer', 'pod', 'foot chase', '8 year old', 'convention', 'magic wand', 'male fairy godmother', 'inner beauty', 'climbing a tree', 'courage', 'radiation sickness', 'radioactive contamination', 'test launch', 'surveillance footage', 'timebomb', 'natural history museum', 'news reporter', 'shooting', 'cbs news', 'tobacco', 'transformation', 'young boy', 'multi protagonist', 'hair stylist', 'landlord', 'palestinian', 'aerialist', 'cirque du soleil', 'learning disability', 'tutor', 'flashing breasts', 'hitchhiker', 'interpol', 'bangkok thailand', 'forks washington', 'protecting a woman', 'title same as book', 'breath on mirror', 'engagement ring', 'guard', 'swordsman', 'lawsuit', 'loud sex', 'admiral', 'thunderstorm', 'barbecue', 'non professional cast', 'cross country trip', 'boat racing', 'hydroplane', 'scolding', 'oil field', 'oil tycoon', 'wildcatter', 'catholicism', 'autocracy', 'based on real events', 'dictatorship', 'exploding airplane', 'fighter pilot', 'hands tied', 'shaft', 'caver', 'dark', 'radical', 'triple crown', 'parachute', 'detainee', 'guantanamo bay', 'infant', 'sports violence', 'father', 'orangutan', 'barrio', 'newcastle united', 'greenhouse', 'old house', 'long con', 'australian aborigine', 'australian outback', 'virginia', 'thrown through a window', 'sadist', 'reading', 'futuristic', 'based on manga', 'gifted child', 'post thermonuclear war', 'music copyist', 'music student', 'musical composition', 'symphony', 'cryptography', 'enigma code', 'instructor', 'animal companion', 'shakespeare adaptation', 'star crossed lovers', '1890s', 'hired gun', 'johnson county war', 'gardening', 'commander', 'african american policeman', 'policewoman', 'beauty salon', 'black college', 'government conspiracy', 'oppression', 'four brothers', 'crown jewels', 'sex change', 'diane arbus', 'steel mill', 'fall to death', 'chameleon', 'western town', 'whipped cream', 'miniaturization', 'sudden change in size', 'monologue', 'television network', 'television news', '12 year old', 'blade', 'anchorage alaska', 'investment', 'investment banker', 'diplomat', 'escapade', 'gallipoli', 'world war one', 'blonde', 'law', 'law school', 'kumite', 'elvis impersonator', 'rascal', 'talk radio', 'vomiting blood', 'rain', 'rain forest', 'gay parent', 'jiminy cricket', 'pinocchio', 'ice pick', 'pitchfork', 'shack', 'lucha libre', 'wrestling', 'damselfish', 'ocean habitat', 'ocean life', 'razorfish', 'spider crab', 'devil worship', 'incubus', 'raped by a demon', 'satanism', 'baron', 'nickname', 'transylvania', 'igloo', 'estranged daughter', 'iranian', 'phone booth', 'dangerous job', 'hurricane', 'school bus', 'shark feature', 'foster home', 'murder investigation', 'elvis', 'iron curtain', 'reality spoof', 'algerian war', 'graveyard', 'murder trial', 'gang that lives together', 'outlaw gang', 'blacksmith', 'gunslinger', 'mumblecore', 'frozen', 'health food', 'penis joke', 'phallic image', 'sexy agent', 'volcano hideout', 'alien race', 'dishwasher', 'flashing', 'integrity', 'responsibility', 'obedience', 'family restaurant', 'remarriage', 'female killer', 'bar fight', 'drug abuse', 'interracial rape', 'booty shake', 'duct tape over mouth', 'female to male foot in crotch', 'alien infection', 'liquid metal', 'multiple cameos', 'sexy woman', '1770s', 'british india', 'female criminal', 'hooker', 'concrete wall', 'open ended', 'electric chair', 'blob', 'ferris wheel', 'drumline', 'multiverse', 'apatosaurus', 'asteroid', 'screwball comedy', 'artificial heart', 'horse penis', 'mayhem', 'rowing', 'artifact', 'incestuous relationship', 'card trick', 'rain machine', 'vigilantism', 'drawn and quartered', 'torture chamber', 'torture device', 'prescription drugs', '911 operator', 'calling 911', 'head held underwater', 'white villain', 'holocaust', 'warsaw ghetto', 'dark humor', 'female antagonist', 'christ', 'cnn reporter', 'matrix', 'sugarcane', 'mental patient', 'scheme', 'battle of the bands', 'school teacher', 'substitute teacher', 'dealer', 'claim in title', 'jazz singer', 'time manipulation', 'middle earth', 'thong bikini', 'tv dance show', 'muggle', 'first love', 'milf', 'card dealer', 'criminal mastermind', 'recruiting', 'ulcer', 'arena', 'boxing movie', 'robot battle', 'brazil', 'soccer movie', 'world cup', 'soccer star', 'mental breakdown', 'post september 11 2001', 'dome', 'the simpsons', 'rap music', 'ivy league', 'failing student', 'payphone', 'telephone booth', 'business tycoon', 'mayoral candidate', 'penthouse', 'reference to star trek', 'reconciliation', 'bush administration', 'hijab', 'reference to mahmoud ahmedinejad', 'jewell', 'mongol', 'nostradamus', 'stepnicka', 'vallone', 'great dane', 'animal control', 'lost animal', 'mongrel', 'pet', 'terrier', 'bayonet', 'college kid', 'hillbilly', 'flying superhero', 'phantom zone', 'challenge', 'drift racing', 'drifting', 'foreign exchange student', 'incestuous desire', 'incestuous kiss', 'bag over head', 'buddhist temple', 'loss of friend', 'male bonding', 'black servant', 'chocolate pie', 'jackson mississippi', 'writing a novel', 'backstage', 'racial slur', 'not taking no for an answer', 'obsessive woman', 'animal as human', 'sheepdog', 'mine', 'plutonium', 'ash', 'pokemon', 'amateur theater', 'travel agent', 'south pole', 'country house', 'producer', 'shooting party', 'boxing match', 'montage', 'philadelphia', 'enema', 'rocky mountains', 'dog hit by a car', 'female in shower', 'unhappily married woman', 'argentina', 'meth', 'meth lab', 'speed freak', 'mussolini', 'toy company', 'ark', 'change', 'depiction of god', 'saigon', 'bambi', 'college basketball', 'national championship', 'ncaa', 'film editor', 'movie screening', 'columbia university', 'bong', 'american actress playing british character', 'book publishing', 'calorie counting', 'rabbit costume', 'construction', 'female agent', 'waterfront', 'lying', 'reference to twitter', 'reference to youtube', 'toronto', 'blizzard', 'buddhism', 'evolution', 'reincarnation', 'stuck in elevator', 'malaysia', 'male model', 'credit card fraud', 'miami', 'returning character with different actor', 'slasher flick', 'based on opera', 'walking wounded', 'documentary filmmaker', 'documentary filmmaking', 'generation z', 'flint michigan', 'merger', 'uprising', 'reference to snow white', 'sleeping bag', 'stepmother stepdaughter relationship', 'totem pole', 'probation', 'hit and run', 'police investigation', 'volleyball', 'car racing', 'racing', 'duplex', 'tenant', 'dishonor', 'masquerade party', 'flea', 'space dog', 'baby boy', 'affection', 'black horse', 'boy horse relationship', 'rescued by a horse', 'dognapping', 'misfiring gun', 'shih tzu', 'stake', 'what happened to epilogue', 'football coach', 'big brother', 'community service', 'live action role playing', 'peter pan syndrome', 'amazing grace the hymn', 'skidding truck', 'neck breaking', 'wraparound story', 'gasoline', 'refinery', 'traffic jam', 'cowboy hat', 'grandparent grandchild relationship', 'reference to facebook', 'reference to the internet', 'full frontal male nudity', 'multiple characters voiced by same person', 'sex in hotel room', 'animal attack', 'french riviera', 'speaker of the house', 'comic relief', 'state flag', 'real estate agent', 'lancer', 'free running', 'james bond', 'tintin', 'ends with quotation', "women's prison", 'year 1964', 'all girl band', 'cross dressing', 'arc de triomphe paris', 'based on autobiography', 'elle magazine', 'goddess', 'sinbad', 'last day of school', 'comeback', 'racer', 'blood of christ', 'firefly', 'mysterious event', 'mount everest', 'mountain climbing', 'teaching oneself to read', 'war crime', 'giant monster', 'kaiju', 'family drama', 'political campaign', 'social decay', 'mormon missionary', 'illustrator', 'chewing gum', 'translation problems', 'chocolate factory', 'businesswoman', 'bruce lee', 'jeet kune do', 'letter of resignation', 'advertising executive', 'commercial', 'lion', 'railway', 'barista', 'coffee', 'coffee shop', 'boot camp', 'fictional talk show', 'phony suicide note', 'dragonfly', 'soaking wet', 'supervillian origin', 'romantic rivalry', 'rule', 'elvis presley', 'cosmonaut', 'footprint', 'eastern philosophy', 'healing', 'reference to george washington', 'ritual sacrifice', 'drag queen', 'physical appearance', 'leonardo da vinci', 'nuclear waste', 'deformed', 'roller derby', 'bank fraud', 'prey', 'best friends', 'reference to matthias steiner', 'railway station', 'magical potion', 'time travel comedy', 'rape victim', 'sexual humiliation', 'classroom', 'fund raising', 'roman', 'theater', 'cheerleader uniform', 'gang banger', "shakespeare's hamlet", 'celibacy', 'synagogue', 'yellow ribbon', 'joy', 'running away', 'sadness', 'campus', 'arcade game', 'driving in reverse', 'video gamer', 'hit by a pickup truck', 'resuscitation', 'greek restaurant', 'travel agency', 'anniversary', 'cellist', 'hardware store', 'camcorder', 'lost passport', 'black ops', 'repeated scene', 'black pantyhose', 'female stockinged legs', 'flubber', 'flying rubber', 'drug overdose', 'gang war', 'adult child friendship', 'acrophobia', 'editing', 'role playing', 'buick', 'last will and testament', 'savant', 'lifted by the throat', 'star trek', 'vulcan', 'new school', 'dragon rider', 'teenage hero', 'boy in a bathtub', 'laptop computer', 'pulled underwater', 'walking in the dark', 'operative', 'mushroom', 'mute', 'roman soldier', 'screaming', 'wood carving', 'college student', 'disfigured face', 'physical therapist', 'retrofuturism', 'living dead', 'blue ribbon', 'jacket', 'jock', 'yogurt', 'coal mine', 'israeli', 'hearing characters thoughts', 'feminism', 'kicked in the testicles', 'paintball', 'paintball gun', 'pepper spray', 'based on multiple works', 'mysterious stranger', 'mail order bride', '15 year old', 'corndog', 'funeral in the rain', 'weatherman', 'high definition television', 'reverse chronology', 'boxing training', 'heavy rain', 'anilingus', 'double cross', 'two timing', 'chihuahua', 'potted plant', 'tijuana mexico', 'desi', 'heritage', 'sail', 'smoker', 'plastic surgeon', 'woman wearing a string bikini', 'exhibition', 'gilbert and sullivan', "gilbert and sullivan's princess ida", 'southern accent', 'title based on the bible', 'brunette', 'film producer', 'liar', 'prank', 'secret government agency', 'troll', 'coin toss', 'tracking device', 'bear attack', 'lingerie model', 'm.i.b.', 'pug', 'geisha', 'chess', 'american soldier', 'naval uniform', 'patrol boat', 'uniform', 'female on male rape', 'lent', 'ice storm', 'first crush', 'social outcast', 'taunting', 'ramanujan', 'nazis', 'bullying', 'cyberbullying', 'septicemia', 'radio show', 'alone', 'aron ralston', 'canyon', 'binge drinking', 'psychic power', 'psychometry', 'ski mask', 'stolen money', 'face', 'face transplant', 'brother in law', 'forgiveness', 'job offer', 'slow motion action scene', 'slow motion sequence', 'crime lord', 'dead body', 'dead body in a freezer', 'truck explosion', 'reference to anastasia romanov', 'romanov', 'promise', 'introduction', 'tron', 'summer job', 'gay cop', 'getaway driver', 'witness protection', 'algeria', 'algerian', 'mario', 'reward', "st trinian's", 'abstract', 'double life', 'rogue agent', 'siberian husky', 'sled dog', 'macabre', 'uncle fester', 'flyer', 'champagne', 'nuremberg', 'alien planet', 'promised land', '20 years later', 'ford mustang', 'james bond 007', 'yanked off bikini top', 'd day', 'normandy', 'rollerskating rink', 'spelman college', 'penknife', 'united kingdom', 'press', 'clock', 'mad hatter', 'fishing', 'fishing trip', 'golden age', 'midnight', 'student film', 'wonder years', 'museum of natural history manhattan new york city', 'tablet', 'champion', 'golf tournament', 'nostalgia', 'maid of honor', 'female therapist', 'therapy session', 'casper', 'friendly ghost', 'kung fu classic', 'industrialist', 'murder disguised as suicide', 'reference to howard university', 'norad', 'road rage', 'hobo', 'rampage', 'english teacher', 'coronation', 'swashbuckler', 'premier', 'alien contact', 'death of girlfriend', 'ouija', 'head cut off', 'necklace', 'safecracker', 'wine dealer', 'godzilla', 'yakuza', 'new orleans louisiana', 'execution by hanging', 'los angeles police department', 'miscarriage of justice', 'convoy', 'crow', 'cynicism', 'false promise of the american dream', 'potion', 'sanctuary', 'beauty', 'bosnia', 'war criminal', 'buenos aires argentina', 'jaguar', 'mayan', 'solar eclipse', 'killing spree', 'one night', 'authority figure as villian', 'gun duel', 'saved from hanging', 'dart board', 'ladder factory', 'cryogenics', "boys' school", "girls' soccer", 'year 2054', 'groupie', 'rock n roll', 'dialectics', 'eiffel tower at night', 'implied insemination', 'pfizer', 'viagra', 'zoloft', 'shot in the crotch', 'mourning', 'ball', 'steam', 'female scientist', 'caribbean', 'anchor', 'chorus line', 'gala premiere', 'environmental', 'oil spill', 'oktoberfest', 'bouquet', 'house fire', 'praying', 'year 1959', 'new hampshire', 'linguistics professor', 'reference to angels in america the play', 'ogre', 'nosebleed', 'inspired by documentary', 'presidential campaign', 'jamming communications', 'space station', 'taunt', 'fourth of july', 'breast', 'rap star', 'reference to tupac shakur', 'blocked road', 'zookeeper', 'batman', 'sequel to a reboot', 'sister love', 'snowman', 'horror movie prequel', 'paranormal activity', 'self sacrifice', 'hibernation', 'dead body wrapped in carpet', 'school janitor', 'criminal organization', 'kgb agent', 'cedar rapids iowa', 'musical quartet', 'filmmaker', 'anticipate', 'dead in absentia', 'foetus', 'unreliable narration', 'phone sex', 'queens new york city', 'sex addiction', 'animal abuse(non graphic)', 'cat killer', 'dead cat', 'death of cat', 'jazz age', 'vaudeville', 'delivery man', 'military police', 'rio de janeiro brazil', 'miami florida', 'murder of family', 'girl next door', 'growing vegetables', 'high standards', 'sent away', 'tennis coach', 'rise to power', 'addict', 'tv reporter', 'warp speed', 'garage', 'narcotics anonymous', 'recovery', 'dick tracy', 'forger', 'passport', 'venture capital', 'slave labor', 'house construction', 'death star', 'empire', 'galactic war', 'toddler', 'teen horror', "noah's ark", 'wrath of god', 'market', 'white picket fence', 'gallantry', 'honesty', 'rape and revenge', 'performing arts school', 'based on article', 'massage therapist', 'regaining sight', 'champagne bottle', 'coca cola', 'jewelry box', 'red bull', 'shaolin', 'political prisoner', 'bankruptcy', 'crystal', 'microwave oven', 'alternate history', 'future time travel', 'adidas', 'waste container', '21st century', 'ear wax cleaning', 'fake newsreel', 'paddington station london', 'taxidermist', 'luck', 'masquerade', 'kryptonite', 'smallville', 'macau', 'selfie', 'prodigy', 'rhapsody', 'hall of mirrors', 'mont saint michel france', 'palace of versailles', 'turkey the country', 'yellowstone national park', 'baseball scout', 'hit on the head with a ball', 'sexy', 'wild horse', 'bombardier', 'loss of wife', 'female female kiss', 'outdoor sex', 'banana boat', 'chips', 'fictional government agency', 'vatican', 'dauphin', 'wartime rape', 'creation', 'paddock', 'supercross', 'winning', 'dentures', 'false teeth', 'auctioneer', 'sorcerer', 'abusive marriage', 'prison wagon', 'roast pig', 'suicide by hanging', 'christian propaganda', 'zealot', 'hostility', 'ultimate fighting championship', 'underdog', 'marijuana joint', 'porn magazine', 'sex obsession', 'breasts growing', 'coors light', 'spinning in grave', 'berlin wall', 'specialist', 'overachiever', 'the flintstones', 'title appears in song', 'work', 'controlled environment', 'fictional reality show', 'innocence lost', 'simulated reality', 'ice hockey', 'turkish bath', 'rags to riches', 'secret letter', 'squire', 'injury', 'hybrid', 'scream', 'disaster movie', 'san andreas fault', 'aging rocker', 'cobbler', 'art exhibition', 'augmented reality', 'speculative fiction', 'hide and seek', 'domineering mother', 'stamp', 'eurocopter ec120 colibri', 'fortune cookie', 'lottery ticket', 'sex lesson from a prostitute', 'spreadeagle', 'topless', 'boxing knockout', 'implied sex', 'keys', 'half brothers', 'white raised as indian', 'breath holding contest', 'plantation', 'pipeline', 'sexual ambiguity', 'sheik', 'armageddon', 'end times', 'reference to satan', 'film in film', 'reference to steven spielberg', 'self referential', 'shopping mall', 'sports team', 'title based on song', 'sydney australia', 'computer virus', 'fight the system', 'locomotive', 'steam locomotive', 'planet invasion', 'year 2046', 'drowning', 'hanging from a flagpole', 'hanging from a rope', 'whiskey', 'surprise', 'tesseract', 'war bond', 'business trip', 'glory hole', 'loser', 'chop socky', 'bahamas', 'hook', 'beverly hills california', 'soap opera', 'erotic thriller', 'marital infidelity', "wife's lover murdered by husband", 'buffalo new york', 'embalming', 'mortuary', 'fiancee', 'animate doll', 'rebirth', 'cafeteria', 'bobby darin', 'sandra dee', 'contortionist', 'gag humor', 'human cannonball', 'perfume', 'radio dj', 'radio station', 'vinyl', 'coming out', 'gay brother', 'toilet sex', 'harem', 'reference to vanilla ice', 'statutory rape', 'teacher student relationship', 'teacher student sex', 'bad taste humour', 'crude humor', 'self obsession', 'cabinet meeting', 'catholic church', 'borneo', 'orchid', 'colonoscopy', 'washington square manhattan new york city', 'based on ed gein', 'hyperactive', 'the doors', 'financial crisis', 'cuban flag', 'fidel castro', 'havana cuba', 'alliterative title', 'breaking down a door', 'haunted hotel', 'identical twins', 'twin sisters', 'selma alabama', 'u.s. politics', 'park ranger', 'drunkenness', 'religious mother', 'strict mother', 'war hammer', 'cb radio', 'arrow', 'imperial seal', 'gratitude', 'upholsterer', 'serial rapist', 'safe', 'parallel worlds', 'secret door', 'bloody mary the drink', 'rules of profession', 'padded cell', 'monster slayer', 'domino', 'fiery redhead', 'nassau', 'underwater battle', 'helicopter', 'stubbornness', 'reference to frankenstein', 'reference to jimi hendrix', 'studio logo segues into film', 'white dinner jacket', 'wien', 'fashion show', 'falling off a roof', 'nypd', 'compassion', 'drive in', 'woman shot in the forehead', 'hungary', 'diarrhea', 'middle aged man', 'sexy nurse', 'balloon', 'skeleton', 'secret base', 'syracuse university', 'guest', 'car breakdown', 'jeep', 'halloween party', 'romcom', 'national spelling bee', 'school principal', 'alien creature', 'body snatching', 'parasite', 'han river', 'seoul', 'happiness', 'tuskegee airmen', 'patrol', 'decoder', 'sexism', 'underage sex', 'promiscuous daughter', 'sex in a bed', 'killed in action', 'child with gun', 'invulnerability', 'population', 'state in title', 'southern gothic', 'newsletter', 'elderly man', 'medical', 'transhumanism', 'sausage', 'sleep', 'portugal', 'switzerland', 'deformed baby', 'premature aging', 'long lost brother', 'marriage bureau', 'professional rivalry', "president's daughter", 'legs', 'cult director', 'drive in classic', 'falling in love with a prostitute', 'rich man poor woman', 'tuition', 'knife held to throat', 'hoop', 'chat room', 'identity theft', 'housing project', 'trapped in an elevator', 'oil drilling', 'kleptomaniac', 'self destructiveness', 'student council president', 'student government', 'water boy', 'case file', 'psycho thriller', 'self expression', 'white rapper', 'aviation', 'spruce goose', 'test flight', 'rocker', 'u2', 'halleys comet', 'life force', 'space vampire', 'suicide forest', '1988 winter olympics', 'ski jumper', 'winter olympics', 'bombing mission', 'munitions', 'royal flying corps', 'surrogate mother', 'homecoming queen', 'insecurity', 'alien probe', 'humpback whales', 'probe', 'female assassin', 'office cubicle', 'gnosticism', 'rare book', 'villain arrested', 'divorce lawyer', 'storyteller', 'time paradox', 'retired boxer', 'airfield', 'car stunt', 'private jet', 'menstrual blood', 'nursing school', 'performing arts', 'sorceress', 'dream world', 'lake tahoe nevada', 'melancholy', 'cold fusion', 'fusion', 'the saint', 'hmo', 'las vegas', 'german expressionism', 'basic training', "reference to shakespeare's hamlet", 'side saddle', 'adult returns home', 'girl with glasses', 'when animals attack', 'boulder', 'villa', 'belief in the afterlife', 'stigmata', 'wound', '1959 cadillac', 'diamonds', 'female villain', 'force field', 'child abduction', 'genius recluse', 'text messaging', 'beverly hills hotel', 'troop', 'easter', 'easter bunny', 'piaggio vespa gtv 250', 'gun running', 'terrorist attack', 'death of main character', 'eiffel tower paris', 'two suitors', 'hit with a door', 'pit boss', 'orange county california', 'ex con', 'young woman', 'friday the thirteenth', 'murder spree', 'class president', 'statue of liberty', 'television van', 'jordan', 'wire fu', 'georgia', 'trailer', 'pakistani', 'thatcher', 'tooth', 'bodybuilder', 'punch into the camera', 'reference to julia roberts', 'song and dance', 'arsonist', 'blood splatter on face', 'cave bear', 'neanderthal', 'return', 'egyptologist', 'mummy', 'exam', 'northeast region of brazil', 'portuguese language', 'eyeglasses', 'woman slaps a man', 'woman slaps man', 'hamburger', 'woman in a bikini', 'toad', 'dirty cop', 'limping', 'individualism', 'arkansas', 'jesus crust pizza parlor', 'burned alive', 'olympus', 'dog fight', 'interracial romance', 'football game', 'coastline', 'capital management', 'financial disaster', 'investment fraud', 'management', 'somali', 'luminol', 'omen', '1st century b.c.', 'lightsaber', 'local blockbuster', 'existentialism', 'catacomb', "philosopher's stone", 'blindfold', 'automaton', 'mechanical', 'clark kent', 'year 1978', 'congo', 'laser', 'lost city', 'art director', 'riding a horse', 'african american lesbian', 'lesbian couple', 'giving birth', 'ultrasonography', 'crotch shot', 'disney spoof', 'one woman army', 'catwoman', 'riddler', 'brighton', 'saloon', 'saloon singer', 'musical filmmaking', 'tango', 'alternate world', 'fairy tale parody', 'island name in title', 'haute cuisine', 'indian cuisine', 'michelin guide', 'evil scientist', 'groundhog day', 'mothman', 'point pleasant west virginia', 'washington d.c. lobbyist', 'entertainer', 'carried in a chair', 'rite of passage', 'mysterious man', 'rhythm and blues', 'dominance submissiveness', 'human pet', 'power exchange', 'nipples visible through clothing', 'pronounced dead', 'sword duel', 'austin texas', 'pap smear', 'hawk', 'lady', 'planet enclosed within shield', 'sci fi spoof', 'solitude', 'football video game', "talking to one's self in a mirror", 'friar', 'inquisition', 'peru', 'rope bridge', 'pain', 'expecting twins', 'chaucer', 'jousting', 'looking at the camera', 'dreaming', 'contraband', 'bird in a cage', 'reunited family', 'poetess', 'ecuador', 'theatre director', 'goat', 'naked woman', 'joker', 'tattoo on breast', 'the joker', 'prawn', 'refugee camp', 'cabinet', 'temptress', 'meteorite', 'cold calculation', 'caregiver', 'department store', 'imprisonment', 'lawlessness', 'hit by a car', 'living statue', 'melting woman', 'food poisoning', 'health inspector', 'rationing', 'vampire society', 'year 2019', 'general motors', 'shangri la', 'treasure hunt', 'dorset', "women's land army", 'nobel prize', 'kidnapper', 'mountaineering', 'scottish highlands', 'nude black woman', 'asylum', 'full moon', 'duplicate earth', 'synchronicity', 'grandson', 'stonewall riots', 'slave revolt', 'call center', 'seminary', 'iron lung', 'sex surrogate', 'bully comeuppance', 'destruction', 'nuclear missile', 'tears', 'opening night', 'swedish', 'gold mine', 'surveyor', 'beavis and butt head', 'score', 'television set', 'autograph', 'journey to the west', 'monkey king', 'talking to inanimate object', 'orga', 'male friendship', 'muscular', 'starvation', 'white castle', 'sertao', 'chinese american', 'lesbian love', 'overweight', 'anesthetic', 'anesthetic awareness', 'heart surgery', 'boxer rebellion', 'peking china', 'quick stop', 'pun', 'gardener', 'oscar wilde', 'pigalle', 'archery', 'defiance', 'toilet training', 'older woman younger woman relationship', 'sex with student', 'avoidant personality disorder', 'emotionally vulnerable', 'video store clerk', 'dunes', 'socialist', 'critic', 'theater critic', 'firm', 'hooliganism', 'west ham supporter', 'yukon', 'minimal dialogue', 'decapitation', 'scavenger hunt', 'sleepover', 'punched in the crotch', 'sparring', 'moral dilemma', 'armored truck', 'husband murders wife', 'overhead shot', 'desert storm', 'fragmentation grenade', 'woman kills attacker', 'man wearing a jock strap', 'man wearing a thong', 'rear male nudity', 'stripped prison uniform', 'lunar eclipse', 'crusades', 'modern minstrel', '10 year old', 'grumpy old man', 'school presentation', 'stroke', 'stroke recovery', 'film school', 'ex soldier', 'chief of staff', 'imdb', 'mediaatlarge.com', 'mediaatlargellc', 'dress', 'fairy godmother', 'pumpkin', 'press conference', 'chatting', 'missing person poster', 'online dating', 'sexual predator', 'mini dress', 'imaginary son', 'vampire slayer', 'clan', 'year 1743', 'fart joke', 'male nipple', 'human versus alien', 'death of loved one', 'nero', 'diva', 'open marriage', 'social climber', 'gambling debt', 'off screen murder', 'cinderella story', 'correspondence', 'evil step mother', 'family secret', 'tour bus', 'bad cop', 'school nurse', 'arachnophobia', 'crystal ball', 'outrage', 'walking through a wall', '1860s', 'schoolteacher', 'siam', 'jewish family', 'based on anime', 'speed', 'irish accent', 'torn between two worlds', 'artificial organ', 'enforcer', 'hippie', 'music festival', 'the who', 'federal agent', 'year 1997', 'divorced parents', 'exploding car', 'black panties', 'perfect man', 'secret admirer', 'lex luthor', 'lois lane', 'return to earth', 'capoeira', 'freeze', 'queen of sheba', 'butterfly', 'entomologist', 'morpho butterfly', 'indian army', 'hot temper', '1955 chevrolet', 'peeping tom', 'reference to brooklyn dodgers', 'spying', 'tire iron', 'parenthesis in title', 'british military', 'ottoman empire', 't.e. lawrence', 'queen of england', 'tall man', 'princeton university', 'working women', 'finger gun', 'killed in an elevator', 'december', '35 mm digital camera', 'anthropophagus', 'blood and gore', 'double feature', 'real twins playing twins', 'turkey', 'tunisia', 'debunking', 'driving through a gate', 'trade union', 'flash of genius', 'mental instability', 'sudden infant death syndrome', 'first date', 'harlem manhattan new york city', 'hello kitty', 'comic book artist', 'genie', 'reference to a thousand and one nights', 'carjacking', 'johannesburg south africa', 'post apartheid', 'singer songwriter', 'male anorexia', 'gremlin', 'horror host', 'hulk hogan', 'brothers hugging', 'parole violation', 'sex in a car', 'sex video', 'sexual desire', '2030s', 'jet', 'grace kelly', 'hollywood star', 'childhood trauma', 'dog show', 'gay character', 'gay couple', 'news leak', 'whistleblowing', 'wikileaks', 'eastern europe', 'ghost rider', 'loudmouth', 'money laundering', "shakespeare's a midsummer night's dream", 'kremlin', 'jumping on a bed', 'librarian', 'pregnant woman in bathtub', 'marshal', 'future war', 'shootout in a police station', 'sabbath', 'ten stories', 'mediterranean', 'leatherface', 'coast guard', 'oil tanker', 'sinking ship', 'forbidden sex', 'sex with a student', 'taboo sex', 'columnist', 'los angeles times', 'wearing a traffic cone as a hat', 'efficiency expert', 'insurance investigator', 'sheffield', 'yorkshire', 'pop star', 'head dunked in water', 'military school', 'plastic bag', 'playing a video game', 'suffocating', 'gay sibling', 'stalingrad', 'american revolution', 'standoff', 'female basketball player', 'personal vendetta', 'reference to ralph nader', 'live television', 'burlesque', 'small town girl', 'italian stereotype', 'man in a swimsuit', 'verona italy', 'university of oregon', 'basketball game', 'monstars', 'college roommate', 'arch villain', 'mexican immigrant', 'network', 'pine forest', 'team building', 'maryland', 'film student', 'street dancing', 'girlfriend in a coma', 'prosthetic leg', 'pawn', 'mysterious island', 'art collector', 'bull riding', 'actor name in title', 'anal sex', 'claw', 'inspector', 'order', 'beekeeping', 'insect in title', 'cleaning a swimming pool', 'male male kiss', 'talking dog', 'game tester', 'snack', 'private eye', 'half human', 'rebellious daughter', 'pipe bomb', 'gangsta rap', 'nwa', 'animator', 'bamboo cane', 'beekeeper', 'false memory', 'implanted memory', 'space colony', 'earl of oxford', 'lost in the woods', 'screaming in horror', 'meddling', 'sobbing female', 'subtitled scene', 'you can say that again joke', 'narcolepsy', 'traveling', 'train robbery', 'answering machine', 'home movie', 'lithium', 'video diary', 'argentine', 'haunted hospital', 'drill in the eye', 'eye gouging', 'hanged woman', 'peep hole', 'based on a true story', 'accidental kiss', 'eating a worm', 'worm', 'close encounter of the third kind', 'child hitman', 'self repair', 'envy', 'homosexual sex', 'male homosexuality', 'noisy neighbor', 'al qaeda', 'bomb making', 'saudi arabia', 'stabbed in the crotch', 'snowglobe', 'series', 'tv series', 'discipline', 'imperialism', 'white slavery', 'breakdown', 'mother in law', 'errand', 'wrong number', 'michelin star', 'violent behavior', 'indian restaurant', 'network marketing', "actor's picture shown in credits", 'animated credits', 'google', 'still images in end credits', 'hansel and gretel', 'same actor playing two characters simultaneously on screen', 'year 1996', 'contamination', 'corporate crime', 'environmental issue', 'working single mother', 'sheriff of nottingham', 'document', 'jurassic', 'inside joke', 'one man show', 'employee', 'laser in eyes', 'tina turner', 'secretary general', 'united nations', '27th century', 'collapsing tunnel', 'weimaraner', 'summertime', 'treehouse', 'walking on train tracks', 'charles ii', 'earl', 'automobile association of america', 'reference to tom jones the singer', 'resistance movement', 'broke', 'promoter', 'trainer', 'soul mate', 'book club', 'cheating boyfriend', 'job promotion', 'planet earth', 'woman in bra and panties', 'interior designer', 'closed ecological system', 'perpetual winter', 'population control', 'witness protection program', 'crying man', "men's clothing store", 'giant vegetable', 'pest control', 'bedroom', 'hug', 'interrupted sex', 'trumpet', 'police corruption', 'cover band', 'tribute band', 'youtube', 'food in title', 'potato', 'car damage', 'computer printer', 'female band', 'fingernail polish', 'obsessive love', 'bread', 'coarse language', 'tricked into lesbian sex', 'chumscrubber', 'glass of water', 'loss of faith', 'girl fight', 'sailing boat', 'tent', 'caught in a net', 'custody', 'fashion photographer', 'bra', 'small village', 'androgynous', 'psychological drama', 'erasmus', 'lebanese', 'loss', 'thesis', 'hitchhiking', 'modern western', 'cayman islands', 'retired actress', '22 year old', 'b.a.r.t.', 'bay area', 'scottish', 'ugandan', '22nd century', 'lyricist', 'drunk wife', 'fbi investigation', 'fire investigator', 'turpentine', 'bravery', 'medal of honor', 'military life', 'unreliable flashback', 'hanged man', 'mysterious past', 'reference to mr. rogers', 'reference to vincent van gogh', 'talking about suicide', 'animated sequence', 'judicial system', 'private investigation', 'suspected murder', 'feminist', 'visit', 'romulan', 'puppet animation', 'red hat', 'poker player', 'reference to frederic chopin', 'reference to johannes brahms', 'slaughterhouse', 'ferry', 'first daughter', 'lobby boy', 'mentor protege relationship', 'dude', 'ensemble cast', 'episodic structure', 'motel room key', 'fat', 'health', 'juice', 'obese', 'german immigrant', 'war orphan', 'grand canyon', 'casualty notification', 'casualty notification team', 'staff sergeant', 'arrogance', 'kennel', 'refrence to liza minnelli', 'quantum mechanics', 'runaway', 'diver', 'diving', 'flare', 'belgium', 'author cameo', 'two part tv movie', 'staff', 'liberty', 'space adventure', 'undercover agent', 'deathbed', 'third part', 'nigerian', 'born in vitro', 'in vitro fertilization', 'siblings', 'year 2065', 'duck', 'stuntman', 'beatnik', 'necrophilia', 'partying', 'lunch wagon', 'back from the dead', 'paranormal research', 'shaky cam', 'mongolia', 'spear', 'speared to death', 'circus vargas', 'body', 'white blood cell', 'missing file', 'staged rape', 'deforestation', 'moose', 'voice', 'female in a shower', 'woman wearing a miniskirt', 'u.s. congressman', 'immorality', 'ukrainian', 'reference to winnie mandela', 'adventurer', 'el dorado', 'high priest', 'cultural revolution', 'baptist church', 'double murder', 'nuclear explosion', 'nuclear weapon', 'nazi germany', 'track and field', 'lds film', 'mormonism', 'gravestone', 'pipeworks', 'friday', 'techno rave', 'death of animal', 'high heels', 'kneed in the crotch', 'communist dictatorship', 'exploding helicopter', 'finger bitten off', 'apology', 'brother in law sister in law relationship', 'expectant father', 'histrionic personality disorder', 'middle aged woman', 'off screen rape', 'high school romance', 'fatal attraction', 'interspecies romance', 'ragtime', 'prosthetics', 'tail', 'manmade earthquake', 'tunnel boring machine', 'christ figure', 'messiah', 'saviour', 'social commentary', 'leo tolstoy', 'construction site', 'man without a country', 'meal', 'overeating', 'male in bathtub', 'seductive fembot', 'beer pong', 'mischief', 'no dialog', 'sign language', 'player', 'tennis player', 'wimbledon', 'corrupt judge', 'tax evasion', 'white suit', 'year 1930', 'aids', 'homophobia', '40th birthday', 'reference to c. thomas howell', 'cards', 'tarot card', 'human race', 'religious faith', 'child murderer', 'art forgery', 'picture framer', 'substance d', 'conversations', 'record store', 'thirty something', 'christmas season', 'release from prison', 'last man standing', 'hiding in a closet', 'cyclops', 'fortress', 'martinique', 'banishment', 'sorcery', 'storm at sea', 'romantic friendship', 'john wilkes booth', 'lincoln assassination', 'vegan', 'feces in a swimming pool', 'female vomiting', 'steel helmet', 'u.s. marine', 'neighbor neighbor relationship', 'remote control car', 'andy kaufman', 'actress name in title', 'bushwhacker', 'based on stage musical based on stage play', '20th century', 'association', 'fifa', 'movie business', 'death row inmate', 'southern u.s.', 'diminution', 'physical flexability', 'ticket', 'at work', 'vermin', 'high school prom', 'prom king', 'shanghai', 'sex in an airplane', 'connection', 'african national congress', 'film starts with text', '12 step program', 'looting', 'nightclub singer', 'breakup', 'outrunning explosion', 'one armed man', 'equal pay', 'shakespeare in modern dress', "shakespeare's coriolanus", 'kite', 'hook and chain', 'ghost hunting', 'cosmetics', 'eiffel tower', 'ritual', 'stew', 'ballroom dancing', 'poise', 'self confidence', 'may december romance', 'rockefeller center', 'surveillance camera', 'war on terror', 'harmonium', 'pudding', 'bilingual', 'three women', 'luberon', 'mystery writer', 'i.d.', 'totalitarian', 'hot shot', 'stock car driver', 'mad max', 'bullet time', 'knife throwing', 'throwing a knife', 'ballroom', 'dance teacher', 'salford', 'armpit fetish', 'electric torture', 'japanese american', 'tree house', 'rapture', 'heart ripped out', 'reference to michael jackson', 'first person title', 'reference to louis de funes', 'amish', 'short order cook', 'clock tower', 'delorean', 'news broadcast', 'tv broadcast', 'dead teenager', 'explosives expert', 'snorting', 'postmodern', 'saint', 'spanish civil war', 'starts with narration', 'alternative reality', 'multiple outcomes', 'twist of fate', 'shot gun', 'bechdel test failed', 'repeated event', 'dwegons', 'judicial', 'judiciary', 'table broken in fight', 'baseball team', 'actor director writer', 'tramp', 'essex', 'little sister', 'pallbearer', 'nail gun', 'catastrophe', 'puerto rican', 'cancun mexico', 'lti txii', 'peugeot 206', 'rover 3 5 litre.coupe p5b', 'skeptic', 'tailing', 'close up of eyes', 'quirkiness', 'comic', 'jay and silent bob', 'plastic surgery', 'pathology', 'bigotry', 'abu dhabi', 'digimon', 'extradition', 'poland', 'miscarriage', 'pact', 'colombian drug cartel', 'reference to ben affleck', 'reference to brad pitt', 'reference to george clooney', 'reference to jack nicholson', 'gramophone', 'missing children', 'flower girl', 'antique shop', 'asian dragon', 'korean', 'falling down an elevator shaft', 'oleander', 'faun', 'oven', 'stove', 'thanksgiving dinner', 'human dragon relationship', 'dark force', 'entity', 'evil force', 'paranormal phenomenon', 'adaptation directed by original author', 'thor', 'dairy farm', 'rustler', 'blackheart', 'mephistopheles', 'protest march', 'literary agent', 'chained', 'cellphone', 'martial arts school', 'wing chun', 'concert film', 'emcee', 'mohawk', 'scent', 'sense of smell', 'smell', 'bottle', 'jogging', 'seeking love', 'single guy', 'tv host', 'women rivals for man', 'hiv positive', 'marshall university', 'female fighter', 'idler', 'poorhouse', 'delusional depression', 'mysterious girl', 'cigarette lighter', 'armed forces', 'political corruption', 'shady', 'war profiteer', 'ghostface', 'east africa', 'hunter gatherer', 'indigenous', 'rift valley', 'swahili', 'bb gun', 'tongue stuck on a pole', 'curtain', 'xeroderma pigmentosum', 'collision course', 'ship run aground', 'narc', 'film director', 'rogue', 'awkward boy', 'shy kid', 'water park', 'talking to self in mirror', 'narrated by a girl', 'younger version of character', 'teenage abduction', 'art class', 'nerve gas', 'video message', 'navy seal', 'popcorn', 'outnumbered', 'bailiff', 'nazi hunter', 'paraguay', 'space shuttle transported on a 747', 'camelot', 'excalibur', 'battle of gallipoli', 'distant past', 'nobleman', 'trapped in the past', 'african american stereotype', 'rent', 'threatening telephone call', 'wine tasting', 'belief', 'passive aggression', 'shot through the head', 'kung fu master', 'crossdresser', 'long tracking shot', 'psychology', 'death squad', 'mass killing', 'musical number', 'reference to guantanamo bay', 'bobble head doll', 'rejuvenation', 'edinburgh scotland', 'closeted lesbian', 'islamophobia', 'propaganda', 'feather', 'bloody water', 'piranha', 'female bounty hunter', 'child fugitive', 'martial art', 'silk road', 'mineral', 'hoodoo', 'amateur radio', 'ham radio', 'based on cult tv series', 'exercise machine', 'midnight movie', 'year 2015', 'bad day', 'kosher', 'vhs', 'king arthur', 'lancelot', 'bomb squad', 'pushed out a window', 'smothered with a pillow', 'telescope', 'gargoyle', 'curb stomping', "son dislikes mother's boyfriend", 'white supremacist', 'first of series', 'bruges', 'alternative comedy', 'buddy comedy', 'gun smuggling', 'anthropomorphism', 'tomato', 'ambulance driver', 'magazine editor', 'iraqi', 'political military conspiracy', 'weapon of mass destruction', 'flying through a thunderstorm', 'sex in a pool', 'knocked out', 'online game', 'fixer', 'valor', 'listener', 'radio talk show', 'in love with an inanimate object', 'sex doll', 'appearing from water', 'college tuition', 'state fair', 'breaking the fourth wall by talking to the audience', 'singapore', 'garden', 'unity', 'female reporter', 'san francisco giants']
  arv=len(keywords)
  keycolumns={'movie_title':[]}
  for index, movie in processedDataFrame.iterrows():
    movieKeywords=[0]*arv;
    currentKeyword=""
    keycolumns['movie_title'].append(movie['movie_title'])
    for i in keywords:
      j=0
      currentKeyword=i
      keycolumns.update({currentKeyword:[0]*processedDataFrame.shape[0]})
      onvei=0
      if(type(movie['plot_keywords'])==float):
        continue;
      elif i in movie['plot_keywords'].split("|"):
        onvei=1
        movieKeywords[j]=1
        keycolumns[currentKeyword][j]=onvei
      j+=1
  df2=pd.DataFrame(keycolumns)
  processedDataFrame=processedDataFrame.merge(df2, on='movie_title', how='inner')
  processedDataFrame = processedDataFrame.drop('plot_keywords', axis=1)
  processedDataFrame = processedDataFrame.drop('movie_title', axis=1)
  #getting rid of nan values
  ratings = {'TV-Y': 0,'TV-Y7': 1,'G': 2,'TV-G': 3,'Approved': 4,'PG': 5,'TV-PG': 6,'GP': 7,'PG-13': 8,'TV-14': 9,'M': 10,'R': 11,'NC-17': 12,'X': 13,'Not Rated': 14,'Passed': 15,'Unrated': 16,'TV-MA': 17}
  try:
    processedDataFrame['content_rating']=processedDataFrame['content_rating'].map(ratings) #ratingud nooremast vanemaks
  except:
    print('no such column')
  for i in processedDataFrame.columns:
    if(i=='content_rating'):
      continue;
    mean_value = processedDataFrame[i].mean()
    processedDataFrame[i].fillna(value=mean_value, inplace=True)
  try:
    processedDataFrame['content_rating'].fillna(value=5,inplace=True)
  except:
    print('no column content rating')
  return processedDataFrame, andmestikud

### 3.2. Treeni mudel

In [9]:
from sklearn.linear_model import LinearRegression

In [15]:
def trainScore(filmiDataFrame):
    '''Õpib sisendandmete põhjal mudeli, mis oskab ennustada IMDB skoori.
       Võib kasutada väliseid funktsioone ja muud endale sobivat.
       Sisendiks saab treeningandmete DataFrame'i.
       Tagastab treenitud mudeli.'''

    treeningandmed, andmestikud = preprocessScore(filmiDataFrame)
    try:
      X=treeningandmed.drop('imdb_score', axis=1)
    except:
      print('tere')
      X=treeningandmed
    test=[]
    for i in X['content_rating']:
      if i not in test:
        test.append(i)
    print(test)
    y=treeningandmed['imdb_score']
    mudel = LinearRegression()
    mudel.fit(X,y)
    ########
    # ToDo
    ########
    return mudel, andmestikud

### 3.3. Ennustamine

Meeldetuletus: andmete eeltöötlus tuleb teha ka testhulgale. Selleks võib kasutada abifunktsioone. Siia võib lisada abiplokke.

In [11]:
def predictScore(mudel,andmestikud, testhulk):
    '''Saab sisendiks treenitud mudeli, eeltöötluseks vajalikud andmed
       ja terve testhulga algsel kujul (ilma skoorita ja eeltöötluseta).
       Tagastab ennustatud skooride järjendi.'''
    # IMDB skoorid vahemikus 0-10
    test_andmed=testhulk
    skoorid = []
    test_andmed = preprocessScore(testhulk)
    X_test=test_andmed[0]
    skoorid = mudel.predict(X_test)
    ########
    # ToDo
    ########
    return skoorid

### 3.4. Testimine

Siin on etteantud testplokk, mis näitab, kuidas see kõik koos toimida võiks. Oluline on see, et see kõik oleks kasutatav erinevatel andmehulkadel, näiteks varem nägemata failil. Jah, muidugi võib seda töö käigus muuta - kontrollimisel eeldatakse aga, et see algne testplokk ka töötab.

Üldiselt on alguses mõttekas väiksema andmehulgaga katsetada, kui süntaksi- ja muud vead on välja roogitud, võib kogu andmehulga ette võtta.  

**NB!** Ärge eeldage, et *testandmed* väärtustamisel saab midagi juba töödeldut. Testandmete eeltöötlus tuleks ära teha ennustusmeetodi raames või mõne abifunktsiooniga enne seda. Tavaelus tulevad testandmed teile uue ja iseseisvana ette.

In [16]:
filmid2 = pd.read_csv(faili_asukoht, delimiter=",",index_col=0)

# Eeldades, et 'filmid2' on äsja failist sisseloetud, 28-tulbaline ja muidu muutmata kujul DataFrame
mudel, andmestikud = trainScore(filmid2[100:])

# Võta testimiseks esimesed 10 kirjet, ilma ennustatava väärtuseta
# Kodutöö hindamisel loetakse testandmed eraldi failist, treenimiseks kasutatakse kogu treeninghulka
# Eeldades, et 'filmid2' on äsja failist sisseloetud, 28-tulbaline ja muidu muutmata kujul DataFrame
filmid2 = pd.read_csv(faili_asukoht, delimiter=",",index_col=0)
testandmed = filmid2[:100]

# Tunnused, mille põhjal ennustada
testandmed_x = testandmed.drop('imdb_score',axis=1)
# Tunnus, mida ennustada
testandmed_y = testandmed['imdb_score']

# Ennustatud filmiskoorid
ennustused = []
ennustused = predictScore(mudel,andmestikud,testandmed_x)

# Arvuta ennustuste ja õigete vastuste põhjal MSE
print("MSE:", mean_squared_error(ennustused, testandmed_y))

4438
[5.0, 2.0, 11.0, 4.0, 8.0, 3.0, 9.0, 16.0, 6.0, 14.0, 17.0, 13.0, 12.0, 7.0, 0.0, 15.0, 1.0, 10.0]
100
Index(['color_x', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'actor_1_name',
       ...
       'sex doll', 'appearing from water', 'college tuition', 'state fair',
       'breaking the fourth wall by talking to the audience', 'singapore',
       'garden', 'unity', 'female reporter', 'san francisco giants'],
      dtype='object', length=7691)
MSE: 0.8572004260674609


## 4. Mudeli kirjeldus

**1 punkti** saab enda mudelite (sh andmete valiku ja eeltöötlusotsuste) kirjeldamise eest. Mis sorti eeltöötluse tegid ja miks? Mille alusel valisid lõpliku mudeli?

Tegin esimese ja teise osa puhul sarnast andmete töötlust, muutes genre ja plot_keywords veerud onehot vektoriteks, kuna muidu oleks nende töötlemine väga raske. Kaotasin nan väärtused erinevates tulpades. Asendades need keskmiste tulemustega. Plot_keywords veerus kasutasin ainult sõnu mis rohkem kui 5 korda esines. Kasutasin esimeses knn meetodit masinõppeks, kuna see on rohkem klassifitseerimiseks loodud kui LinearRegression, oleks võinud ka kasutada decision tree-d aga ei jõudnud seekord. Teises ülessandes sobib Linear regression väga hästi kuna vaja on ennustada ujukoma arvu

....

## 5. Tagasiside

Siia võib järgi anda tagasiside kodutööle. Tagasiside on vabatahtlik ja sellega aitate maailma (täpsemalt tehisintellekti ainet) paremaks muuta.  
Kas 1. kodutöö oli kohutavalt lihtne, meeletult keeruline, teistmoodi huvitav või lihtsalt igav? Kas miski tekitas segadust või oli liiga detailselt ette antud? Mis osa kodutööst oli kõige raskem? Mitu tundi see umbes aega võttis ja kas midagi tuleks järgmisel korral kindlasti teisiti teha?

Kulus üle 10 tunni kindlasti, kuid paljud asjad ei olnud meeles ja vajasid turgutamist, sain midagi kätte lõpuks, aga kasuks oleks võibolla mingi meeldetuletus mis mudelid milliseid andmeid taluvad. Üldiselt oli põnev aga ma oleks pidanud varem alustama tegemisega.

....